In [1]:
import pandas as pd
import numpy as np
import openpyxl
from upper_case_dict import D_dict
from upper_case_dict import D_describe
import traduce as tr
pd.options.display.max_columns=None
pd.options.display.max_colwidth=300

Los **diccionarios de las variables originales** en la encuesta se importan del **módulo upper_case_dict**:  

- **D_dict**; diccionarios de traducción de las variables categóricas. Construidos de acuerdo con los códigos facilitados en el archivo de diseño de la encuesta 'dr_EILU_GRAD_2019.xlsx'. Para cada variable: D_dict[variable][código_categórico]=interpretación_código. 
- **D_describe**; diccionario de interpretación de las variables: D_describe[variable]=interpretación_variable.  

El proceso de construcción de estos diccionarios se detalla en el fichero upper_case_dict.py.

En el proceso de inspección se redefinen algunas variables originales y se definen otras nuevas. Estas se añaden a los diccionarios D_dict y D_describe para ser recuperados con el módulo lower_case_dict. 

Se importa el **módulo traduce** y se transforman sus **funciones traduce() y leyenda()** para simplificar sus argumentos: 

In [2]:
def traduce(col,check=True,save=True):
    return tr.traduce(df=df,dicc=D_dict,LO_=v_originales_L,LM=v_modelos,LM_=v_modelos_L,
                      col=col,check=check,save=save)
def leyenda(col):
    return tr.leyenda(D_dict,col)

# Loading the data
Dataset 'Encuesta_insercion_laboral_titulados_universitarios_2019' en formato csv.   

Enlace a fuente (INE): https://www.ine.es/dyngs/INEbase/es/operacion.htm?c=Estadistica_C&cid=1254736176991&menu=resultados&idp=1254735573113#!tabs-1254736195339

In [3]:
ruta_dataset='data/EILU_GRAD_2019.csv'

In [4]:
df_raw=pd.read_csv(ruta_dataset,sep='\t')

/home/bea/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (119) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
df=df_raw.copy()

In [6]:
df.shape

(31651, 299)

In [7]:
for n,col in enumerate(df.columns):
    print(n,col,df[col].dtype)

0 IDENT int64
1 SEXO int64
2 EDAD int64
3 NACIO int64
4 NACIO1 object
5 NACIO2 object
6 PAIS_NACI int64
7 TITU int64
8 RAMA int64
9 AMBITO int64
10 T_UNIV int64
11 DISCA int64
12 PROXY int64
13 LUG_RES_PAIS int64
14 PAIS_NAC_PADRE int64
15 ESTUDIOS_PADRE int64
16 PAIS_NAC_MADRE int64
17 ESTUDIOS_MADRE int64
18 EST_B1 int64
19 EST_B2_1 object
20 EST_B2_2 object
21 EST_B2_3 object
22 EST_B2_4 object
23 EST_B2_5 object
24 EST_M1 int64
25 EST_M2 object
26 EST_M3 object
27 EST_M4 object
28 EST_MES int64
29 EST_B8 int64
30 SAT1 int64
31 SAT2 object
32 EST_B11_1 int64
33 EST_B11_2 int64
34 EST_B11_3 int64
35 EST_B11_4 int64
36 EST_B11_5 int64
37 EST_B11_6 int64
38 EST_B11_7 int64
39 EST_B12 object
40 EST_B13_AMB1 object
41 EST_B14_1 object
42 EST_B15_1 object
43 EST_B13_AMB2 object
44 EST_B14_2 object
45 EST_B15_2 object
46 EST_B13_AMB3 object
47 EST_B14_3 object
48 EST_B15_3 object
49 EST_B16 object
50 EST_B17_M1 object
51 EST_B18_1 object
52 EST_B19_1 object
53 EST_B17_M2 object
54 EST_B18_

# Análisis exploratorio: funciones 

Funciones definidas para facilitar la inspección de variables categóricas: distribución de frecuencias relativas, distribuciones condicionadas, etc.    

In [8]:
def proporciones(col,dropna=False):
    """ Frecuencias relativas de los valores que toma una variable categórica en orden ascendente.
    Si dropna=False, incluye los valores NaN."""
    out=df[col].value_counts(normalize=True,ascending=False,dropna=dropna)
    return out

In [9]:
def dist_condicionada(col,cond,rev=False,dropna=False):
    """ Distribución de la columna col condicionada por la columna cond. Si rev=True, distribución de 
    cond condicionada por col.
    Si dropna=True, prescinde de registros NaN."""
    if rev==True: 
        temp=col
        col=cond
        cond=temp
    out=df[[cond,col]].groupby([cond,col],dropna=dropna).agg({col:'count'})\
                .div(df[[cond,col]].groupby(cond,dropna=dropna).agg({col:'count'}),level=cond)
    out['dif respecto distrib '+str(col)]=out[col].sub(proporciones(col,dropna=dropna),level=col)
    out['dif relativa']=out['dif respecto distrib '+str(col)].div(proporciones(col,dropna=dropna),level=col)

    return out.style.format({'dif respecto distrib '+str(col):'{:+f}',\
                             'dif relativa':'{:+,.2%}'})

# Inspección y ajustes

**Objetivo**: Inspección del dataset, imputación de valores perdidos debido a los itinerarios de la encuesta y creación de variables nuevas para análisis descriptivo y predictivo.   

De esta etapa del proceso se obtienen **dos datasets**:    

1- Un dataset con las **variables originales** (designadas con nombres en mayúsculas) a las que, a lo sumo, se aplica alguna de las transformaciones siguientes:  
- Imputación de valores perdidos que pueden deducirse de los itinerarios de la encuesta.  
- Sustitución de las no respuestas y las respuestas NS/NC por el valor None.  

2- Un **dataset nuevo** formado por algunas de las variables originales escogidas como candidatas a formar parte de los modelos descriptivos o predictivos y por algunas variables nuevas creadas a partir de las originales. Las variables de este dataset se designan por nombres en minúsculas para facilitar su identificación. Algunas de las transformaciones que dan lugar a este conjunto de variables son:  
- Sustitución del valor asociado a la respuesta No en las preguntas de respuesta Sí o No: se cambia el valor 2 asociado al No en el dataset original por el valor 0.  
- Creación de variables binarias para cada una de las categorías de algunas de las variables cualitativas del dataset original.  
- Creación de variables nuevas para información que puede deducirse de una o varias de las columnas del dataset original.  
- Creación de variables con información del dataset (año de la encuesta, población objeto de muestreo, etc.) con el fin de que sea posible identificar la procedencia de cada observación del el dataset que resulte de reunir todos los empleados en este estudio.  

En esta etapa **no se imputan valores a las no respuestas o a las respuestas NS/NC** que no puedan deducirse de los itinerarios de la encuesta. Solo se sustituyen por el valor None. Se procede así para evitar que los valores imputados interfieran en los resultados del análisis descriptivo. 

In [10]:
# Lista de variables originales. Permite recuperar el dataset original después de imputar los valores perdidos ...
# ... como consecuencia de los itinerarios de la encuesta. 
v_originales=df.columns.tolist()
# Lista a la que se agregarán las variables candidatas a ser incluidas en el modelo predictivo
v_modelos=list()
# Listas de variables literales (interpretación de códigos de variables categóricas)
v_originales_L=list()
v_modelos_L=list()

In [11]:
def no_respuesta(col,blank=' ',nsnc='9',tipo=None):
    """ Sustituye respuestas en blanco y NS/NC por None y establece tipo 'float' si es posible'.
    """
    if tipo=='int':
        blank=None
        nsnc=int(nsnc)
    try: df[col].replace({blank:None,nsnc:None},inplace=True)
    except: 
        serie=df[col].astype('O') # Evita conflictos cuando se aplica más de una vez
        serie.replace({blank:None,str(nsnc):None},inplace=True)
    try:df[col]=df[col].astype('float')
    except:pass # Algunas variables categóricas tienen valores no numéricos (e.g. CNAE)
    try:traduce(col,check=False)
    except:pass # Para variables no categóricas (sin diccionario)
    print (df[col].sort_values().unique())
    return

In [12]:
def redefine(col,nuevo=True,cat=True,si_no=False):
    """ Crea una columna denominada 'nuevo' (por defecto, el nombre original en minúsculas) a partir 
    de la columna 'col', la añade a la lista de variables para los modelos descriptivos y predictivos. 
    Si 'cat' True, asocia la variable a su diccionario, traduce y conserva la variable traducida.
    Si si_no=True, modifica el valor asociado a respuesta No (0 en lugar de 2).  
    """
    if nuevo==True:
        nuevo=col.lower()
    df[nuevo]=df[col]
    D_describe[nuevo]=D_describe[col]
    if str(nuevo) not in v_modelos: 
        v_modelos.append(nuevo)
    if cat==True: 
        D_dict[nuevo]=D_dict[col]
        traduce(nuevo,check=False)
        if str(nuevo)+'_' not in v_modelos_L: 
            v_modelos_L.append(str(nuevo)+'_')
    if si_no==True: 
        df[nuevo].replace({2:0},inplace=True)
        D_dict[nuevo]=D_dict['si_no']
        traduce(nuevo)
    return

In [13]:
def a_lista(col):
    if col==col.lower() and col==col.strip('_') and str(col) not in v_modelos:
        v_modelos.append(col)
    return

In [14]:
def print_listas():
    print ('v_originales_L:','\t',v_originales_L)
    print ('v_modelos:','\t',v_modelos,'\nv_modelos_L:','\t',v_modelos_L)
    return

In [15]:
def reset(col_or_list):
    """ Argumentos: columna o lista de columnas.
    Elimina la variable col y su literal col_ de las listas de variables redefinidas o literales.
    Si col es una variable en el dataset original, reestablece sus valores iniciales. 
    Si el argumento es una lista de columnas, aplica reset a cada columna de la lista.
    """
    global v_originales_L, v_modelos, v_modelos_L
    if type(col_or_list)!=list:
        L=[col_or_list]
    else: 
        L=col_or_list
    for col in L:
        if col==col.upper() and str(col)+'_' in v_originales_L:
            v_originales_L.remove(str(col)+'_')
        if col==col.lower() and str(col) in v_modelos:
            v_modelos.remove(str(col))
        if col==col.lower() and str(col)+'_' in LM_:
            v_modelos_L.remove(str(col)+'_')
        try:
            df[col]=df_raw[col]
            del df[str(col)+'_']
        except:pass
    return

# BLOQUE A. DATOS PERSONALES Y SOCIODEMOGRÁFICOS

## 'IDENT' 
**Índice** único del entrevistado.

In [16]:
df['IDENT'].nunique()

31651

## Nueva variable: 'año' 
**Año** en el que se realizó la encuesta.  
Se añade para que las observaciones de este dataset sean identificables cuando se reunan varios datasets.

In [17]:
df['año']=2019 
df['año'].unique() # Comprobación

array([2019])

In [18]:
a_lista('año')
D_describe['año']='Año en que se realizó la encuesta'

## Nueva variable: 'est_encuesta' 
**Estudios** oficiales objeto de la encuesta (los que definen la población objeto de muestreo).   
Se añade para que las observaciones de este dataset sean identificables cuando se reunan varios datasets.

In [19]:
df['est_encuesta']='graduado_u' # Título de graduado universitario, licenciado, diplomado o equivalente. 
df['est_encuesta'].unique() # Comprobación

array(['graduado_u'], dtype=object)

In [20]:
a_lista('est_encuesta')
D_describe['est_encuesta']='Estudios oficiales objeto de la encuesta (los que definen la población objeto \
de muestreo)'

## 'SEXO' (redefinida como 'fem')
**Sexo** del entrevistado.  

**'fem'**: 1= Mujer, 0=Hombre. 

In [21]:
df['SEXO'].unique() # No missing values

array([1, 2])

In [22]:
traduce('SEXO')

     aux_traduce
          unique
SEXO            
1       [Hombre]
2        [Mujer]


In [23]:
df['fem']=0
df['fem'].mask(df['SEXO']==2,1,inplace=True)
df[['SEXO','fem']].groupby('fem').agg(['unique']) # Comprobación

,SEXO
,unique
fem,
0,[1]
1,[2]


In [24]:
D_describe['fem']='Mujer u hombre'
D_dict['fem']={1:'Mujer',0:'Hombre'}
traduce('fem')

    aux_traduce
         unique
fem            
0      [Hombre]
1       [Mujer]


In [25]:
proporciones('fem_')

Mujer     0.570472
Hombre    0.429528
Name: fem_, dtype: float64

## 'EDAD' (redefinida como 'edad')
**Edad** del entrevistado.

In [26]:
D_describe['EDAD']

'Grupo de edad (a 31-dic-2019)'

In [27]:
D_describe['EDAD']='Grupo de edad en el momento de la encuesta'

In [28]:
df['EDAD'].unique() # No missing values

array([1, 2, 3])

In [29]:
traduce('EDAD')

               aux_traduce
                    unique
EDAD                      
1     [Menores de 30 años]
2        [De 30 a 34 años]
3       [De 35 y más años]


In [30]:
redefine('EDAD')

In [31]:
proporciones('edad_')

Menores de 30 años    0.496256
De 30 a 34 años       0.278822
De 35 y más años      0.224922
Name: edad_, dtype: float64

## 'NACIO'
**Nacionalidad** del entrevistado.

In [32]:
df['NACIO'].unique() # No missing values

array([1, 2, 3])

In [33]:
traduce('NACIO')

               aux_traduce
                    unique
NACIO                     
1               [Española]
2        [Española y otra]
3      [Otra nacionalidad]


In [34]:
proporciones('NACIO_')

Española             0.978137
Otra nacionalidad    0.011027
Española y otra      0.010837
Name: NACIO_, dtype: float64

## Nueva variable: 'esp'
**Nacionalidad española** u otra: 0=Nacionalidad no española, 1=Nacionalidad española. 

In [35]:
df['esp']=1
df['esp'].mask(df['NACIO']==3,0,inplace=True)
df[['esp','NACIO']].groupby(['NACIO']).agg(['unique']) # Comprobación

,esp
,unique
NACIO,
1,[1]
2,[1]
3,[0]


In [36]:
D_describe['esp']='Nacionalidad española u otra'
D_dict['esp']={0:'Nacionalidad no española',1:'Nacionalidad española'}
traduce('esp')

                    aux_traduce
                         unique
esp                            
0    [Nacionalidad no española]
1       [Nacionalidad española]


In [37]:
proporciones('esp_')

Nacionalidad española       0.988973
Nacionalidad no española    0.011027
Name: esp_, dtype: float64

In [38]:
# Distribución condicionada de sexos condicionada por nacionalidad española u otra: 
dist_condicionada('fem_','esp_')
# La mayor presencia femenina entre los graduados se acentúa en el caso quienes no tienen nacionalidad española

## 'NACIO1'

(Solo a quienes declaran nacionalidad distinta de 'Española' -respuesta distinta a 1 en 'NACIO'-)  

**País de primera nacionalidad**.

In [39]:
df['NACIO1'].unique() # Missing values as ' ' or '9'

array([' ', '2', '9', '1'], dtype=object)

In [40]:
traduce('NACIO1')

                     aux_traduce
                          unique
NACIO1                          
                  [No aplicable]
1             [Un país de la UE]
2       [Un país fuera de la UE]
9                        [NS/NC]


In [41]:
df[['NACIO_','NACIO1_']].groupby('NACIO_',dropna=False).agg(['unique'])
# Respuesta en blanco para quienes declararon solo nacionalidad española en 'NACIO'

,NACIO1_
,unique
NACIO_,
Española,[No aplicable]
Española y otra,"[Un país fuera de la UE, Un país de la UE, NS/NC]"
Otra nacionalidad,"[NS/NC, Un país fuera de la UE, Un país de la UE]"


In [42]:
# Se imputa None a respuestas en blanco y NS/NC
no_respuesta('NACIO1')

[ 1.  2. nan]


In [43]:
# Se imputa nacionalidad de un país de la UE (1) a quien declaró solo nacionalidad española en variable 'NACIO'
df['NACIO1'].mask(df['NACIO']==1,1,inplace=True)
traduce('NACIO1',check=False)
df[['NACIO_','NACIO1_']].groupby('NACIO_').agg(['unique']) # Comprobación

,NACIO1_
,unique
NACIO_,
Española,[Un país de la UE]
Española y otra,"[Un país fuera de la UE, Un país de la UE, nan]"
Otra nacionalidad,"[nan, Un país fuera de la UE, Un país de la UE]"


In [44]:
proporciones('NACIO1_')

Un país de la UE          0.985751
Un país fuera de la UE    0.012511
NaN                       0.001738
Name: NACIO1_, dtype: float64

## 'NACIO2'

(Solo a algunos de los que declaran 'Otra nacionalidad' -respuesta 3 en variable 'NACIO')  

**País de segunda nacionalidad**.

In [45]:
df['NACIO2'].unique() # Missing values as ' '

array([' ', '1', '2'], dtype=object)

In [46]:
traduce('NACIO2')

                     aux_traduce
                          unique
NACIO2                          
                  [No aplicable]
1             [Un país de la UE]
2       [Un país fuera de la UE]


In [47]:
# reset('NACIO')
# reset('NACIO1')
df[['NACIO_','NACIO1_','NACIO2_']].groupby(['NACIO_','NACIO1_']).agg(['unique'])
# Sin respuesta para quienes declaran 'nacionalidad española' y 'nacionalidad española y otra' (1 y 2 en 'NACIO').
# No se consigue afinar por completo el criterio para establecer cuándo es 'No aplicable'.

NACIO2_
                                                                                            unique
NACIO_            NACIO1_                                                                         
Española          Un país de la UE                                                  [No aplicable]
Española y otra   Un país de la UE                                                  [No aplicable]
                  Un país fuera de la UE                                            [No aplicable]
Otra nacionalidad Un país de la UE        [No aplicable, Un país de la UE, Un país fuera de la UE]
                  Un país fuera de la UE                          [No aplicable, Un país de la UE]

In [48]:
# Provisionalmente se imputa None a las no respuestas:
no_respuesta('NACIO2')

[ 1.  2. nan]


In [49]:
# Se imputa 'NACIO2'=1 (UE)  a quienes declararon 'nacionalidad española y otra' (2 en 'NACIO') y la primera ...
# ... no es nacionalidad EU ('NACIO1' distinta de 1), ya que la española es su segunda nacionalidad:
df['NACIO2'].mask((df['NACIO']==2)&(df['NACIO1']!=1),1,inplace=True)
traduce('NACIO2',check=False)
df[['NACIO_','NACIO2_']].groupby('NACIO_').agg(['unique']) # Comprobación: 

,NACIO2_
,unique
NACIO_,
Española,[nan]
Española y otra,"[Un país de la UE, nan]"
Otra nacionalidad,"[nan, Un país de la UE, Un país fuera de la UE]"


In [50]:
proporciones('NACIO2_')

NaN                       0.991406
Un país de la UE          0.008341
Un país fuera de la UE    0.000253
Name: NACIO2_, dtype: float64

In [51]:
proporciones('NACIO2_',dropna=True)

Un país de la UE          0.970588
Un país fuera de la UE    0.029412
Name: NACIO2_, dtype: float64

## 'PAIS_NACI'
**País de nacimiento**.

In [52]:
df['PAIS_NACI'].unique() # No missing values 

array([1, 3, 2])

In [53]:
traduce('PAIS_NACI')

                          aux_traduce
                               unique
PAIS_NACI                            
1                            [España]
2                [Otro país de la UE]
3          [Otro país fuera de la UE]


In [54]:
proporciones('PAIS_NACI_')

España                      0.971786
Otro país fuera de la UE    0.019462
Otro país de la UE          0.008752
Name: PAIS_NACI_, dtype: float64

## Nueva variable 'eu27'
**Nacido en EU27**: 1=Nacido en EU27, 0=No nacido en EU27

In [55]:
df['eu27']=0
df['eu27'].mask((df['PAIS_NACI']==1)|(df['PAIS_NACI']==2),1,inplace=True)
df[['eu27','PAIS_NACI_']].groupby('eu27').agg(['unique']) # Comprobación

,PAIS_NACI_
,unique
eu27,
0,[Otro país fuera de la UE]
1,"[España, Otro país de la UE]"


In [56]:
D_describe['eu27']='Nacido o no en EU27'
D_dict['eu27']={0:'No nacido en EU27',1:'Nacido en EU27'}
traduce('eu27')

              aux_traduce
                   unique
eu27                     
0     [No nacido en EU27]
1        [Nacido en EU27]


In [57]:
proporciones('eu27')

1    0.980538
0    0.019462
Name: eu27, dtype: float64

## 'TITU' (redefinida como 'titulo_g1')
**Titulación** de grado universitario, diplomatura, licenciatura o equivalente.

In [58]:
df['TITU'].unique() # No missing values

array([ 11101,  11201,  11301,  11401,  11901,  21101,  21201,  21301,
        21302,  21401,  21502,  22201,  22202,  22301,  22901,  23101,
        23102,  23103,  23104,  23201,  23202,  23901,  31101,  31201,
        31202,  31301,  31401,  31402,  31404,  31406,  32101,  32102,
        32201,  41201,  41202,  41301,  41302,  41303,  41401,  41402,
        41403,  41601,  42101,  51101,  51201,  51202,  51901,  52101,
        53101,  53201,  53202,  53203,  53301,  54101,  54201,  61301,
        61901,  71101,  71301,  71302,  71401,  71402,  71403,  71404,
        71405,  71501,  71502,  71503,  71601,  71603,  71901,  72101,
        72102,  72201,  72401,  73101,  73102,  73201,  73202,  81102,
        81103,  81201,  82101,  84101,  91101,  91201,  91301,  91401,
        91402,  91501,  91502,  91503,  91504,  91505,  91601,  92301,
       101401, 101501, 104101, 104103, 109999])

In [59]:
df['TITU'].nunique()

101

In [60]:
leyenda('TITU')

011101 	 Pedagogía
011201 	 Educación infantil
011301 	 Educación primaria
011401 	 Otros maestros
011901 	 Educación social
021101 	 Audiovisual, imagen y multimedia
021201 	 Diseño
021301 	 Bellas artes
021302 	 Historia del arte
021401 	 Conservación y restauración
021502 	 Música y Artes escénicas
022201 	 Arqueología
022202 	 Historia
022301 	 Filosofía
022901 	 Humanidades
023101 	 Lengua inglesa
023102 	 Lenguas clásicas
023103 	 Otras lenguas extranjeras
023104 	 Traducción e interpretación
023201 	 Lenguas y dialectos españoles
023202 	 Literatura
023901 	 Lenguas modernas y aplicadas
031101 	 Economía
031201 	 Política y gestión pública
031202 	 Relaciones internacionales
031301 	 Psicología
031401 	 Antropología social y cultural y Estudios y gestión de la cultura
031402 	 Criminología
031404 	 Geografía
031406 	 Sociología e Igualdad de género
032101 	 Comunicación
032102 	 Periodismo
032201 	 Información y documentación
041201 	 Financiera y actuarial
041202 	 Finanzas y c

In [61]:
traduce('TITU',check=False)

In [62]:
redefine('TITU','titulo_g1')

In [63]:
proporciones('titulo_g1_').head(20)

Administración y empresa                  0.062810
Educación primaria                        0.060251
Derecho                                   0.054343
Educación infantil                        0.043127
Enfermería                                0.039019
Informática                               0.033553
Psicología                                0.032132
Trabajo social                            0.025592
Medicina                                  0.024233
Economía                                  0.024107
Periodismo                                0.019210
Actividad física y del deporte            0.019052
Turismo                                   0.018483
Ingeniería civil                          0.018388
Historia                                  0.018072
Pedagogía                                 0.017946
Biología                                  0.016935
Audiovisual, imagen y multimedia          0.016714
Ciencias ambientales                      0.015450
Ingeniería en tecnologías indus

In [64]:
D_describe['titulo_g1']='Titulación de grado universitario, diplomatura, licenciatura o equivalente'

## 'RAMA' (redefinida como 'rama_g1')
**Rama de conocimiento** de la titulación.

In [65]:
df['RAMA'].unique() # No missing values

array([3, 1, 4, 5, 2])

In [66]:
traduce('RAMA')

                          aux_traduce
                               unique
RAMA                                 
1               [Artes y humanidades]
2                          [Ciencias]
3     [Ciencias sociales y jurídicas]
4         [Ingeniería y arquitectura]
5              [Ciencias de la salud]


In [67]:
redefine('RAMA',nuevo='rama_g1')

In [68]:
proporciones('rama_g1_')

Ciencias sociales y jurídicas    0.458248
Ingeniería y arquitectura        0.211842
Ciencias de la salud             0.141796
Artes y humanidades              0.100376
Ciencias                         0.087738
Name: rama_g1_, dtype: float64

In [69]:
# Algunas titulaciones adscritas a más de una rama del conocimiento (e.g. Traducción e interpretación)
df[['rama_g1_','titulo_g1_']].groupby('titulo_g1_').agg(['unique'])

,rama_g1_
,unique
titulo_g1_,
Actividad física y del deporte,"[Ciencias sociales y jurídicas, Ciencias de la salud]"
Administración y empresa,[Ciencias sociales y jurídicas]
Antropología social y cultural y Estudios y gestión de la cultura,"[Artes y humanidades, Ciencias sociales y jurídicas]"
Arqueología,[Artes y humanidades]
Arquitectura técnica,[Ingeniería y arquitectura]
...,...
Trabajo social,[Ciencias sociales y jurídicas]
Traducción e interpretación,"[Artes y humanidades, Ciencias sociales y jurídicas]"


In [70]:
df[['rama_g1_','titulo_g1_']].groupby('rama_g1_').agg(['unique'])

,titulo_g1_
,unique
rama_g1_,
Artes y humanidades,"[Pedagogía, Audiovisual, imagen y multimedia, Diseño, Bellas artes, Historia del arte, Conservación y restauración, Música y Artes escénicas, Arqueología, Historia, Filosofía, Humanidades, Lengua inglesa, Lenguas clásicas, Otras lenguas extranjeras, Traducción e interpretación, Lenguas y dialect..."
Ciencias,"[Biología, Bioquímica, Biotecnología, Biomedicina, Ciencias ambientales, Química, Ciencias del mar, Geología, Física, Matemáticas, Estadística, Ingeniería de organización industrial y Nanotecnología, Ciencia y tecnología de los alimentos e Ingeniería alimentaria, Enología, Óptica y optometría, N..."
Ciencias de la salud,"[Psicología, Biología, Biomedicina, Ciencia y tecnología de los alimentos e Ingeniería alimentaria, Veterinaria, Odontología, Medicina, Enfermería, Óptica y optometría, Fisioterapia, Logopedia, Nutrición humana y dietética, Podología, Terapia ocupacional, Farmacia, Actividad física y del deporte]"
Ciencias sociales y jurídicas,"[Pedagogía, Educación infantil, Educación primaria, Otros maestros, Educación social, Audiovisual, imagen y multimedia, Traducción e interpretación, Economía, Política y gestión pública, Relaciones internacionales, Psicología, Antropología social y cultural y Estudios y gestión de la cultura, Cr..."
Ingeniería y arquitectura,"[Audiovisual, imagen y multimedia, Diseño, Geología, Matemáticas, Desarrollo de software y de aplicaciones e Ingeniería multimedia, Informática, Ingeniería química industrial e Ingeniería medioambiental, Ingeniería de la energía, Ingeniería eléctrica, Ingeniería de computadores, Ingeniería de so..."


In [71]:
# Distribucion de sexos condicionada por ramas de conocimiento: 
dist_condicionada('rama_g1_','fem_',rev=True)
# Solo la rama ciencias tiene una distribución de sexos similar a la general entre los graduados. 
# La mayor presencia femenina se da en ciencias de la salud. La menor, en ingeniería y arquitectura. 

In [72]:
# Distribucion de ramas de conocimiento condicionada por sexos: 
dist_condicionada('rama_g1_','fem_')
# La mayor presencia femenina en Ciencias se debe a la mayor presencia de mujeres entre los graduados, ... 
# ... que compensa que las mujeres son menos propensas que los hombres a elegir estudios de Ciencias.

In [73]:
# Distribucion de nacionalidad española y otras condicionada por ramas de conocimiento: 
dist_condicionada('rama_g1_','esp_',rev=True)
# Artes y humanidades y ciencias de la salud son las ramas de conocimiento en las que la presencia de ...
# ... nacionalidades no españolas es superior a la general entre los graduados. 

In [74]:
# Distribucion de ramas de conocimiento según nacionalidad española u otras: 
dist_condicionada('rama_g1_','esp_')

## Nuevas variables dicotómicas: 'art_hum', 'ciencias', 'cc_soc_ju', 'ing_arq', 'cc_salud'
**'art_hum'**: 1=Rama artes y humanidades, 0=Otra rama,  
**'ciencias'**: 1=Rama ciencias, 0=Otra rama,  
**'cc_soc_ju'**: 1=Rama ciencias sociales y jurídicas, 0=Otra rama,  
**'ing_arq'**: 1=Rama ingeniería y arquitectura, 0=Otra rama,  
**'cc_salud'**: 1=Rama cc_salud, 0=Otra rama,

In [75]:
L_ramas=['art_hum','ciencias','cc_soc_ju','ing_arq','cc_salud']
for i,rama in enumerate (L_ramas): 
    df[rama]=0
    df[rama].mask(df['rama_g1']==i+1,1,inplace=True)
df[L_ramas+['rama_g1_']].groupby('rama_g1_').agg(['unique']) # Comprobación: 

,art_hum,ciencias,cc_soc_ju,ing_arq,cc_salud
,unique,unique,unique,unique,unique
rama_g1_,,,,,
Artes y humanidades,[1],[0],[0],[0],[0]
Ciencias,[0],[1],[0],[0],[0]
Ciencias de la salud,[0],[0],[0],[0],[1]
Ciencias sociales y jurídicas,[0],[0],[1],[0],[0]
Ingeniería y arquitectura,[0],[0],[0],[1],[0]


In [76]:
for vble in L_ramas: 
    a_lista(vble)

## 'AMBITO' (redefinida como 'ambito_g1')
**Ámbito de estudio**.

In [77]:
df['AMBITO'].sort_values().unique() # No missing values

array([  11,   21,   22,   23,   31,   32,   41,   42,   51,   52,   53,
         54,   61,   71,   72,   73,   81,   82,   84,   91,   92,  109,
        112,  113,  211,  311,  313,  413,  912,  913, 1014, 1015])

In [78]:
df['AMBITO'].nunique()

32

In [79]:
leyenda('AMBITO')

0112 	 Formación de docentes de enseñanza infantil 
0113 	 Formación de docentes de enseñanza primaria 
011 	 Educación (Otros estudios) 
0211 	 Técnicas audiovisuales y medios de comunicación 
021 	 Artes (Otros estudios) 
022 	 Humanidades 
023 	 Lenguas 
0311 	 Economía 
0313 	 Psicología 
031 	 Ciencias sociales y del comportamiento (Otros estudios) 
032 	 Periodismo y documentación 
0413 	 Dirección y administración 
041 	 Negocios y administración (Otros estudios) 
042 	 Derecho 
051 	 Ciencias de la vida 
052 	 Medio ambiente 
053 	 Ciencias químicas, físicas y geológicas 
054 	 Matemáticas y estadística 
061 	 Informática
071 	 Ingeniería y profesiones afines 
072 	 Industria manufacturera y producción 
073 	 Arquitectura y construcción 
081 	 Agricultura y ganadería 
082 	 Silvicultura 
084 	 Veterinaria 
0912 	 Medicina 
0913 	 Enfermería 
091 	 Salud (Otros estudios) 
092 	 Servicios sociales 
1014 	 Actividades físicas y deportivas 
1015 	 Viajes, turismo y ocio 
109 	 Serv

In [80]:
traduce('AMBITO',check=False)

In [81]:
redefine('AMBITO','ambito_g1')

In [82]:
proporciones('ambito_g1_')

Ingeniería y profesiones afines                             0.082430
Dirección y administración                                  0.079460
Formación de docentes de enseñanza primaria                 0.060251
Derecho                                                     0.054343
Arquitectura y construcción                                 0.048719
Formación de docentes de enseñanza infantil                 0.043127
Salud (Otros estudios)                                      0.042842
Enfermería                                                  0.039019
Lenguas                                                     0.036492
Informática                                                 0.035765
Educación (Otros estudios)                                  0.033553
Psicología                                                  0.032132
Ciencias de la vida                                         0.029794
Humanidades                                                 0.028941
Artes (Otros estudios)            

In [83]:
# Todas las titulaciones adscritas a un único ámbito de estudio.  
for row in range(0,len(df[['titulo_g1_','ambito_g1_']].groupby('titulo_g1_'))):
    print (df[['titulo_g1_','ambito_g1_']].groupby('titulo_g1_').agg(['unique']).iloc[row])

ambito_g1_  unique    [Actividades físicas y deportivas ]
Name: Actividad física y del deporte, dtype: object
ambito_g1_  unique    [Dirección y administración ]
Name: Administración y empresa, dtype: object
ambito_g1_  unique    [Ciencias sociales y del comportamiento (Otros estudios) ]
Name: Antropología social y cultural y Estudios y gestión de la cultura, dtype: object
ambito_g1_  unique    [Humanidades ]
Name: Arqueología, dtype: object
ambito_g1_  unique    [Arquitectura y construcción ]
Name: Arquitectura técnica, dtype: object
ambito_g1_  unique    [Arquitectura y construcción ]
Name: Arquitectura y Urbanismo y paisajismo, dtype: object
ambito_g1_  unique    [Técnicas audiovisuales y medios de comunicación ]
Name: Audiovisual, imagen y multimedia, dtype: object
ambito_g1_  unique    [Artes (Otros estudios) ]
Name: Bellas artes, dtype: object
ambito_g1_  unique    [Ciencias de la vida ]
Name: Biología, dtype: object
ambito_g1_  unique    [Ciencias de la vida ]
Name: Biomedicina,

ambito_g1_  unique    [Salud (Otros estudios) ]
Name: Podología, dtype: object
ambito_g1_  unique    [Ciencias sociales y del comportamiento (Otros estudios) ]
Name: Política y gestión pública, dtype: object
ambito_g1_  unique    [Negocios y administración (Otros estudios) ]
Name: Protocolo y eventos, dtype: object
ambito_g1_  unique    [Psicología ]
Name: Psicología, dtype: object
ambito_g1_  unique    [Negocios y administración (Otros estudios) ]
Name: Publicidad y relaciones públicas, dtype: object
ambito_g1_  unique    [Ciencias químicas, físicas y geológicas ]
Name: Química, dtype: object
ambito_g1_  unique    [Ciencias sociales y del comportamiento (Otros estudios) ]
Name: Relaciones internacionales, dtype: object
ambito_g1_  unique    [Servicios (Otros estudios) ]
Name: Servicio de transporte terrestre y Servicio de transporte aéreo, dtype: object
ambito_g1_  unique    [Servicios (Otros estudios) ]
Name: Servicios (otros estudios), dtype: object
ambito_g1_  unique    [Ciencias s

In [84]:
# 'AMBITO' determinado por 2, 3 o 4 primeros dígitos de 'titulacion_gr_u' excepto los siguientes casos: 
cond1=(df['ambito_g1'].astype('str')!=df['titulo_g1'].astype('str').str[:2])
cond2=(df['ambito_g1'].astype('str')!=df['titulo_g1'].astype('str').str[:3])
cond3=(df['ambito_g1'].astype('str')!=df['titulo_g1'].astype('str').str[:4])
df[['ambito_g1','titulo_g1','rama_g1']][cond1&cond2&cond3].value_counts()

ambito_g1  titulo_g1  rama_g1
109        104101     4          109
           104103     3           47
                      4           10
dtype: int64

In [85]:
# Algunos ámbitos de estudio asociados a más de una rama de conocimiento: 
df[['rama_g1_','ambito_g1_']].groupby('ambito_g1_').agg(['unique'])

,rama_g1_
,unique
ambito_g1_,
Actividades físicas y deportivas,"[Ciencias sociales y jurídicas, Ciencias de la salud]"
Agricultura y ganadería,[Ingeniería y arquitectura]
Arquitectura y construcción,"[Ingeniería y arquitectura, Artes y humanidades]"
Artes (Otros estudios),"[Artes y humanidades, Ingeniería y arquitectura]"
Ciencias de la vida,"[Ciencias, Ciencias de la salud]"
"Ciencias químicas, físicas y geológicas","[Ciencias, Artes y humanidades, Ciencias sociales y jurídicas, Ingeniería y arquitectura]"
Ciencias sociales y del comportamiento (Otros estudios),"[Ciencias sociales y jurídicas, Artes y humanidades]"
Derecho,[Ciencias sociales y jurídicas]


## 'T_UNIV'
**Tipo de universidad**.

In [86]:
df['T_UNIV'].unique() # No missing values

array([2, 1, 4, 3])

In [87]:
traduce('T_UNIV')

                              aux_traduce
                                   unique
T_UNIV                                   
1        [Universidad Pública presencial]
2       [Universidad Pública a distancia]
3        [Universidad Privada presencial]
4       [Universidad Privada a distancia]


In [88]:
proporciones('T_UNIV')

1    0.818236
3    0.119838
2    0.034122
4    0.027803
Name: T_UNIV, dtype: float64

## Nueva variable: 'tipo_centro'
**Tipo de centro educativo**: 1=Centros públicos, 0=Centros privados  

Variable proxi (en otros datasets las preguntas sobre tipo de centro están referidas a niveles educativos diferentes).

In [89]:
df['tipo_centro']=0
df['tipo_centro'].mask((df['T_UNIV']==1)|(df['T_UNIV']==2),1,inplace=True)
df[['T_UNIV','tipo_centro']].groupby('tipo_centro').agg(['unique']) # Comprobación: 

,T_UNIV
,unique
tipo_centro,
0,"[4, 3]"
1,"[2, 1]"


In [90]:
D_describe['tipo_centro']='Tipo de centro educativo: público o privado'
D_dict['tipo_centro']={0:'Privado',1:'Público'}
traduce('tipo_centro')

            aux_traduce
                 unique
tipo_centro            
0             [Privado]
1             [Público]


In [91]:
proporciones('tipo_centro_')

Público    0.852359
Privado    0.147641
Name: tipo_centro_, dtype: float64

In [92]:
# Distribucion de centros públicos y privados condicionada por nacionalidad española u otras: 
dist_condicionada('esp_','tipo_centro_',rev=True)
# La mayor frecuencia de elección de centros públicos en general se acentúa en el caso de aquellos cuya ...
# ... nacionalidad no es española. 

## Nueva variable: 'a_dist'
**Formación presencial o a distancia**: 1=Formación a distancia, 0=Formación presencial  

In [93]:
df['a_dist']=0
df['a_dist'].mask((df['T_UNIV']==2)|(df['T_UNIV']==4),1,inplace=True)
df[['T_UNIV','a_dist']].groupby('a_dist').agg(['unique']) # Comprobación: 

,T_UNIV
,unique
a_dist,
0,"[1, 3]"
1,"[2, 4]"


In [94]:
D_describe['a_dist']='Tipo de formación: a distancia o presencial'
D_dict['a_dist']={0:'Formación presencial',1:'Formación a distancia'}
traduce('a_dist')

                    aux_traduce
                         unique
a_dist                         
0        [Formación presencial]
1       [Formación a distancia]


In [95]:
proporciones('a_dist_')

Formación presencial     0.938075
Formación a distancia    0.061925
Name: a_dist_, dtype: float64

## 'DISCA' (redefinida como 'disca')
Tiene **discapacidad reconocida** superior al 33 %.  
Nota: Se modifica en 'disca' el valor No: 0=No. 

In [96]:
df['DISCA'].unique() # No missing values

array([2, 1])

In [97]:
traduce('DISCA')

      aux_traduce
           unique
DISCA            
1            [Sí]
2            [No]


In [98]:
redefine('DISCA',si_no=True)

      aux_traduce
           unique
disca            
0            [No]
1            [Sí]


In [99]:
proporciones('disca_')

No    0.987804
Sí    0.012196
Name: disca_, dtype: float64

In [100]:
# Distribución de discapacidad según sexos: 
dist_condicionada('fem_','disca_',rev=True)
# La proporción de estudiantes con discapacidad es mayor entre los hombres que entre las mujeres. 

In [101]:
# Distribución de tipo de enseñanza según discapacidad: 
dist_condicionada('disca_','a_dist_',rev=True)
# Los estudiantes con discapacidad se forman a distancia con el doble de frecuencia que el resto de estudiantes.

In [102]:
# Distribución discapacidad segun el tipo de centro: 
dist_condicionada('disca_','tipo_centro_')
# Los centros públicos tienen mayor proporción de estudiantes con discapacidad que los privados.

## 'PROXY'
**Tipo de entrevista**.

In [103]:
df['PROXY'].unique() # No missing values

array([1, 2])

In [104]:
traduce('PROXY',save=False) # No conserva la columna traducida

                                            aux_traduce
                                                 unique
PROXY                                                  
1      [Entrevista directa con la persona seleccionada]
2                                    [Entrevista proxy]


In [105]:
proporciones('PROXY')

1    0.987647
2    0.012353
Name: PROXY, dtype: float64

## 'LUG_RES_PAIS' (redefinida como 'reside')
**País de residencia actual**.

In [106]:
df['LUG_RES_PAIS'].sort_values().unique() # No missing values

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

In [107]:
traduce('LUG_RES_PAIS')

                                                 aux_traduce
                                                      unique
LUG_RES_PAIS                                                
1                                                   [España]
2                                              [Reino Unido]
3                                                 [Alemania]
4                                                  [Francia]
5                            [Otro país de la Unión Europea]
6                       [Otro país europeo (fuera de la UE)]
7                             [Un país de América del Norte]
8             [Un país de América Central o América del Sur]
9                                          [Un país de Asia]
10                                       [Un país de África]
11                                      [Un país de Oceanía]


In [108]:
redefine('LUG_RES_PAIS','reside')

In [109]:
proporciones('reside_')

España                                          0.944109
Reino Unido                                     0.015987
Otro país de la Unión Europea                   0.013586
Alemania                                        0.006793
Un país de América del Norte                    0.004644
Francia                                         0.004486
Otro país europeo (fuera de la UE)              0.003791
Un país de América Central o América del Sur    0.002591
Un país de Asia                                 0.002117
Un país de Oceanía                              0.001517
Un país de África                               0.000379
Name: reside_, dtype: float64

## 'PAIS_NAC_PADRE'
**País de nacimiento del padre**.

In [110]:
df['PAIS_NAC_PADRE'].unique() # Missing values as 9

array([1, 9, 2, 3])

In [111]:
traduce('PAIS_NAC_PADRE')

                               aux_traduce
                                    unique
PAIS_NAC_PADRE                            
1                                 [España]
2                     [Otro país de la UE]
3               [Otro país fuera de la UE]
9                                  [NS/NC]


In [112]:
# Se imputa None a NS/NC:
no_respuesta('PAIS_NAC_PADRE',tipo='int')

[ 1.  2.  3. nan]


In [113]:
proporciones('PAIS_NAC_PADRE_')

España                      0.954567
Otro país fuera de la UE    0.026855
Otro país de la UE          0.011848
NaN                         0.006730
Name: PAIS_NAC_PADRE_, dtype: float64

## 'ESTUDIOS_PADRE'
**Nivel de formación máximo alcanzado por el padre**.

In [114]:
df['ESTUDIOS_PADRE'].unique() # Missing values as 9

array([2, 3, 7, 8, 5, 4, 6, 9, 1])

In [115]:
traduce('ESTUDIOS_PADRE')

                                                                                                 aux_traduce
                                                                                                      unique
ESTUDIOS_PADRE                                                                                              
1                                                                                  [No sabe leer o escribir]
2                                 [Educación primaria incompleta (ha asistido menos de 5 años a la escuela)]
3                                                                              [Educación primaria completa]
4               [Primera etapa de Enseñanza Secundaria, con o sin título (ESO, EGB, Bachillerato Elemental)]
5                                            [Estudios de Bachillerato (Bachillerato LOGSE, BUP, COU, Preu)]
6                                                   [Enseñanzas profesionales de grado medio o equivalentes]
7                  

In [116]:
# Se imputa None a NS/NC (9):
no_respuesta('ESTUDIOS_PADRE',tipo='int')

[ 1.  2.  3.  4.  5.  6.  7.  8. nan]


In [117]:
proporciones('ESTUDIOS_PADRE_')
# El grupo más numeroso es el de padres con estudios universitarios. 
# Los nivele de formación más bajos son menos frecuentes que la falta de respuesta. 

Estudios universitarios (Diplomatura, Licenciatura, Doctorado) o equivalentes                 0.285109
Educación primaria completa                                                                   0.166219
Primera etapa de Enseñanza Secundaria, con o sin título (ESO, EGB, Bachillerato Elemental)    0.159458
Estudios de Bachillerato (Bachillerato LOGSE, BUP, COU, Preu)                                 0.114025
Enseñanzas profesionales de grado medio o equivalentes                                        0.076111
Enseñanzas profesionales de grado superior o equivalentes                                     0.073078
NaN                                                                                           0.069508
Educación primaria incompleta (ha asistido menos de 5 años a la escuela)                      0.052921
No sabe leer o escribir                                                                       0.003570
Name: ESTUDIOS_PADRE_, dtype: float64

## Nueva variable: 'univ_pa'
**Padre** con o sin **estudios universitarios**: 1=Con estudios universitarios, 0=Sin estudios universitarios 

In [118]:
df['univ_pa']=0
df['univ_pa'].mask(df['ESTUDIOS_PADRE']==8,1,inplace=True)
df[['ESTUDIOS_PADRE','univ_pa']].groupby('univ_pa').agg(['unique']) # Comprobación: 

,ESTUDIOS_PADRE
,unique
univ_pa,
0,"[2.0, 3.0, 7.0, 5.0, 4.0, 6.0, nan, 1.0]"
1,[8.0]


In [119]:
D_describe['univ_pa']='Padre con o sin estudios universitarios'
D_dict['univ_pa']={0:'Sin estudios universitarios',1:'Con estudios universitarios'}
traduce('univ_pa')

                           aux_traduce
                                unique
univ_pa                               
0        [Sin estudios universitarios]
1        [Con estudios universitarios]


In [120]:
# Distribucion de padre universitario o no según nacionalidad española u otras: 
dist_condicionada('univ_pa_','esp_')
# La frecuencia de padre universitario es mayor entre los graduados con nacionalidad no española.

In [121]:
# Distribucion de padre universitario o no según nacidos o no en EU27: 
dist_condicionada('univ_pa_','eu27_')
# La frecuencia de padre universitario es aun mayor entre los no nacidos en EU27. Puede reflejar mayores ...
# ... barreras de acceso a la universidad española para los estudiantes no europeos. 

In [122]:
# Distribucion de sexos según estudios universitarios del padre: 
dist_condicionada('univ_pa_','fem_',rev=True)
# La presencia de mujeres es mayor en el conjunto de graduados con padres no universitarios que entre los ...
# ... universitarios.

## 'PAIS_NAC_MADRE'
**País de nacimiento de la madre**.

In [123]:
df['PAIS_NAC_MADRE'].unique() # Missing values as 9

array([1, 3, 9, 2])

In [124]:
traduce('PAIS_NAC_MADRE')

                               aux_traduce
                                    unique
PAIS_NAC_MADRE                            
1                                 [España]
2                     [Otro país de la UE]
3               [Otro país fuera de la UE]
9                                  [NS/NC]


In [125]:
# Se imputa None a NS/NC (9):
no_respuesta('PAIS_NAC_MADRE',tipo='int')

[ 1.  2.  3. nan]


In [126]:
proporciones('PAIS_NAC_MADRE_')

España                      0.947490
Otro país fuera de la UE    0.025908
Otro país de la UE          0.014249
NaN                         0.012353
Name: PAIS_NAC_MADRE_, dtype: float64

## 'ESTUDIOS_MADRE'
**Nivel de formación máximo alcanzado por la madre**.

In [127]:
df['ESTUDIOS_MADRE'].unique() # Missing values as 9

array([3, 7, 2, 4, 5, 6, 8, 9, 1])

In [128]:
traduce('ESTUDIOS_MADRE')

                                                                                                 aux_traduce
                                                                                                      unique
ESTUDIOS_MADRE                                                                                              
1                                                                                  [No sabe leer o escribir]
2                                 [Educación primaria incompleta (ha asistido menos de 5 años a la escuela)]
3                                                                              [Educación primaria completa]
4               [Primera etapa de Enseñanza Secundaria, con o sin título (ESO, EGB, Bachillerato Elemental)]
5                                            [Estudios de Bachillerato (Bachillerato LOGSE, BUP, COU, Preu)]
6                                                   [Enseñanzas profesionales de grado medio o equivalentes]
7                  

In [129]:
# Se imputa None a NS/NC (9):
no_respuesta('ESTUDIOS_MADRE',tipo='int')

[ 1.  2.  3.  4.  5.  6.  7.  8. nan]


In [130]:
proporciones('ESTUDIOS_MADRE_')
# La proporción de madres universitarias es solo un poco menor que la de padres universitarios. 
# La proporción de madres que no saben leer o escribir es solo un poco superior a la de padres en la misma sit.
# Lo mismo ocurre en cuanto a la educación primaria incompleta. 

Estudios universitarios (Diplomatura, Licenciatura, Doctorado) o equivalentes                 0.272977
Primera etapa de Enseñanza Secundaria, con o sin título (ESO, EGB, Bachillerato Elemental)    0.183849
Educación primaria completa                                                                   0.167420
Estudios de Bachillerato (Bachillerato LOGSE, BUP, COU, Preu)                                 0.123440
Enseñanzas profesionales de grado medio o equivalentes                                        0.091498
Enseñanzas profesionales de grado superior o equivalentes                                     0.059177
Educación primaria incompleta (ha asistido menos de 5 años a la escuela)                      0.051341
NaN                                                                                           0.046349
No sabe leer o escribir                                                                       0.003949
Name: ESTUDIOS_MADRE_, dtype: float64

In [131]:
proporciones('ESTUDIOS_PADRE_') 

Estudios universitarios (Diplomatura, Licenciatura, Doctorado) o equivalentes                 0.285109
Educación primaria completa                                                                   0.166219
Primera etapa de Enseñanza Secundaria, con o sin título (ESO, EGB, Bachillerato Elemental)    0.159458
Estudios de Bachillerato (Bachillerato LOGSE, BUP, COU, Preu)                                 0.114025
Enseñanzas profesionales de grado medio o equivalentes                                        0.076111
Enseñanzas profesionales de grado superior o equivalentes                                     0.073078
NaN                                                                                           0.069508
Educación primaria incompleta (ha asistido menos de 5 años a la escuela)                      0.052921
No sabe leer o escribir                                                                       0.003570
Name: ESTUDIOS_PADRE_, dtype: float64

## Nueva variable: 'univ_ma'
**Madre** con o sin **estudios universitarios**: 1=Con estudios universitarios, 0=Sin estudios universitarios 

In [132]:
df['univ_ma']=0
df['univ_ma'].mask(df['ESTUDIOS_MADRE']==8,1,inplace=True)
df[['ESTUDIOS_MADRE','univ_ma']].groupby('univ_ma').agg(['unique']) # Comprobación: 

,ESTUDIOS_MADRE
,unique
univ_ma,
0,"[3.0, 7.0, 2.0, 4.0, 5.0, 6.0, nan, 1.0]"
1,[8.0]


In [133]:
D_describe['univ_ma']='Madre con o sin estudios universitarios'
D_dict['univ_ma']=D_dict['univ_pa']
traduce('univ_ma')

                           aux_traduce
                                unique
univ_ma                               
0        [Sin estudios universitarios]
1        [Con estudios universitarios]


In [134]:
# Distribucion de madre universitaria o no según nacionalidad española u otras: 
dist_condicionada('univ_ma_','esp_')
# La frecuencia de madre universitaria es mayor entre los graduados con nacionalidad no española.

In [135]:
# Distribucion de madre universitaria o no según nacidos o no en EU27: 
dist_condicionada('univ_ma_','eu27_')
# De nuevo, la frecuencia de madre universitaria es aun mayor entre los no nacidos en EU27, aunque con menor ...
# ... diferencia que en el caso de los padres. 

In [136]:
# Distribucion de sexos según estudios universitarios de la madre: 
dist_condicionada('univ_ma_','fem_',rev=True)
# Misma relación que en el caso de los padres, aunque con menor distancia entre grupos. 

# BLOQUE B. EDUCACIÓN Y APRENDIZAJE DEL GRADUADO UNIVERSITARIO

## 'EST_B1' (redefinida como 'beca')
¿Disfrutó de **alguna beca** durante sus estudios?: 1=Sí, 2=No, 9=NS/NC.  

Nota: se modifica en 'beca' el valor asociado a No: 0=No

In [137]:
df['EST_B1'].unique() # Missing values as 9

array([2, 1, 9])

In [138]:
traduce('EST_B1')

       aux_traduce
            unique
EST_B1            
1             [Sí]
2             [No]
9          [NS/NC]


In [139]:
# Se imputa None a NS/NC:
no_respuesta('EST_B1',tipo='int')

[ 1.  2. nan]


In [140]:
redefine('EST_B1','beca',si_no=True)

     aux_traduce
          unique
beca            
0.0         [No]
1.0         [Sí]


In [141]:
proporciones('beca_')

No     0.532874
Sí     0.466304
NaN    0.000821
Name: beca_, dtype: float64

In [142]:
# Distribución de becas condicionadas por el sexo: 
dist_condicionada('beca_','fem_',dropna=True)
# La proporción de becas de estudios es superior entre las mujeres que entre los hombres. 

In [143]:
# Distribucion de becas condicionadas por nacionalidad española u otras: 
dist_condicionada('beca_','esp_',dropna=True)
# La proporción de becas de estudios es superior entre aquellos cuya nacionalidad es no española

In [144]:
# Distribucion de becas condicionadas por continente de nacimento:  
dist_condicionada('beca_','eu27_',dropna=True)
# La proporción de becas de estudios es superior entre los no nacidos en EU27. 

In [145]:
# Distribucion de becas condicionadas por discapacidad:  
dist_condicionada('beca_','disca_',dropna=True)
# La proporción de becas de estudios es superior entre los estudiantes sin discapacidad reconocida.

## 'EST_B2_1' (redefinida como 'beca_general')
(Solo para respuesta Sí (1) en variable 'EST_B1' (beca))  

¿Disfrutó de una **beca general para el estudio**?: 1=Sí, 2=No, 9=NS/NC.

Nota: se modifica en 'beca_general' el valor asociado a No: 0=No

In [146]:
df['EST_B2_1'].unique() # Missing values as ' ' or '9'

array([' ', '1', '2', '9'], dtype=object)

In [147]:
traduce('EST_B2_1')

             aux_traduce
                  unique
EST_B2_1                
          [No aplicable]
1                   [Sí]
2                   [No]
9                [NS/NC]


In [148]:
def re_beca(col):
    """Inspeccion e imputación de valores de variables condicionadas por respuesta a 'EST_B1' cuando pueden 
    deducirse de los itinerarios de la encuesta.
    """
    print (df[['EST_B1',col]].groupby('EST_B1').agg(['unique']))
    # Se imputa respuesta No (2) a quienes manifestaron no haber disfrutado de beca (2 en 'EST_B1')
    df[col].mask(df['EST_B1']==2,2,inplace=True)
    # Se imputa None al resto de respuestas en blanco y a NS/NC:
    try: df[col].replace({' ':None,'9':None,},inplace=True)
    except:
        df[col]=df[col].astype('O')
        df[col].replace({' ':None,'9':None,},inplace=True) # Resuelve conflictos cuando se aplica más de una vez
    try: df[col]=df[col].astype('int')
    except: df[col]=df[col].astype('float')
    traduce(col,check=False)
    print (df[['EST_B1',col]].groupby('EST_B1').agg(['unique']))
    return

In [149]:
re_beca('EST_B2_1')

         EST_B2_1
           unique
EST_B1           
1.0     [1, 2, 9]
2.0           [ ]
               EST_B2_1
                 unique
EST_B1                 
1.0     [1.0, 2.0, nan]
2.0               [2.0]


In [150]:
redefine('EST_B2_1','beca_general',si_no=True)

             aux_traduce
                  unique
beca_general            
0.0                 [No]
1.0                 [Sí]


In [151]:
proporciones('beca_general_')

No     0.613156
Sí     0.381536
NaN    0.005308
Name: beca_general_, dtype: float64

## 'EST_B2_2' (redefinida como 'beca_excel')
(Solo para respuesta Sí (1) en variable 'beca')  

¿Disfrutó de algún **premio o beca de excelencia**?: 1=Sí, 2=No, 9=NS/NC.  

Nota: se modifica en 'beca_excel' el valor asociado a No: 0=No.

In [152]:
df['EST_B2_2'].unique() # Missing values as ' ' or '9'

array([' ', '2', '1', '9'], dtype=object)

In [153]:
traduce('EST_B2_2')

             aux_traduce
                  unique
EST_B2_2                
          [No aplicable]
1                   [Sí]
2                   [No]
9                [NS/NC]


In [154]:
re_beca('EST_B2_2')

         EST_B2_2
           unique
EST_B1           
1.0     [2, 1, 9]
2.0           [ ]
               EST_B2_2
                 unique
EST_B1                 
1.0     [2.0, 1.0, nan]
2.0               [2.0]


In [155]:
redefine('EST_B2_2','beca_excel',si_no=True)

           aux_traduce
                unique
beca_excel            
0.0               [No]
1.0               [Sí]


In [156]:
proporciones('beca_excel_')

No     0.950554
Sí     0.044138
NaN    0.005308
Name: beca_excel_, dtype: float64

## 'EST_B2_3' (redefinida como 'beca_colab')
(Solo para respuesta Sí (1) en variable 'beca')  

¿Disfrutó de alguna **beca de colaboración** en la universidad?: 1=Sí, 2=No, 9=NS/NC.  

Nota: se modifica en 'beca_colab' el valor asociado a No: 0=No.

In [157]:
df['EST_B2_3'].unique() # Missing values as ' ' or '9'

array([' ', '2', '1', '9'], dtype=object)

In [158]:
traduce('EST_B2_3')

             aux_traduce
                  unique
EST_B2_3                
          [No aplicable]
1                   [Sí]
2                   [No]
9                [NS/NC]


In [159]:
re_beca('EST_B2_3')

         EST_B2_3
           unique
EST_B1           
1.0     [2, 1, 9]
2.0           [ ]
               EST_B2_3
                 unique
EST_B1                 
1.0     [2.0, 1.0, nan]
2.0               [2.0]


In [160]:
redefine('EST_B2_3','beca_colab',si_no=True)

           aux_traduce
                unique
beca_colab            
0.0               [No]
1.0               [Sí]


In [161]:
proporciones('beca_colab_')

No     0.959401
Sí     0.035291
NaN    0.005308
Name: beca_colab_, dtype: float64

## 'EST_B2_4' (redefinida como 'beca_pract')
(Solo para respuesta Sí (1) en variable 'beca')  

¿Disfrutó de alguna **beca de prácticas externas** o para prestar un servicio o desempeñar un trabajo fuera de la Universidad?: 1=Sí, 2=No, 9=NS/NC.  


Nota: se modifica en 'beca_pract' el valor asociado a No: 0=No.

In [162]:
df['EST_B2_4'].unique() # Missing values as ' ' or '9'

array([' ', '2', '1', '9'], dtype=object)

In [163]:
traduce('EST_B2_4')

             aux_traduce
                  unique
EST_B2_4                
          [No aplicable]
1                   [Sí]
2                   [No]
9                [NS/NC]


In [164]:
re_beca('EST_B2_4')

         EST_B2_4
           unique
EST_B1           
1.0     [2, 1, 9]
2.0           [ ]
               EST_B2_4
                 unique
EST_B1                 
1.0     [2.0, 1.0, nan]
2.0               [2.0]


In [165]:
redefine('EST_B2_4','beca_pract',si_no=True)

           aux_traduce
                unique
beca_pract            
0.0               [No]
1.0               [Sí]


In [166]:
proporciones('beca_pract_')

No     0.957000
Sí     0.037692
NaN    0.005308
Name: beca_pract_, dtype: float64

## 'EST_B2_5' (redefinida como 'beca_extr')
(Solo para respuesta Sí (1) en variable 'beca')  

¿Disfrutó de alguna **beca para realizar estudios fuera de España** (incluidas becas complemento a programas Erasmus)?: 1=Sí, 2=No, 9=NS/NC.  

Nota: se modifica en 'beca_extr' el valor asociado a No: 0=No.

In [167]:
df['EST_B2_5'].unique() # Missing values as ' ' or '9'

array([' ', '2', '1', '9'], dtype=object)

In [168]:
traduce('EST_B2_5')

             aux_traduce
                  unique
EST_B2_5                
          [No aplicable]
1                   [Sí]
2                   [No]
9                [NS/NC]


In [169]:
re_beca('EST_B2_5')

         EST_B2_5
           unique
EST_B1           
1.0     [2, 1, 9]
2.0           [ ]
               EST_B2_5
                 unique
EST_B1                 
1.0     [2.0, 1.0, nan]
2.0               [2.0]


In [170]:
redefine('EST_B2_5','beca_extr',si_no=True)

          aux_traduce
               unique
beca_extr            
0.0              [No]
1.0              [Sí]


In [171]:
proporciones('beca_extr_')
# La beca más frecuente es la general de estudios, seguida de las destinadas a estudiar fuera de España.

No     0.889040
Sí     0.105652
NaN    0.005308
Name: beca_extr_, dtype: float64

## 'EST_M1' (redefinida como 'extr')
(Solo para respuesta distinta de Sí (1) en variable 'EST_B2_5')  

¿Realizó parte de sus **estudios en el extranjero**?: 1=Sí, 2=No, 9=NS/NC.  

Nota: se modifica el valor asociado a No: 0=No.

In [172]:
df['EST_M1'].unique() # Missing values as '9'

array([2, 1, 9])

In [173]:
traduce('EST_M1')

       aux_traduce
            unique
EST_M1            
1             [Sí]
2             [No]
9          [NS/NC]


In [174]:
df[['EST_B2_5_','EST_M1_']].groupby('EST_B2_5_').agg(['unique']) # Inspección
# Imputada por el encuestador respuesta Si (1) a quienes manifestaron haber disfrutado de beca para ...
# ... estudiar fuera de España ('EST_B2_5')

,EST_M1_
,unique
EST_B2_5_,
No,"[No, Sí, NS/NC]"
Sí,[Sí]


In [175]:
# Se imputa None a NS/NC: 
no_respuesta('EST_M1',tipo='int')

[ 1.  2. nan]


In [176]:
df[['EST_B2_5_','EST_M1_']].groupby('EST_B2_5_').agg(['unique']) # Comprobacion: 

,EST_M1_
,unique
EST_B2_5_,
No,"[No, Sí, nan]"
Sí,[Sí]


In [177]:
redefine('EST_M1','extr',si_no=True)

     aux_traduce
          unique
extr            
0.0         [No]
1.0         [Sí]


In [178]:
proporciones('extr_')

No     0.832201
Sí     0.167167
NaN    0.000632
Name: extr_, dtype: float64

In [179]:
# Proporción de beca específica para estudiar fuera entre quienes estudiaron en el extranjero: 
df['beca_extr_'][df['extr']==1].value_counts(normalize=True)

Sí    0.634054
No    0.365946
Name: beca_extr_, dtype: float64

## Nueva variable: 'extr_sin_beca'
¿Realizó parte de sus **estudios en el extranjero sin beca** específica?: 1=Si, 0=No.  

Variable proxi de renta familiar alta.  

In [180]:
df['extr_sin_beca']=0
df['extr_sin_beca'].mask((df['extr']==1)&(df['EST_B2_5']!=1),1,inplace=True)
df[['extr_sin_beca','extr','EST_B2_5']].groupby(['extr','EST_B2_5']).agg(['unique']) # Comprobación

extr_sin_beca
                     unique
extr EST_B2_5              
0.0  2.0                [0]
1.0  1.0                [0]
     2.0                [1]

In [181]:
D_describe['extr_sin_beca']='Realizó parte de sus estudios en el extranjero sin beca específica (proxy renta \
familiar)'
D_dict['extr_sin_beca']=D_dict['si_no']
traduce('extr_sin_beca')

              aux_traduce
                   unique
extr_sin_beca            
0                    [No]
1                    [Sí]


In [182]:
proporciones('extr_sin_beca_')

No    0.938485
Sí    0.061515
Name: extr_sin_beca_, dtype: float64

## 'EST_M2' (redefinida como 'tipo_beca_extr')

(Solo para respuesta Sí (1) en variable 'EST_B2_5' (o 'beca_extr'))  

**Programa o beca de movilidad**.

In [183]:
df['EST_M2'].unique() # Missing values as ' ' or '9'

array([' ', '3', '1', '2', '9'], dtype=object)

In [184]:
traduce('EST_M2')

                                      aux_traduce
                                           unique
EST_M2                                           
                                   [No aplicable]
1                              [Programa Erasmus]
2       [Otros programas o becas dentro de la UE]
3        [Otros programas o becas fuera de la UE]
9                                         [NS/NC]


In [185]:
df[['beca_extr_','EST_M2_']].groupby('beca_extr_',dropna=False).agg(['unique']) # Inspección

,EST_M2_
,unique
beca_extr_,
No,[No aplicable]
Sí,"[Otros programas o becas fuera de la UE, Programa Erasmus, Otros programas o becas dentro de la UE, NS/NC]"
NaN,[No aplicable]


In [186]:
# Se imputa None a NS/NC (9) y a respuesta en blanco: 
no_respuesta('EST_M2')

[ 1.  2.  3. nan]


In [187]:
redefine('EST_M2','tipo_beca_extr')

In [188]:
df[['beca_extr_','tipo_beca_extr_']].groupby('beca_extr_',dropna=False).agg(['unique']) # Comprobación

,tipo_beca_extr_
,unique
beca_extr_,
No,[nan]
Sí,"[Otros programas o becas fuera de la UE, Programa Erasmus, Otros programas o becas dentro de la UE, nan]"
NaN,[nan]


In [189]:
proporciones('tipo_beca_extr_')

NaN                                        0.894379
Programa Erasmus                           0.091687
Otros programas o becas fuera de la UE     0.008594
Otros programas o becas dentro de la UE    0.005339
Name: tipo_beca_extr_, dtype: float64

In [190]:
# Distribución de tipos de becas de estudios en el extranjero entre quienes manifestaron haber disfrutado alguna: 
proporciones('tipo_beca_extr_',dropna=True)

Programa Erasmus                           0.868083
Otros programas o becas fuera de la UE     0.081364
Otros programas o becas dentro de la UE    0.050553
Name: tipo_beca_extr_, dtype: float64

## 'EST_M3'  (redefinida como 'pais_beca_extr')  

(Solo para respuesta Sí (1) en variable 'EST_M1' (o 'extr')) 

**País de destino** cuando estudió fuera de España.

In [191]:
df['EST_M3'].sort_values().unique() # Missing values as '  ' or '99'

array(['  ', '01', '02', '03', '04', '05', '06', '07', '08', '11', '12',
       '21', '31', '41', '99'], dtype=object)

In [192]:
traduce('EST_M3')

                                                                 aux_traduce
                                                                      unique
EST_M3                                                                      
                                                              [No aplicable]
01                                                             [Reino Unido]
02                                                                [Alemania]
03                                                                 [Francia]
04                                                                  [Italia]
05                                                                [Portugal]
06      [Países Nórdicos (Suecia, Noruega, Finlandia, Dinamarca e Islandia)]
07                                           [Otro país de la Unión Europea]
08                                                       [Otro país europeo]
11                                            [Un país de América del Norte]

In [193]:
df[['extr_','EST_M3_']].groupby('extr_',dropna=False).agg(['unique']) # Inspección

,EST_M3_
,unique
extr_,
No,[No aplicable]
Sí,"[Un país de América Central o América del Sur, Francia, Otro país europeo, Portugal, Italia, Alemania, Un país de América del Norte, Países Nórdicos (Suecia, Noruega, Finlandia, Dinamarca e Islandia), Reino Unido, Otro país de la Unión Europea, Un país de Asia, NS/NC, Un país de África, Un país ..."
NaN,[No aplicable]


In [194]:
# Se imputa None a respuestas en blanco y NS/NC: 
no_respuesta('EST_M3',blank='  ',nsnc='99')

[ 1.  2.  3.  4.  5.  6.  7.  8. 11. 12. 21. 31. 41. nan]


In [195]:
df[['extr_','EST_M3_']].groupby('extr_',dropna=False).agg(['unique']) # Comprobación: 

,EST_M3_
,unique
extr_,
No,[nan]
Sí,"[Un país de América Central o América del Sur, Francia, Otro país europeo, Portugal, Italia, Alemania, Un país de América del Norte, Países Nórdicos (Suecia, Noruega, Finlandia, Dinamarca e Islandia), Reino Unido, Otro país de la Unión Europea, Un país de Asia, nan, Un país de África, Un país de..."
NaN,[nan]


In [196]:
redefine('EST_M3','pais_beca_extr')

In [197]:
# Distribución de países extranjeros donde se realiza una parte de los estudios entre quienes han estudiado fuera: 
proporciones('pais_beca_extr_',dropna=True)
# Italia y Reino Unido son los destinos más frecuentes (excluyendo la categoría 'Otros países de la UE')

Otro país de la Unión Europea                                         0.214326
Italia                                                                0.172715
Reino Unido                                                           0.127874
Francia                                                               0.089303
Alemania                                                              0.088543
Países Nórdicos (Suecia, Noruega, Finlandia, Dinamarca e Islandia)    0.076572
Un país de América Central o América del Sur                          0.057192
Portugal                                                              0.054722
Un país de América del Norte                                          0.049211
Otro país europeo                                                     0.047691
Un país de Asia                                                       0.015200
Un país de Oceanía                                                    0.004180
Un país de África                                   

## 'EST_M4'
(Solo para respuesta Sí (1) en variable 'EST_M1' (o 'extr'))  

**Duración** en meses del periodo en el que estudió fuera de España.

In [198]:
df['EST_M4'].unique() # Missing values as ' ' or '9'

array([' ', '1', '4', '3', '2', '5', '9'], dtype=object)

In [199]:
traduce('EST_M4')

               aux_traduce
                    unique
EST_M4                    
            [No aplicable]
1         [De 1 a 3 meses]
2         [De 4 a 6 meses]
3         [De 7 a 9 meses]
4       [De 10 a 12 meses]
5          [Más de un año]
9                  [NS/NC]


In [200]:
df[['extr_','EST_M4_']].groupby('extr_',dropna=False).agg(['unique']) # Inspección: 

,EST_M4_
,unique
extr_,
No,[No aplicable]
Sí,"[De 1 a 3 meses, De 10 a 12 meses, De 7 a 9 meses, De 4 a 6 meses, Más de un año, NS/NC]"
NaN,[No aplicable]


In [201]:
# Se imputa None a respuestas en blanco y NS/NC
no_respuesta('EST_M4')

[ 1.  2.  3.  4.  5. nan]


In [202]:
df[['extr_','EST_M4_']].groupby('extr_',dropna=False).agg(['unique']) # Comprobación: 

,EST_M4_
,unique
extr_,
No,[nan]
Sí,"[De 1 a 3 meses, De 10 a 12 meses, De 7 a 9 meses, De 4 a 6 meses, Más de un año, nan]"
NaN,[nan]


## 'EST_MES' 
¿Realizó parte de sus estudios para obtener su título de grado en **otra universidad española**? 1=Sí, 2=No, 9=NS/NC.  

In [203]:
df['EST_MES'].unique() # Missing values as '9'

array([2, 1, 9])

In [204]:
traduce('EST_MES')

        aux_traduce
             unique
EST_MES            
1              [Sí]
2              [No]
9           [NS/NC]


In [205]:
# Se imputa None a NS/NC:
no_respuesta('EST_MES',tipo='int')

[ 1.  2. nan]


In [206]:
proporciones('EST_MES_')

No     0.902626
Sí     0.096679
NaN    0.000695
Name: EST_MES_, dtype: float64

## 'EST_B8' (redefinida como 'motivacion')
**Motivo principal** por el que ha realizado sus **estudios** de grado.

In [207]:
df['EST_B8'].unique() # Missing values as '9'

array([1, 2, 3, 9])

In [208]:
traduce('EST_B8')

                                              aux_traduce
                                                   unique
EST_B8                                                   
1                [Formarse pensando en su futuro laboral]
2       [Ampliar conocimientos por satisfacción personal]
3                                         [Otros motivos]
9                                                 [NS/NC]


In [209]:
# Se imputa None a NS/NC:
no_respuesta('EST_B8',tipo='int')

[ 1.  2.  3. nan]


In [210]:
redefine('EST_B8','motivacion')

In [211]:
proporciones('motivacion_')

Formarse pensando en su futuro laboral             0.747938
Ampliar conocimientos por satisfacción personal    0.160816
Otros motivos                                      0.087612
NaN                                                0.003633
Name: motivacion_, dtype: float64

In [212]:
# Distribución de la motivación condicionada por la rama de conocimiento de la titulación (1=Artes y humanidades,
# ...  2=Ciencias, 3=Ciencias sociales y jurídicas, 4=Ingeniería y arquitectura, 5=Ciencias de la salud):
dist_condicionada('motivacion_','rama_g1_',dropna=True)
# La motivación principal son las expectativas laborales para todas las ramas de conocimiento.
# Artes y humanidades es la rama que menos se escoge pensando en las expectativas laborales, seguida por Ciencias.
# También son estas, en ese orden, las que más se escogen para ampliar conocimientos por satisfacción personal. 

## 'SAT1' (redefinida como 'sat_est_u')
¿Si tuviera que empezar de nuevo **volvería a cursar estudios universitarios**?  1=Sí, 2=No, 9=NS/NC.  

Nota: Se modifica en 'sat_est_u' el valor asociado a la respuesta No: 0=No.

In [213]:
df['SAT1'].unique() # Missing values as '9'

array([1, 2, 9])

In [214]:
traduce('SAT1')

     aux_traduce
          unique
SAT1            
1           [Sí]
2           [No]
9        [NS/NC]


In [215]:
# Se imputa None a NS/NC:
no_respuesta('SAT1',tipo='int')

[ 1.  2. nan]


In [216]:
redefine('SAT1','sat_est_u',si_no=True)

          aux_traduce
               unique
sat_est_u            
0.0              [No]
1.0              [Sí]


In [217]:
proporciones('sat_est_u_')

Sí     0.857919
No     0.126410
NaN    0.015671
Name: sat_est_u_, dtype: float64

In [218]:
# Distribución de la disposición a estudiar si tuviera que hacerlo de nuevo según ...
# ... la rama de conocimiento de la titulación:
dist_condicionada('sat_est_u_','rama_g1_',dropna=True)
# La disposición a estudiar de nuevo es similar para todas las ramas excepto para los graduados ...
# ... en ciencias de la salud, más dispuestos a estudiar de nuevo que el resto. 

In [219]:
# Distribución de la disposición a estudiar si tuviera que hacerlo de nuevo según el sexo: 
dist_condicionada('sat_est_u_','fem_',dropna=True)
# Las diferencias entre sexos son pequeñas. 

In [220]:
# Distribución de la disposición a estudiar si tuviera que hacerlo de nuevo según el discapacidad: 
dist_condicionada('sat_est_u_','disca_',dropna=True)
# Las diferencias según discapacidad son pequeñas. 

In [221]:
# Distribución de la disposición a estudiar según las motivaciones para estudiar un grado: 
dist_condicionada('sat_est_u_','motivacion_',dropna=True)
# La disposición a estudiar de nuevo más alta se da entre quienes estudiaron ... 
# ... principalmente para ampliar conocimientos por satisfacción personal. 

## 'SAT2' (redefinida como 'sat_titulo')  
(Solo para respuesta Sí (1) en 'SAT1' (o 'sat_est_u')) 

¿Si tuviera que empezar de nuevo **volvería a estudiar la misma titulación**?  1=Sí, 2=No, 9=NS/NC.  

Nota: Se modifica en 'sat_titulo' el valor asociado a la respuesta No: 0=No.

In [222]:
df['SAT2'].unique() # Missing values as ' ' or '9'

array(['2', '1', ' ', '9'], dtype=object)

In [223]:
traduce('SAT2')

         aux_traduce
              unique
SAT2                
      [No aplicable]
1               [Sí]
2               [No]
9            [NS/NC]


In [224]:
df[['SAT1_','SAT2_']].groupby('SAT1_',dropna=False).agg(['unique']) # Imspección

,SAT2_
,unique
SAT1_,
No,[No aplicable]
Sí,"[No, Sí, NS/NC]"
NaN,[No aplicable]


In [225]:
# Se imputa respuesta No (2) a quienes declararon que no volverían a cursar estudios universitarios: 
df['SAT2'].mask(df['sat_est_u']==0,2,inplace=True)
# Se imputa None a resto de respuestas en blanco y NS/NC:
no_respuesta('SAT2')

[ 1.  2. nan]


In [226]:
df[['SAT1_','SAT2_']].groupby('SAT1_',dropna=False).agg(['unique'])

,SAT2_
,unique
SAT1_,
No,[No]
Sí,"[No, Sí, nan]"
NaN,[nan]


In [227]:
redefine('SAT2','sat_titulo')

In [228]:
proporciones('sat_titulo_'), proporciones ('sat_est_u_')
# La disposición a estudiar de nuevo algún grado universitario es muy superior a la disposición a ...
# ... volver a elegir los mismos estudios. 

(Sí     0.615115
 No     0.365612
 NaN    0.019273
 Name: sat_titulo_, dtype: float64,
 Sí     0.857919
 No     0.126410
 NaN    0.015671
 Name: sat_est_u_, dtype: float64)

In [229]:
# Distribución de la disposición a estudiar la misma titulación si tuviera que hacerlo de nuevo según ...
# ... la rama de conocimiento de la titulación:
dist_condicionada('sat_titulo_','rama_g1_',dropna=True)
# La mayor disposición a estudiar de nuevo la misma titulación se da entre los graduados en ciencias de la salud. 

In [230]:
# Distribución de la disposición a estudiar de nuevo la misma titulación según las motivaciones para ...
# ... estudiar un grado: 
dist_condicionada('sat_titulo_','motivacion_',dropna=True)
# Quienes estudiaron principalmente para ampliar conocimientos por satifacción personal se muestran más ...
# ... dispuestos a estudiar de nuevo la misma titulación. 

## Variables sobre otra formación realizada: variables  'EST_B11_1' a 'EST_B11_7'
Nota: Debe tenerse en cuenta que se pregunta sobre otros estudios completados, no sobre los iniciados, y que para los graduados recientes se pueden estar subestimando las titulaciones posteriores por no haber habido tiempo suficiente para terminarlas. 

## 'EST_B11_1'  (redefinida como 'varios_grad')
¿Ha completado **otros estudios de grado**, diplomatura, licenciatura o equivalente?  1=Sí, 2=No.  

In [231]:
df['EST_B11_1'].unique() # No missing values

array([1, 2])

In [232]:
traduce('EST_B11_1')

          aux_traduce
               unique
EST_B11_1            
1                [Sí]
2                [No]


In [233]:
redefine('EST_B11_1','varios_grad',si_no=True)

            aux_traduce
                 unique
varios_grad            
0                  [No]
1                  [Sí]


In [234]:
proporciones('varios_grad_')

No    0.765505
Sí    0.234495
Name: varios_grad_, dtype: float64

In [235]:
# Distribución de uno o varios grados según rama de conocimiento de la titulación:
dist_condicionada('varios_grad_','rama_g1_')
# Los graduados en ciencias sociales y jurídicas, y en ingeniería y arquitectura son los que ...
# ... tienen más de un grado con mayor frecuencia. 
# Los graduados en ciencias de la salud son los que con menos frecuencia tienen más de un grado, a pesar de que ...
# ... también son los que con más frecuencia manifiestan su disposición a estudiar si tuvieran que hacerlo ...
# ... de nuevo (el mismo u otro grado) (variables 'SAT1' y 'SAT2'). 

In [236]:
# Distribución de la disposición a estudiar si tuvieran que empezar de nuevo según se tengan uno o varios grados: 
dist_condicionada('sat_est_u_','varios_grad_')
# La disposición a estudiar si tuvieran que hacerlo de nuevo es algo mayor entre quienes tienen más de un grado. 

In [237]:
# Distribución de uno o varios grados según sexos:
dist_condicionada('varios_grad_','fem_')
# No hay diferencias significativas 

In [238]:
# Distribución de uno o varios grados según discapacidad:
dist_condicionada('varios_grad_','disca_')
# La proporción de graduados en más de una titulación es superior entre quienes declaran discapacidad reconocida.

In [239]:
# Distribución de uno o varios grados según las motivaciones para estudiar: 
dist_condicionada('varios_grad_','motivacion_',dropna=True)
# Tener más de un grado es más frecuente entre quienes estudiaron principalmente para ampliar conocimientos ...
# ... por satifacción personal que entre quienes estudiaron principalmente por las espectativas profesionales. 

## 'EST_B11_2'  (redefinida como 'master')   
¿Ha completado otros estudios de **máster universitario**?  1=Sí, 2=No.  

Nota: Se modifica en 'master' el valor asociado a la respuesta No: 0=No.

In [240]:
df['EST_B11_2'].unique() # No missing values

array([2, 1])

In [241]:
traduce('EST_B11_2')

          aux_traduce
               unique
EST_B11_2            
1                [Sí]
2                [No]


In [242]:
redefine('EST_B11_2','master',si_no=True)

       aux_traduce
            unique
master            
0             [No]
1             [Sí]


In [243]:
df[['varios_grad','master']].mean()
# La proporcion de graduados con un máster universitario ronda el 50 %. 
# La frecuencia de estudios de máster es el doble que la de una segunda titulación de grado. 

varios_grad    0.234495
master         0.474740
dtype: float64

In [244]:
# Distribución de estudios de máster universitario según uno o varios grados:
dist_condicionada('master_','varios_grad_')
# Es más frecuente tener un máster universitario entre quienes tienen solo un título de grado. 

In [245]:
# Distribución de estudios de máster universitario según las motivaciones para estudiar: 
dist_condicionada('master_','motivacion_',dropna=True)
# Diferencias pequeñas entre quienes declaran motivacion por expectativas profesionales y por ...
# ... adquirir conocimientos.

In [246]:
# Distribución de estudios de máster universitario según rama de conocimiento de la titulación:
dist_condicionada('master_','rama_g1_')
# Los graduados en ciencias y en artes y humanidades son quienes con mayor frecuencia tienen estudios de máster. 
# La menor frecuencia se da entre los graduados en ingeniería y arquitectura.

In [247]:
# Distribución de estudios de máster universitario según sexos:
dist_condicionada('master_','fem_')
# Diferencias por sexos muy pequeñas.

In [248]:
# Distribución del sexo según se tengan o no estudios de máster universitario:
dist_condicionada('master_','fem_',rev=True)
# Las mujeres también son mayoría entre los graduados. En consecuencia, el número de mujeres con estudios de ...
# ... máster es algo superior al de hombres. 

In [249]:
# Distribución de estudios de máster universitario según discapacidad:
dist_condicionada('master_','disca_')
# Entre los graduados, los que tienen discapacidad reconocida tienen estudios de máster con menos frecuencia ...
# ... que el resto (al contrario de lo que ocurre con la titulación en más de un grado universitario).

## 'EST_B11_3'   (redefinida como 'doctor')  
¿Ha completado otros estudios de **doctorado universitario**?  1=Sí, 2=No.  

Nota: Se modifica en 'doctor' el valor asociado a la respuesta No: 0=No.

In [250]:
df['EST_B11_3'].unique() # No missing values

array([2, 1])

In [251]:
traduce('EST_B11_3')

          aux_traduce
               unique
EST_B11_3            
1                [Sí]
2                [No]


In [252]:
redefine('EST_B11_3','doctor',si_no=True)

       aux_traduce
            unique
doctor            
0             [No]
1             [Sí]


In [253]:
df[['varios_grad','master','doctor']].mean()
# Tener un doctorado universitario es mucho menos frecuente que tener un máster universitario o más de un grado. 

varios_grad    0.234495
master         0.474740
doctor         0.015355
dtype: float64

In [254]:
# Distribución de estudios de doctorado según el sexo: 
dist_condicionada('doctor_','fem_')
# Tener un doctorado es más frecuente entre los hombres graduados que entre las mujeres graduadas. 
# La diferencia es suficiente para contrarestar que hay más mujeres entre los graduados, haciendo que ...
# ... la proporción de doctores sea mayor que la de doctoras. En efecto, (celda siguiente)

In [255]:
dist_condicionada('doctor_','fem_',rev=True)
# Entre los doctores, la proporción hombres es superior a la de mujeres. 

In [256]:
dist_condicionada('doctor_','disca_')
# La proporción de doctores es ligeramente inferior entre los estudiantes con discapacidad.

In [257]:
# Distribucion de estudios de doctorado según la motivación para estudiar: 
dist_condicionada('doctor_','motivacion_')
# La mayor propensión a terminar un doctorado se da entre los que declaran estudiar para adquirir conocimientos

In [258]:
# Distribucion de las motivaciones para estudiar según se tenga o no el título de doctor: 
dist_condicionada('doctor_','motivacion_',rev=True)
# La motivación para estudiar más frecuente de quienes terminan un doctorado son las expectativas laborales. 
# Sin embargo, esta motivación es menos frecuente entre los doctores que entre los no doctores. Ocurre lo ...
# ... contrario con la motivación de adquirir conocimientos por satisfacción personal. 

In [259]:
# Distribucion de estudios de doctorado según las ramas de conocimiento: 
dist_condicionada('doctor_','rama_g1_')
# La frecuencia más alta de doctores se da entre los graduados en ciencias. También son los que ...
# ... terminan un máster universitario con mayor frecuencia (véase la celda siguiente). 
# Los graduados en ciencias sociales y jurídicas con los menos propensos a terminar un doctorado. También ...
# ... son los segundos menos propensos a estudiar un máster universitario. 
# Para interpretar estos resultados debe tenerse en cuenta la mayor duración de los estudios de doctorado ...
# ... (incluyendo el tiempo dedicado a la tesis). Hace más probable que haya encuestados que cursan estos ...
# ... estudios sin haberlos concluido. 
# Con los datos disponibles no podemos inferir cuántos de los estudios de máster son habilitantes para el ...
# ... doctorado, que podrían considerarse estudios de doctorado no terminados. 

In [260]:
# Distribucion de estudios de máster universitario según las ramas de conocimiento: 
dist_condicionada('master_','rama_g1_')
# Para facilitar comparación con resultados de la celda anterior. 

## 'EST_B11_4'  (redefinida como 'est_art')
¿Ha completado otros **estudios artísticos superiores** (artes plásticas, diseño, música y danza, arte dramático o conservación y restauración de bienes culturales)?  1=Sí, 2=No.  

Nota: Se modifica en 'est_art' el valor asociado a la respuesta No: 0=No.

In [261]:
df['EST_B11_4'].unique() # No missing values

array([2, 1])

In [262]:
traduce('EST_B11_4')

          aux_traduce
               unique
EST_B11_4            
1                [Sí]
2                [No]


In [263]:
redefine('EST_B11_4','est_art',si_no=True)

        aux_traduce
             unique
est_art            
0              [No]
1              [Sí]


In [264]:
df[['varios_grad','master','doctor','est_art']].mean()
# Los estudios artísticos superiores son menos frecuentes que los de doctorado. 

varios_grad    0.234495
master         0.474740
doctor         0.015355
est_art        0.007993
dtype: float64

In [265]:
# Distribución de estudios artísticos superiores según las ramas de conocimiento: 
dist_condicionada('est_art_','rama_g1_')
# La frecuencia más alta corresponde a los graduados en artes y humanidades (la única superior al 1 %). 

In [266]:
# Distribución de las ramas de conocimiento según se tengan o no estudios artísticos superiores: 
dist_condicionada('est_art_','rama_g1_',rev=True)
# Sin embargo, la rama de conocimiento más frecuente entre quienes tienen estudios artísticos superiores ...
# ... son las ciencias sociales y jurídicas. Esto se debe a que representan casi la mitad de los ...
# ... graduados.

In [267]:
# Distribución de estudios artísticos superiores según las motivaciones para estudiar: 
dist_condicionada('est_art_','motivacion_')
# La menor frecuencia de estudios artísticos superiores corresponde a quienes estudian para mejorar sus ...
# ... expectativas profesionales. 

In [268]:
# Distribución de las motivaciones para estudiar según se tengan o no estudios artísticos superiores: 
dist_condicionada('est_art_','motivacion_',rev=True)
# La motivación mayoritaria para estudiar de quienes tienen estudios artísticos superiores son las ...
# ... expectativas profesionales, aunque en proporción inferior a la general. 

In [269]:
# Distribución de estudios artísticos superiores según el sexo: 
dist_condicionada('est_art_','fem_')
# La proporcion de estudios artísticos superiores entre los graduados es mayor para las mujeres 
# ... que para los hombres. En consecuencia, (celda siguiente)

In [270]:
# Distribución del sexo según se tengan o no estudios artísticos superiores: 
dist_condicionada('est_art_','fem_',rev=True)
# El número de mujeres casi dobla el de hombres entre quienes tienen estudios artísticos superiores. 

In [271]:
# Distribución de estudios artísticos superiores según discapacidad: 
dist_condicionada('est_art_','disca_')
# Diferencias no significativas

## 'EST_B11_5'   (redefinida como 'g_sup')
¿Ha completado algún **ciclo formativo de grado superior** (formación profesional, artes plásticas y diseño) o **enseñanzas deportivas de grado superior**?  1=Sí, 2=No.  

Nota: Se modifica en 'g_sup' el valor asociado a la respuesta No: 0=No.

In [272]:
df['EST_B11_5'].unique() # No missing values

array([2, 1])

In [273]:
traduce('EST_B11_5')

          aux_traduce
               unique
EST_B11_5            
1                [Sí]
2                [No]


In [274]:
redefine('EST_B11_5','g_sup',si_no=True)

      aux_traduce
           unique
g_sup            
0            [No]
1            [Sí]


In [275]:
df[['varios_grad','master','doctor','est_art','g_sup']].mean()
# La proporción de graduados con ciclos formativos o enseñanzas deportivas de grado superior es menor ...
# ... que la de un segundo título de grado y muy superior a la de estudios de doctorado. 

varios_grad    0.234495
master         0.474740
doctor         0.015355
est_art        0.007993
g_sup          0.128242
dtype: float64

In [276]:
# Distribución de ciclos formativos y enseñanzas deportivas de grado superior según el sexo: 
dist_condicionada('g_sup_','fem_')
# La proporcion de ciclos formativos y enseñanzas deportivas de grado superior entre los graduados es mayor ...
# ... para las mujeres que para los hombres. En consecuencia, y puesto que las mujeres son mayoria entre ...
# ... los graduados, (celda siguiente)

In [277]:
# Distribución del sexo según se tengan o no ciclos formativos y enseñanzas deportivas de grado superior: 
dist_condicionada('g_sup_','fem_',rev=True)
# El número de mujeres es superior al de hombres entre quienes tienen ciclos formativos o enseñanzas ...
# ... deportivas de grado superior.

In [278]:
# Distribución de ciclos formativos y enseñanzas deportivas de grado superior según discapacidad: 
dist_condicionada('g_sup_','disca_')
# La proporcion de ciclos formativos y enseñanzas deportivas de grado superior es mayor entre los graduados ...
# ... con discapacidad.

In [279]:
# Distribución de ciclos formativos y enseñanzas deportivas de grado superior según la rama de conocimiento: 
dist_condicionada('g_sup_','rama_g1_')
# Los graduados en ciencias de la salud y en ciencias sociales y jurídicas son los que con mayor frecuencia ...
# ... han completado ciclos formativos y enseñanzas deportivas de grado superior.

In [280]:
# Distribución de ciclos formativos y enseñanzas deportivas de grado superior según doctorado: 
dist_condicionada('g_sup_','doctor_')
# La presencia de graduados con ciclos formativos y enseñanzas deportivas de grado superior es más de tres ..
# ... veces superior entre los no doctores que entre los doctores. 

In [281]:
# Distribución de ciclos formativos y enseñanzas deportivas de grado superior según estudios de máster: 
dist_condicionada('g_sup_','master_')
# La presencia de graduados con ciclos formativos y enseñanzas deportivas de grado superior entre quienes ...
# ... no tienen un título de máster. 

In [282]:
# Distribución de las motivaciones para estudiar según se tengan o no ciclos formativos y enseñanzas ...
# ... deportivas de grado superior: 
dist_condicionada('g_sup_','motivacion_',rev=True)
# No hay diferencias muy notables. 

In [283]:
# Distribución de los ciclos formativos y enseñanzas deportivas de grado superior según la edad: 
dist_condicionada('g_sup_','edad_')
# El grupo de gradudados de edad de 30 a 34 es el que con mayor frecuencia tiene un título de ciclos formativos ...
# ... o enseñanzas deportivas de grado superior. 
# La diferencia de frecuencia es suficiente para contrarestar que los graduados de menos de 30 años son muchos ...
# ... más. En efecto, (celda siguiente)

In [284]:
dist_condicionada('g_sup_','edad_',rev=True)
# La edad más frecuente entre los graduados que tienen un título de ciclos formativos o enseñanzas deportivas ...
# ... de grado superior es el grupo de 30 a 34 años. 

## 'EST_B11_6'   (redefinida como 'g_medio')
¿Ha completado algún **ciclo formativo de grado medio** (formación profesional, artes plásticas y diseño) o **enseñanzas deportivas de grado medio**?  1=Sí, 2=No.  

Nota: Se modifica en 'g_medio' el valor asociado a la respuesta No: 0=No.

In [285]:
df['EST_B11_6'].unique() # No missing values

array([2, 1])

In [286]:
traduce('EST_B11_6')

          aux_traduce
               unique
EST_B11_6            
1                [Sí]
2                [No]


In [287]:
redefine('EST_B11_6','g_medio',si_no=True)

        aux_traduce
             unique
g_medio            
0              [No]
1              [Sí]


In [288]:
df[['varios_grad','master','doctor','est_art','g_sup','g_medio']].mean()
# Menos frecuentes que los mismos estudios de grado superior, más frecuentes que estudios artisticos superiores ...
# ... y que estudios de doctorado.

varios_grad    0.234495
master         0.474740
doctor         0.015355
est_art        0.007993
g_sup          0.128242
g_medio        0.033332
dtype: float64

In [289]:
# Distribución de los ciclos formativos y educación deportiva de grado medio según esos estudios en su ...
# ... grado superior: 
dist_condicionada('g_medio_','g_sup_')
# Casi el 90 % de los graduados universitarios que terminan ciclos formativos ...
# ... o educación deportiva de grado superior acceden a esos estudios sin haber cursado los de grado medio. 

In [290]:
# Distribución de los ciclos formativos y educación deportiva de grado medio según el sexo: 
dist_condicionada('g_medio_','fem_')
# Las distribuciones condicionadas por el sexo son casi idénticas.  

In [291]:
# Distribución de los ciclos formativos y educación deportiva de grado medio según discapacidad: 
dist_condicionada('g_medio_','disca_')
# Mayor proporción de estudios de ciclos de grado medio entre los graduados con discapacidad.

## 'EST_B11_7'     
No ha completado **ninguno de los estudios anteriores**: 1=Sí, 2=No.  

In [292]:
df['EST_B11_7'].unique() # No missing values

array([2, 1])

In [293]:
traduce('EST_B11_7',save=False)

          aux_traduce
               unique
EST_B11_7            
1                [Sí]
2                [No]


In [294]:
df[['varios_grad','master','doctor','est_art','g_sup','g_medio','EST_B11_7']].groupby(
        ['EST_B11_7']).agg(['unique'])
# Comprobación: solo toma valor 1 cuando todas las variables de otros estudios anteriores toman valor 0. 

,varios_grad,master,doctor,est_art,g_sup,g_medio
,unique,unique,unique,unique,unique,unique
EST_B11_7,,,,,,
1,[0],[0],[0],[0],[0],[0]
2,"[1, 0]","[0, 1]","[0, 1]","[0, 1]","[0, 1]","[0, 1]"


## 'EST_B12' 
(Solo para quienes declaran haber terminado otros estudios de grado, diplomatura, licenciatura o equivalente -respuesta 1 en variable 'EST_B11_1' (o 'varios_grad')-)  

**Número de otros estudios de grado**, diplomatura, licenciatura o equivalente (aparte de aquel al que se refiere la encuesta).

In [295]:
df['EST_B12'].unique() # Missing values as ' '

array(['2', '1', ' ', '3'], dtype=object)

In [296]:
traduce('EST_B12')

            aux_traduce
                 unique
EST_B12                
         [No aplicable]
1                 [Uno]
2                [Dos ]
3          [Tres o más]


In [297]:
df[['varios_grad_','EST_B12_']].groupby('varios_grad_').agg(['unique']) # Inspección

,EST_B12_
,unique
varios_grad_,
No,[No aplicable]
Sí,"[Dos , Uno, Tres o más]"


##  Nueva variable 'n_grad'     
**Número de títulos de grado**, diplomatura, licenciatura o equivalente (incluyendo aquella que motiva esta encuesta).

In [298]:
df['n_grad']=df['EST_B12'].replace({' ':0}).astype('int')+1
df[['n_grad','EST_B12']].groupby('n_grad').agg(['unique']) # Comprobación

,EST_B12
,unique
n_grad,
1,[ ]
2,[1]
3,[2]
4,[3]


In [299]:
D_describe['n_grad']='Número de títulos de grado, diplomatura, licenciatura o equivalente'
a_lista('n_grad')

In [300]:
proporciones('n_grad')

1    0.765505
2    0.204133
3    0.025181
4    0.005182
Name: n_grad, dtype: float64

In [301]:
# Distribución de doctores y no doctores según el número de títulos de grado:  
dist_condicionada('n_grad','doctor_',rev=True)
# Terminar los estudios de doctorado es más frecuente cuanto mayor es el número de títulos de grado. 

In [302]:
# Distribución de número de grados universitarios según se tenga o no un título de doctor: 
dist_condicionada('n_grad','doctor_')
# Los doctores tienen más de un grado universitario con más frecuencia que los no doctores. En concreto, ...
# ... ser doctor multiplica por más de 2 la probabilidad de tener 3 grados y por 3 la de tener 4 o más títulos ...
# ... de grado (incluyendo aquel al que se refiere la encuesta). 

In [303]:
# Distribución de títulos de máster universitario según el número de títulos de grado:  
dist_condicionada('n_grad','master_',rev=True)
# Los titulados en un único grado universitario son los que con más frecuencia terminan estudios de máster ...
# ... universitario. Les siguen quienes tienen 4 o más títulos de grado. 

In [304]:
# Distribución de número de grados universitarios según se tenga o no un título de máster universitario: 
dist_condicionada('n_grad','master_')
# Quienes terminan un máster universitario tienen solo un título de grado con más frecuencia que quienes ...
# ... no tienen un título de máster. 

In [305]:
# Distribución de número de grados universitarios según el sexo: 
dist_condicionada('n_grad','fem_')
# Las diferencias entre sexos son pequeñas. 

In [306]:
# Distribución de número de grados universitarios según la edad: 
dist_condicionada('n_grad','edad_')
# Como es esperabe, a medida que aumenta la edad aumenta también la frecuencia de titulados en un número ...
# ... mayor de grados. 

In [307]:
# Distribución de número de grados universitarios según discapacidad: 
dist_condicionada('n_grad','disca_')
# La proporción de graduados en 3 o 4 titulaciones es muy superior entre quienes tienen discapacidad.

## 'EST_B13_AMB1' (redefinida como 'ambito_g2')
(Solo para quienes declaran haber terminado otros estudios de grado, diplomatura, licenciatura o equivalente -respuesta 1 en variable 'EST_B11_1' (o 'varios_grad')-) 

**Ambito de estudio del segundo grado**, diplomatura, licenciatura o equivalente en el que está titulado.

In [308]:
reset('EST_B13_AMB1')

In [309]:
df['EST_B13_AMB1'].sort_values().unique() # Missing values as '    ' or '9999'

array(['    ', '011 ', '0112', '0113', '021 ', '0211', '022 ', '023 ',
       '031 ', '0311', '0313', '032 ', '041 ', '0413', '042 ', '051 ',
       '052 ', '053 ', '054 ', '061 ', '071 ', '072 ', '073 ', '081 ',
       '082 ', '084 ', '091 ', '0912', '0913', '092 ', '1014', '1015',
       '109 ', '9999'], dtype=object)

In [310]:
df['EST_B13_AMB1'].nunique()

34

In [311]:
df['EST_B13_AMB1']=df['EST_B13_AMB1'].str.strip()
traduce('EST_B13_AMB1')

                                                             aux_traduce
                                                                  unique
EST_B13_AMB1                                                            
                                                                      []
011                                        [Educación (Otros estudios) ]
0112                      [Formación de docentes de enseñanza infantil ]
0113                      [Formación de docentes de enseñanza primaria ]
021                                            [Artes (Otros estudios) ]
0211                  [Técnicas audiovisuales y medios de comunicación ]
022                                                       [Humanidades ]
023                                                           [Lenguas ]
031           [Ciencias sociales y del comportamiento (Otros estudios) ]
0311                                                         [Economía ]
0313                                               

In [312]:
def re_otros_estudios(col,nuevo='',n=0,blank=' ',nsnc='9',trad=True):
    """Inspección e imputación de valores de col. condicionadas por la respuesta sobre los estudios otros_estudios
    Si se especifica un valor n y un nombre 'nuevo', crea una variable 'nuevo' para la que define 
    valor 0 = No tiene n títulos de otros_estudios. Se añade la variable nueva a la lista de variables para 
    modelos descriptivos o predicctivos.
    """
    # Se imputa None a respuestas en blanco y NS/NC:
    try: df[col].replace({blank:None,nsnc:None},inplace=True)
    except: 
        df[col]=df[col].astype('O')
        df[col].replace({blank:None,nsnc:None},inplace=True) # Resuelve conflictos cuando se aplica varias veces
    try: df[col]=df[col].astype('float')
    except:pass
    if trad==True: traduce(col,check=False)
    if len(nuevo)>0 and n>0:
        df[nuevo]=df[col]
        D_dict[nuevo]=D_dict[col]
        traduce(nuevo,check=False)
        print(df[[str(nuevo)+'_',otros_estudios]].groupby(otros_estudios).agg(['unique']))
        # Se imputa valor 0 a quienes tienen menos de n títulos: 
        df[nuevo].mask(df[otros_estudios]<n,0,inplace=True)
        traduce(nuevo,check=False)
        print(df[[str(nuevo)+'_',otros_estudios]].groupby(otros_estudios).agg(['unique']))
        if str(nuevo) not in v_modelos:
            v_modelos.append(nuevo)
        if str(nuevo)+'_' not in v_modelos_L:
            v_modelos_L.append(str(nuevo)+'_')
    else: print (df[col].sort_values().unique())
    return

In [313]:
# Se establece la variable otros_estudios referida a otros estudios de grado: 
otros_estudios='n_grad'

In [314]:
re_otros_estudios('EST_B13_AMB1',nuevo='ambito_g2',n=2,blank=None,nsnc='9999')

                                                                                                                                                                                                                                                                                                         ambito_g2_
                                                                                                                                                                                                                                                                                                             unique
n_grad                                                                                                                                                                                                                                                                                                             
1                                                                           

In [315]:
D_dict['ambito_g2']=D_dict['ambito_g1']
D_dict['ambito_g2'][0]='No tiene'
traduce('ambito_g2',check=False)

In [316]:
# El ámbito de estudio es menos manejable (muchas categorías) que la rama de conocimiento (5 valores). 
# Para identificar la rama que corresponde a cada ámbito: 
df[['rama_g1_','ambito_g1_']].groupby('ambito_g1_').agg(['value_counts']).head(15)

rama_g1_
                                                                                       value_counts
ambito_g1_                                               rama_g1_                                  
Actividades físicas y deportivas                         Ciencias sociales y jurídicas          519
                                                         Ciencias de la salud                    84
Agricultura y ganadería                                  Ingeniería y arquitectura              545
Arquitectura y construcción                              Ingeniería y arquitectura             1541
                                                         Artes y humanidades                      1
Artes (Otros estudios)                                   Artes y humanidades                    887
                                                         Ingeniería y arquitectura               19
Ciencias de la vida                                      Ciencias                               882
                                                         Ciencias de la salud                    61
Ciencias químicas, físicas y geológicas                  Ciencias                               761
                                                         Ciencias sociales y jurídicas           63
                                                         Artes y humanidades                     38
                                                         Ingeniería y arquitectura               26
Ciencias sociales y del comportamiento (Otros estudios)  Ciencias sociales y jurídicas          547
                                                         Artes y humanidades                    160

### DataFrame auxiliar df_ambito_to_rama
Dataframe auxiliar para identificar la rama de conocimiento asociada a cada ámbito de estudios.  

In [317]:
# Cuando hay varias ramas asociadas a un ámbito, elegimos la rama más frecuente (posición 1 en ranking): 
df_ambito_rama=df[['rama_g1','ambito_g1']].groupby('ambito_g1').agg(['value_counts']
                    ).groupby(['ambito_g1']).rank(ascending=False).reset_index()
df_ambito_rama.head(10)

ambito_g1 rama_g1             
                    value_counts
0        11       3          1.0
1        11       1          2.0
2        21       1          1.0
3        21       4          2.0
4        22       1          1.0
5        23       1          1.0
6        23       3          2.0
7        31       3          1.0
8        31       1          2.0
9        32       3          1.0

In [318]:
# Ajuste de índices: de multiíndice a índices simples: 
df_ambito_rama.columns=df_ambito_rama.columns.to_flat_index()
df_ambito_rama.columns

Index([('ambito_g1', ''), ('rama_g1', ''), ('rama_g1', 'value_counts')], dtype='object')

In [319]:
# Ajuste de índices: cambio de denominación de columnas: 
df_ambito_rama.rename(columns={df_ambito_rama.columns[0]:'ambito',
                               df_ambito_rama.columns[1]:'rama',
                               df_ambito_rama.columns[2]:'rank'},inplace=True)
df_ambito_rama.columns

Index(['ambito', 'rama', 'rank'], dtype='object')

In [320]:
# Ajuste de índices: columna 'ambito' como índice
df_ambito_rama.set_index('ambito',inplace=True)
df_ambito_rama.head(10)

,rama,rank
ambito,,
11,3,1.0
11,1,2.0
21,1,1.0
21,4,2.0
22,1,1.0
23,1,1.0
23,3,2.0
31,3,1.0
31,1,2.0


In [321]:
# Selección de la rama más frecuente para cada ámbito: 
df_ambito_rama=df_ambito_rama[df_ambito_rama['rank']==1]
del df_ambito_rama['rank']
df_ambito_rama.head(10)

,rama
ambito,
11,3
21,1
22,1
23,1
31,3
32,3
41,3
42,3
51,2


In [322]:
# Para transformar el dataset auxiliar en un diccionario: 
D_ambito_to_rama=dict()
for ambito in df_ambito_rama.index:
    D_ambito_to_rama[ambito]=df_ambito_rama['rama'].loc[ambito]
del df_ambito_rama

In [323]:
D_ambito_to_rama.keys(), D_ambito_to_rama.values()

(dict_keys([11, 21, 22, 23, 31, 32, 41, 42, 51, 52, 53, 54, 61, 71, 72, 73, 81, 82, 84, 91, 92, 109, 112, 113, 211, 311, 313, 413, 912, 913, 1014, 1015]),
 dict_values([3, 1, 1, 1, 3, 3, 3, 3, 2, 2, 2, 2, 4, 4, 4, 4, 4, 4, 5, 5, 3, 4, 3, 3, 3, 3, 5, 3, 5, 5, 3, 3]))

## Nueva variable 'rama_g2'
**Rama de conocimiento de la segunda titulación de grado** universitario.

In [324]:
# Columna nueva: rama de conocimiento asociada al ámbito declarado: 
df['rama_g2']=df['ambito_g2'].astype('float').replace(D_ambito_to_rama)

In [325]:
df['rama_g2'].unique()

array([ 3.,  0.,  5., nan,  1.,  4.,  2.])

In [326]:
D_dict['rama_g2']=D_dict['rama_g1']
D_dict['rama_g2'][0]='No tiene'
traduce('rama_g2')

                             aux_traduce
                                  unique
rama_g2                                 
0.0                           [No tiene]
1.0                [Artes y humanidades]
2.0                           [Ciencias]
3.0      [Ciencias sociales y jurídicas]
4.0          [Ingeniería y arquitectura]
5.0               [Ciencias de la salud]


In [327]:
df[['EST_B13_AMB1_','rama_g2_']].groupby('EST_B13_AMB1_').agg(['unique']).head() # Comprobación 

,rama_g2_
,unique
EST_B13_AMB1_,
,[No tiene]
Actividades físicas y deportivas,[Ciencias sociales y jurídicas]
Agricultura y ganadería,[Ingeniería y arquitectura]
Arquitectura y construcción,[Ingeniería y arquitectura]
Artes (Otros estudios),[Artes y humanidades]


In [328]:
# Comprobación:
df[['ambito_g2_','rama_g2_','EST_B13_AMB1_']].count()

ambito_g2_       31570
rama_g2_         31570
EST_B13_AMB1_    31570
dtype: int64

In [329]:
df['EST_B13_AMB1_'][df['EST_B13_AMB1_']=='NS/NC'].count(), df['ambito_g2_'].count()-df['EST_B13_AMB1_'].count()
# La diferencia en el número de registros se debe a los NS/NC transformados en Nan.

(0, 0)

## 'EST_B14_1'     
(Solo para quienes declaran haber terminado otros estudios de grado, diplomatura, licenciatura o equivalente -respuesta 1 en variable 'EST_B11_1' (o 'varios_grad')-)   

**Momento en que terminó la segunda titulación de grado**, diplomatura, licenciatura o equivalente mencionada.

In [330]:
df['EST_B14_1'].sort_values().unique() # Missing values as ' ' or '9'

array([' ', '1', '2', '3', '9'], dtype=object)

In [331]:
traduce('EST_B14_1',save=False)

                                                                              aux_traduce
                                                                                   unique
EST_B14_1                                                                                
                                                                           [No aplicable]
1                  [Antes de comenzar los estudios de la titulación que finalizó en 2014]
2          [Durante la realización de los estudios de la titulación que finalizó en 2014]
3                                 [Después de obtener la titulación que finalizó en 2014]
9                                                                                 [NS/NC]


In [332]:
re_otros_estudios('EST_B14_1',trad=False)

[ 1.  2.  3. nan]


## 'EST_B15_1'     
(Solo para quienes declaran haber terminado otros estudios de grado, diplomatura, licenciatura o equivalente -respuesta 1 en variable 'EST_B11_1' (o 'varios_grad')-) 

**Dónde estudió la segunda titulación de grado**, diplomatura, licenciatura o equivalente mencionada.

In [333]:
df['EST_B15_1'].sort_values().unique() # Missing values as ' ' or '9'

array([' ', '1', '2', '3', '9'], dtype=object)

In [334]:
traduce('EST_B15_1')

                                                   aux_traduce
                                                        unique
EST_B15_1                                                     
                                                [No aplicable]
1                                    [Completamente en España]
2          [Una parte en España y otra parte en el extranjero]
3                             [Completamente en el extranjero]
9                                                      [NS/NC]


In [335]:
re_otros_estudios('EST_B15_1')

[ 1.  2.  3. nan]


## Nueva variable: 'extr_proxy'
Variable proxy de **estudios al menos en parte en el extranjero**. 

In [336]:
df['extr_proxy']=0
df['extr_proxy'].mask((df['EST_B15_1']==2)|(df['EST_B15_1']==3),1,inplace=True)
df[['EST_B15_1_','extr_proxy']].groupby('EST_B15_1_',dropna=False).agg(['unique']) # Comprobación

,extr_proxy
,unique
EST_B15_1_,
Completamente en España,[0]
Completamente en el extranjero,[1]
Una parte en España y otra parte en el extranjero,[1]
NaN,[0]


In [337]:
D_describe['extr_proxy']='Cursó al menos parte de sus estudios en el extranjero'
D_dict['extr_proxy']={0:'No estudió fuera de España',1:'Al menos parte de los estudios en el extranjero'}
traduce('extr_proxy')

                                                  aux_traduce
                                                       unique
extr_proxy                                                   
0                                [No estudió fuera de España]
1           [Al menos parte de los estudios en el extranjero]


In [338]:
def actualiza_extr_proxy(col):
    """Actualiza la variable 'extr_proxy' añadiendo los estudios en el extranjero declarados en la variable col. 
    """
    df['extr_proxy'].mask((df[col]==2)|(df[col]==3),1,inplace=True)
    traduce('extr_proxy')
    print (df[[str(col),'extr_proxy_']].groupby(str(col),dropna=False).agg(['unique'])) # Comprobación
    return

## 'EST_B13_AMB2' (redefinida como 'ambito_g3')     
(Solo para quienes declaran haber terminado otros estudios de grado, diplomatura, licenciatura o equivalente -respuesta 1 en variable 'EST_B11_1' (o 'varios_grad')-) 

**Ambito de estudio del tercer grado**, diplomatura, licenciatura o equivalente en el que está titulado.

In [339]:
df['EST_B13_AMB2'].sort_values().unique() # Missing values as '    ' or '9999'

array(['    ', '011 ', '0112', '0113', '021 ', '0211', '022 ', '023 ',
       '031 ', '0311', '0313', '032 ', '041 ', '0413', '042 ', '051 ',
       '052 ', '053 ', '054 ', '061 ', '071 ', '072 ', '073 ', '081 ',
       '082 ', '084 ', '091 ', '0912', '0913', '092 ', '1014', '1015',
       '109 ', '9999'], dtype=object)

In [340]:
df['EST_B13_AMB2']=df['EST_B13_AMB2'].str.strip()
traduce('EST_B13_AMB2')

                                                             aux_traduce
                                                                  unique
EST_B13_AMB2                                                            
                                                                      []
011                                        [Educación (Otros estudios) ]
0112                      [Formación de docentes de enseñanza infantil ]
0113                      [Formación de docentes de enseñanza primaria ]
021                                            [Artes (Otros estudios) ]
0211                  [Técnicas audiovisuales y medios de comunicación ]
022                                                       [Humanidades ]
023                                                           [Lenguas ]
031           [Ciencias sociales y del comportamiento (Otros estudios) ]
0311                                                         [Economía ]
0313                                               

In [341]:
re_otros_estudios('EST_B13_AMB2','ambito_g3',3,blank='',nsnc='9999')

                                                                                                                                                                                                                                                                                                         ambito_g3_
                                                                                                                                                                                                                                                                                                             unique
n_grad                                                                                                                                                                                                                                                                                                             
1                                                                           

In [342]:
D_dict['ambito_g3']=D_dict['ambito_g2']
traduce('ambito_g3',check=False)

## Nueva variable 'rama_g3'
**Rama de conocimiento de la tercera titulación de grado** universitario.

In [343]:
# Columna nueva: rama de conocimiento asociada al ámbito declarado: 
df['rama_g3']=df['ambito_g3'].replace(D_ambito_to_rama)

In [344]:
df['rama_g3'].unique()

array([ 3.,  0.,  1.,  4.,  5., nan,  2.])

In [345]:
D_dict['rama_g3']=D_dict['rama_g2']
traduce('rama_g3')

                             aux_traduce
                                  unique
rama_g3                                 
0.0                           [No tiene]
1.0                [Artes y humanidades]
2.0                           [Ciencias]
3.0      [Ciencias sociales y jurídicas]
4.0          [Ingeniería y arquitectura]
5.0               [Ciencias de la salud]


## 'EST_B14_2'     
(Solo para quienes declaran haber terminado otros estudios de grado, diplomatura, licenciatura o equivalente -respuesta 1 en variable 'EST_B11_1' (o 'varios_grad')-)  

**Momento en que terminó la tercera titulación de grado**, diplomatura, licenciatura o equivalente mencionada.

In [346]:
df['EST_B14_2'].sort_values().unique() # Missing values as ' ' or '9'

array([' ', '1', '2', '3', '9'], dtype=object)

In [347]:
traduce('EST_B14_2')

                                                                              aux_traduce
                                                                                   unique
EST_B14_2                                                                                
                                                                           [No aplicable]
1                  [Antes de comenzar los estudios de la titulación que finalizó en 2014]
2          [Durante la realización de los estudios de la titulación que finalizó en 2014]
3                                 [Después de obtener la titulación que finalizó en 2014]
9                                                                                 [NS/NC]


In [348]:
re_otros_estudios('EST_B14_2')

[ 1.  2.  3. nan]


In [349]:
proporciones('EST_B14_2_',dropna=True)

Antes de comenzar los estudios de la titulación que finalizó en 2014            0.600000
Después de obtener la titulación que finalizó en 2014                           0.213542
Durante la realización de los estudios de la titulación que finalizó en 2014    0.186458
Name: EST_B14_2_, dtype: float64

## 'EST_B15_2'     
(Solo para quienes declaran haber terminado otros estudios de grado, diplomatura, licenciatura o equivalente -respuesta 1 en variable 'EST_B11_1' (o 'varios_grad')-) 

**Dónde estudió la tercera titulación de grado**, diplomatura, licenciatura o equivalente mencionada.

In [350]:
df['EST_B15_2'].sort_values().unique() # Missing values as ' ' or '9'

array([' ', '1', '2', '3', '9'], dtype=object)

In [351]:
traduce('EST_B15_2')

                                                   aux_traduce
                                                        unique
EST_B15_2                                                     
                                                [No aplicable]
1                                    [Completamente en España]
2          [Una parte en España y otra parte en el extranjero]
3                             [Completamente en el extranjero]
9                                                      [NS/NC]


In [352]:
re_otros_estudios('EST_B15_2')

[ 1.  2.  3. nan]


In [353]:
actualiza_extr_proxy('EST_B15_2') 

                                                  aux_traduce
                                                       unique
extr_proxy                                                   
0                                [No estudió fuera de España]
1           [Al menos parte de los estudios en el extranjero]
                                                                             extr_proxy_
                                                                                  unique
EST_B15_2                                                                               
1.0        [No estudió fuera de España, Al menos parte de los estudios en el extranjero]
2.0                                    [Al menos parte de los estudios en el extranjero]
3.0                                    [Al menos parte de los estudios en el extranjero]
NaN        [No estudió fuera de España, Al menos parte de los estudios en el extranjero]


## 'EST_B13_AMB3' (redefinida como 'ambito_g4') 
(Solo para quienes declaran haber terminado otros estudios de grado, diplomatura, licenciatura o equivalente -respuesta 1 en variable 'EST_B11_1' (o 'varios_grad')-) 

**Ambito de estudio del cuarto grado**, diplomatura, licenciatura o equivalente en el que está titulado.

In [354]:
reset('EST_B13_AMB3')

In [355]:
df['EST_B13_AMB3'].sort_values().unique() # Missing values as '    ' or '9999'

array(['    ', '011 ', '0112', '0113', '021 ', '022 ', '023 ', '031 ',
       '0311', '032 ', '041 ', '0413', '042 ', '054 ', '061 ', '071 ',
       '072 ', '073 ', '082 ', '091 ', '0913', '1014', '109 ', '9999'],
      dtype=object)

In [356]:
df['EST_B13_AMB3']=df['EST_B13_AMB3'].str.strip()
traduce('EST_B13_AMB3')

                                                             aux_traduce
                                                                  unique
EST_B13_AMB3                                                            
                                                                      []
011                                        [Educación (Otros estudios) ]
0112                      [Formación de docentes de enseñanza infantil ]
0113                      [Formación de docentes de enseñanza primaria ]
021                                            [Artes (Otros estudios) ]
022                                                       [Humanidades ]
023                                                           [Lenguas ]
031           [Ciencias sociales y del comportamiento (Otros estudios) ]
0311                                                         [Economía ]
032                                        [Periodismo y documentación ]
041                        [Negocios y administraci

In [357]:
re_otros_estudios('EST_B13_AMB3','ambito_g4',4,blank='',nsnc='9999')

                                                                                                                                                                                                                                                                                                         ambito_g4_
                                                                                                                                                                                                                                                                                                             unique
n_grad                                                                                                                                                                                                                                                                                                             
1                                                                           

In [358]:
D_dict['ambito_g4']=D_dict['ambito_g2']
traduce('ambito_g4',check=False)

## Nueva variable 'rama_g4'
**Rama de conocimiento de la cuarta titulación de grado** universitario.

In [359]:
# Columna nueva: rama de conocimiento asociada al ámbito declarado: 
df['rama_g4']=df['ambito_g4'].replace(D_ambito_to_rama)

In [360]:
df['rama_g4'].unique()

array([ 0.,  3., nan,  1.,  5.,  4.,  2.])

In [361]:
D_dict['rama_g4']=D_dict['rama_g2']
traduce('rama_g4')

                             aux_traduce
                                  unique
rama_g4                                 
0.0                           [No tiene]
1.0                [Artes y humanidades]
2.0                           [Ciencias]
3.0      [Ciencias sociales y jurídicas]
4.0          [Ingeniería y arquitectura]
5.0               [Ciencias de la salud]


## 'EST_B14_3'     
(Solo para quienes declaran haber terminado otros estudios de grado, diplomatura, licenciatura o equivalente -respuesta 1 en variable 'EST_B11_1' (o 'varios_grad')-) 

**Momento en que terminó la cuarta titulación de grado**, diplomatura, licenciatura o equivalente mencionada.

In [362]:
df['EST_B14_3'].sort_values().unique() # Missing values as ' ' or '9'

array([' ', '1', '2', '3'], dtype=object)

In [363]:
traduce('EST_B14_3')

                                                                              aux_traduce
                                                                                   unique
EST_B14_3                                                                                
                                                                           [No aplicable]
1                  [Antes de comenzar los estudios de la titulación que finalizó en 2014]
2          [Durante la realización de los estudios de la titulación que finalizó en 2014]
3                                 [Después de obtener la titulación que finalizó en 2014]


In [364]:
re_otros_estudios('EST_B14_3')

[ 1.  2.  3. nan]


## 'EST_B15_3'     
(Solo para quienes declaran haber terminado otros estudios de grado, diplomatura, licenciatura o equivalente -respuesta 1 en variable 'EST_B11_1' (o 'varios_grad')-)   

**Dónde estudió la cuarta titulación de grado**, diplomatura, licenciatura o equivalente mencionada.

In [365]:
df['EST_B15_3'].sort_values().unique() # Missing values as ' ' or '9'

array([' ', '1', '2', '3'], dtype=object)

In [366]:
traduce('EST_B15_3')

                                                   aux_traduce
                                                        unique
EST_B15_3                                                     
                                                [No aplicable]
1                                    [Completamente en España]
2          [Una parte en España y otra parte en el extranjero]
3                             [Completamente en el extranjero]


In [367]:
re_otros_estudios('EST_B15_3')

[ 1.  2.  3. nan]


In [368]:
actualiza_extr_proxy('EST_B15_3')

                                                  aux_traduce
                                                       unique
extr_proxy                                                   
0                                [No estudió fuera de España]
1           [Al menos parte de los estudios en el extranjero]
                                                                             extr_proxy_
                                                                                  unique
EST_B15_3                                                                               
1.0        [Al menos parte de los estudios en el extranjero, No estudió fuera de España]
2.0                                    [Al menos parte de los estudios en el extranjero]
3.0                                    [Al menos parte de los estudios en el extranjero]
NaN        [No estudió fuera de España, Al menos parte de los estudios en el extranjero]


In [369]:
L_ordinal=['primer','segundo','tercer','cuarto']
for i in range(1,5):
    D_describe['rama_g'+str(i)]='Rama de conocimiento del '+L_ordinal[i-1]+ ' titulo de grado'
    D_describe['ambito_g'+str(i)]='Ámbito de estudio del '+L_ordinal[i-1]+  ' titulo de grado'

## 'EST_B16'  (redefinida como 'n_master')   
(Solo para quienes declaran haber terminado estudios de máster -respuesta 1 en variable 'EST_B11_2' (o 'master')-)  

**Número de títulos de máster** universitario que ha completado.

In [370]:
df['EST_B16'].unique() # Missing values as ' '

array([' ', '1', '2', '3'], dtype=object)

In [371]:
traduce('EST_B16')

            aux_traduce
                 unique
EST_B16                
         [No aplicable]
1                 [Uno]
2                [Dos ]
3          [Tres o más]


In [372]:
df[['EST_B16','master']].groupby('master',dropna=False).agg(['unique'])

,EST_B16
,unique
master,
0,[ ]
1,"[1, 2, 3]"


In [373]:
redefine('EST_B16','n_master')

In [374]:
# Se imputa valor 0 en 'n_master' a quienes declaran no tener títulos de máster:
df['n_master'].mask(df['master']==0,0,inplace=True)
df['n_master']=df['n_master'].astype('int')
traduce('n_master')
df[['n_master_','master_']].groupby('master_',dropna=False).agg(['unique']) # Comprobación

           aux_traduce
                unique
n_master              
0                  [0]
1                [Uno]
2               [Dos ]
3         [Tres o más]


,n_master_
,unique
master_,
No,[0]
Sí,"[Uno, Dos , Tres o más]"


In [375]:
proporciones('n_master')

0    0.525260
1    0.382421
2    0.083031
3    0.009289
Name: n_master, dtype: float64

In [376]:
# Establece el valor de 'otros_estudios' referido a estudios de máster para función re_otros_estudios
otros_estudios='n_master'

## 'EST_B17_M1'  (redefinida como 'rama_m1')   
(Solo para quienes declaran haber terminado estudios de máster -respuesta 1 en variable 'EST_B11_2' (o 'master')-) 

**Rama de conocimiento del primer título de máster** mencionado como completado.

In [377]:
df['EST_B17_M1'].unique() # Missing values as ' '

array([' ', '1', '5', '3', '2', '4'], dtype=object)

In [378]:
traduce('EST_B17_M1')

                                aux_traduce
                                     unique
EST_B17_M1                                 
                                        [ ]
1                     [Artes y humanidades]
2                                [Ciencias]
3           [Ciencias sociales y jurídicas]
4               [Ingeniería y arquitectura]
5                    [Ciencias de la salud]


In [379]:
re_otros_estudios('EST_B17_M1','rama_m1',1)

                                                                                                                 rama_m1_
                                                                                                                   unique
n_master                                                                                                                 
0                                                                                                                   [nan]
1         [Artes y humanidades, Ciencias de la salud, Ciencias sociales y jurídicas, Ciencias, Ingeniería y arquitectura]
2         [Ciencias sociales y jurídicas, Artes y humanidades, Ciencias, Ciencias de la salud, Ingeniería y arquitectura]
3         [Ciencias sociales y jurídicas, Ciencias, Ciencias de la salud, Artes y humanidades, Ingeniería y arquitectura]
                                                                                                                 rama_m1_
                        

In [380]:
D_dict['rama_m1']=D_dict['rama_g2']
traduce('rama_m1')

                             aux_traduce
                                  unique
rama_m1                                 
0.0                           [No tiene]
1.0                [Artes y humanidades]
2.0                           [Ciencias]
3.0      [Ciencias sociales y jurídicas]
4.0          [Ingeniería y arquitectura]
5.0               [Ciencias de la salud]


In [381]:
proporciones('rama_m1_')

No tiene                         0.525260
Ciencias sociales y jurídicas    0.189725
Ciencias de la salud             0.088718
Ciencias                         0.066696
Artes y humanidades              0.065748
Ingeniería y arquitectura        0.063853
Name: rama_m1_, dtype: float64

## 'EST_B18_1'  
(Solo para quienes declaran haber terminado estudios de máster -respuesta 1 en variable 'EST_B11_2' (o 'master')-)  

**Momento en que terminó el primer título de máster** mencionado como completado.

In [382]:
df['EST_B18_1'].unique() # Missing values as ' ' or '9'

array([' ', '3', '2', '1', '9'], dtype=object)

In [383]:
traduce('EST_B18_1')

                                                                              aux_traduce
                                                                                   unique
EST_B18_1                                                                                
                                                                           [No aplicable]
1                  [Antes de comenzar los estudios de la titulación que finalizó en 2014]
2          [Durante la realización de los estudios de la titulación que finalizó en 2014]
3                                 [Después de obtener la titulación que finalizó en 2014]
9                                                                                 [NS/NC]


In [384]:
re_otros_estudios('EST_B18_1')

[ 1.  2.  3. nan]


## 'EST_B19_1'     
(Solo para quienes declaran haber terminado estudios de máster -respuesta 1 en variable 'EST_B11_2' (o 'master')-)

**Dónde estudió el primer título de máster** mencionado como completado.

In [385]:
df['EST_B19_1'].unique() # Missing values as ' ' or '9'

array([' ', '1', '2', '3', '9'], dtype=object)

In [386]:
traduce('EST_B19_1')

                                                   aux_traduce
                                                        unique
EST_B19_1                                                     
                                                [No aplicable]
1                                    [Completamente en España]
2          [Una parte en España y otra parte en el extranjero]
3                             [Completamente en el extranjero]
9                                                      [NS/NC]


In [387]:
re_otros_estudios('EST_B19_1')

[ 1.  2.  3. nan]


In [388]:
actualiza_extr_proxy('EST_B19_1')

                                                  aux_traduce
                                                       unique
extr_proxy                                                   
0                                [No estudió fuera de España]
1           [Al menos parte de los estudios en el extranjero]
                                                                             extr_proxy_
                                                                                  unique
EST_B19_1                                                                               
1.0        [No estudió fuera de España, Al menos parte de los estudios en el extranjero]
2.0                                    [Al menos parte de los estudios en el extranjero]
3.0                                    [Al menos parte de los estudios en el extranjero]
NaN        [No estudió fuera de España, Al menos parte de los estudios en el extranjero]


## 'EST_B17_M2'  ( redefinida como 'rama_m2')   
(Solo para quienes declaran haber terminado estudios de máster -respuesta 1 en variable 'EST_B11_2' (o 'master')-)  

**Rama de conocimiento del segundo título de máster** mencionado como completado.  

In [389]:
df['EST_B17_M2'].unique() # Missing values as ' '

array([' ', '3', '1', '2', '5', '4'], dtype=object)

In [390]:
traduce('EST_B17_M2')

                                aux_traduce
                                     unique
EST_B17_M2                                 
                                        [ ]
1                     [Artes y humanidades]
2                                [Ciencias]
3           [Ciencias sociales y jurídicas]
4               [Ingeniería y arquitectura]
5                    [Ciencias de la salud]


In [391]:
re_otros_estudios('EST_B17_M2','rama_m2',2)

                                                                                                                 rama_m2_
                                                                                                                   unique
n_master                                                                                                                 
0                                                                                                                   [nan]
1                                                                                                                   [nan]
2         [Ciencias sociales y jurídicas, Artes y humanidades, Ciencias de la salud, Ciencias, Ingeniería y arquitectura]
3         [Artes y humanidades, Ciencias, Ciencias sociales y jurídicas, Ciencias de la salud, Ingeniería y arquitectura]
                                                                                                                 rama_m2_
                        

In [392]:
D_dict['rama_m2']=D_dict['rama_g2']
traduce('rama_m2')

                             aux_traduce
                                  unique
rama_m2                                 
0.0                           [No tiene]
1.0                [Artes y humanidades]
2.0                           [Ciencias]
3.0      [Ciencias sociales y jurídicas]
4.0          [Ingeniería y arquitectura]
5.0               [Ciencias de la salud]


## 'EST_B18_2'  
(Solo para quienes declaran haber terminado estudios de máster -respuesta 1 en variable 'EST_B11_2' (o 'master')-) 

**Momento en que terminó el segundo título de máster** mencionado como completado.

In [393]:
df['EST_B18_2'].unique() # Missing values as ' ' or '9'

array([' ', '3', '2', '1', '9'], dtype=object)

In [394]:
traduce('EST_B18_2')

                                                                              aux_traduce
                                                                                   unique
EST_B18_2                                                                                
                                                                           [No aplicable]
1                  [Antes de comenzar los estudios de la titulación que finalizó en 2014]
2          [Durante la realización de los estudios de la titulación que finalizó en 2014]
3                                 [Después de obtener la titulación que finalizó en 2014]
9                                                                                 [NS/NC]


In [395]:
re_otros_estudios('EST_B18_2')

[ 1.  2.  3. nan]


## 'EST_B19_2'     
(Solo para quienes declaran haber terminado estudios de máster -respuesta 1 en variable 'EST_B11_2' (o 'master')-)

**Dónde estudió el segundo título de máster** mencionado como completado.

In [396]:
df['EST_B19_2'].unique() # Missing values as ' ' or '9'

array([' ', '1', '2', '3', '9'], dtype=object)

In [397]:
traduce('EST_B19_2')

                                                   aux_traduce
                                                        unique
EST_B19_2                                                     
                                                [No aplicable]
1                                    [Completamente en España]
2          [Una parte en España y otra parte en el extranjero]
3                             [Completamente en el extranjero]
9                                                      [NS/NC]


In [398]:
re_otros_estudios('EST_B19_2')

[ 1.  2.  3. nan]


In [399]:
actualiza_extr_proxy('EST_B19_2')

                                                  aux_traduce
                                                       unique
extr_proxy                                                   
0                                [No estudió fuera de España]
1           [Al menos parte de los estudios en el extranjero]
                                                                             extr_proxy_
                                                                                  unique
EST_B19_2                                                                               
1.0        [No estudió fuera de España, Al menos parte de los estudios en el extranjero]
2.0                                    [Al menos parte de los estudios en el extranjero]
3.0                                    [Al menos parte de los estudios en el extranjero]
NaN        [No estudió fuera de España, Al menos parte de los estudios en el extranjero]


## 'EST_B17_M3'  (redefinida como 'rama_m3')   
(Solo para quienes declaran haber terminado estudios de máster -respuesta 1 en variable 'EST_B11_2' (o 'master')-) 

**Rama de conocimiento del tercer título de máster** mencionado como completado.

In [400]:
df['EST_B17_M3'].unique() # Missing values as ' '

array([' ', '3', '2', '5', '1', '4'], dtype=object)

In [401]:
traduce('EST_B17_M3')

                                aux_traduce
                                     unique
EST_B17_M3                                 
                                        [ ]
1                     [Artes y humanidades]
2                                [Ciencias]
3           [Ciencias sociales y jurídicas]
4               [Ingeniería y arquitectura]
5                    [Ciencias de la salud]


In [402]:
re_otros_estudios('EST_B17_M3','rama_m3',3)

                                                                                                                 rama_m3_
                                                                                                                   unique
n_master                                                                                                                 
0                                                                                                                   [nan]
1                                                                                                                   [nan]
2                                                                                                                   [nan]
3         [Ciencias sociales y jurídicas, Ciencias, Ciencias de la salud, Artes y humanidades, Ingeniería y arquitectura]
                                                                                                                 rama_m3_
                        

In [403]:
D_dict['rama_m3']=D_dict['rama_g2']
traduce('rama_m3')

                             aux_traduce
                                  unique
rama_m3                                 
0.0                           [No tiene]
1.0                [Artes y humanidades]
2.0                           [Ciencias]
3.0      [Ciencias sociales y jurídicas]
4.0          [Ingeniería y arquitectura]
5.0               [Ciencias de la salud]


## 'EST_B18_3'  
(Solo para quienes declaran haber terminado estudios de máster -respuesta 1 en variable 'EST_B11_2' (o 'master')-)  

**Momento en que terminó el tercer título de máster** mencionado como completado.

In [404]:
df['EST_B18_3'].unique() # Missing values as ' '

array([' ', '3', '1', '2'], dtype=object)

In [405]:
traduce('EST_B18_3')

                                                                              aux_traduce
                                                                                   unique
EST_B18_3                                                                                
                                                                           [No aplicable]
1                  [Antes de comenzar los estudios de la titulación que finalizó en 2014]
2          [Durante la realización de los estudios de la titulación que finalizó en 2014]
3                                 [Después de obtener la titulación que finalizó en 2014]


In [406]:
re_otros_estudios('EST_B18_3')

[ 1.  2.  3. nan]


## 'EST_B19_3'     
(Solo para quienes declaran haber terminado estudios de máster -respuesta 1 en variable 'EST_B11_2' (o 'master')-)

**Dónde estudió el tercer título de máster** mencionado como completado.

In [407]:
df['EST_B19_3'].unique() # Missing values as ' '

array([' ', '1', '3', '2'], dtype=object)

In [408]:
traduce('EST_B19_3')

                                                   aux_traduce
                                                        unique
EST_B19_3                                                     
                                                [No aplicable]
1                                    [Completamente en España]
2          [Una parte en España y otra parte en el extranjero]
3                             [Completamente en el extranjero]


In [409]:
re_otros_estudios('EST_B19_3')

[ 1.  2.  3. nan]


In [410]:
actualiza_extr_proxy('EST_B19_3')

                                                  aux_traduce
                                                       unique
extr_proxy                                                   
0                                [No estudió fuera de España]
1           [Al menos parte de los estudios en el extranjero]
                                                                             extr_proxy_
                                                                                  unique
EST_B19_3                                                                               
1.0        [No estudió fuera de España, Al menos parte de los estudios en el extranjero]
2.0                                    [Al menos parte de los estudios en el extranjero]
3.0                                    [Al menos parte de los estudios en el extranjero]
NaN        [No estudió fuera de España, Al menos parte de los estudios en el extranjero]


In [411]:
for i in range(1,4):
    D_describe['rama_m'+str(i)]='Rama de conocimiento del '+L_ordinal[i-1]+ ' titulo de master'

## 'EST_B21'  (redefinida como 'rama_doc')   
(Solo para quienes declaran haber terminado estudios de doctorado -respuesta 1 en variable 'EST_B11_3' (o'doctor')-)  

**Rama de conocimiento de sus estudios de doctorado** (si tiene más de un título de doctor, del que considere el principal).

In [412]:
df['EST_B21'].unique() # Missing values as ' '

array([' ', '3', '1', '5', '2', '4'], dtype=object)

In [413]:
traduce('EST_B21')

                             aux_traduce
                                  unique
EST_B21                                 
                                     [ ]
1                  [Artes y humanidades]
2                             [Ciencias]
3        [Ciencias sociales y jurídicas]
4            [Ingeniería y arquitectura]
5                 [Ciencias de la salud]


In [414]:
# Establece otros_estudios referido al título de doctor: 
otros_estudios='doctor'

In [415]:
re_otros_estudios('EST_B21','rama_doc',1)

                                                                                                              rama_doc_
                                                                                                                 unique
doctor                                                                                                                 
0                                                                                                                 [nan]
1       [Ciencias sociales y jurídicas, Artes y humanidades, Ciencias de la salud, Ciencias, Ingeniería y arquitectura]
                                                                                                              rama_doc_
                                                                                                                 unique
doctor                                                                                                                 
0                                       

In [416]:
D_dict['rama_doc']=D_dict['rama_g2']
traduce('rama_doc')

                              aux_traduce
                                   unique
rama_doc                                 
0.0                            [No tiene]
1.0                 [Artes y humanidades]
2.0                            [Ciencias]
3.0       [Ciencias sociales y jurídicas]
4.0           [Ingeniería y arquitectura]
5.0                [Ciencias de la salud]


## 'EST_B22'  
(Solo para quienes declaran haber terminado estudios de doctorado -respuesta 1 en variable 'EST_B11_3' (o'doctor')-)  

**Momento en que terminó sus estudios de doctorado** (si tiene más de un título de doctor, el que considere el principal).

In [417]:
df['EST_B22'].unique() # Missing values as ' '

array([' ', '3', '1', '2'], dtype=object)

In [418]:
traduce('EST_B22')

                                                                            aux_traduce
                                                                                 unique
EST_B22                                                                                
                                                                         [No aplicable]
1                [Antes de comenzar los estudios de la titulación que finalizó en 2014]
2        [Durante la realización de los estudios de la titulación que finalizó en 2014]
3                               [Después de obtener la titulación que finalizó en 2014]


In [419]:
re_otros_estudios('EST_B22')

[ 1.  2.  3. nan]


## 'EST_B23'  
(Solo para quienes declaran haber terminado estudios de doctorado -respuesta 1 en variable 'EST_B11_3' (o'doctor')-)   

**Donde cursó sus estudios de doctorado** (si tiene más de un título de doctor, el que considere el principal).

In [420]:
df['EST_B23'].unique() # Missing values as ' '

array([' ', '1', '2', '9', '3'], dtype=object)

In [421]:
traduce('EST_B23')

                                                 aux_traduce
                                                      unique
EST_B23                                                     
                                              [No aplicable]
1                                  [Completamente en España]
2        [Una parte en España y otra parte en el extranjero]
3                           [Completamente en el extranjero]
9                                                    [NS/NC]


In [422]:
re_otros_estudios('EST_B23')

[ 1.  2.  3. nan]


In [423]:
actualiza_extr_proxy('EST_B23')

                                                  aux_traduce
                                                       unique
extr_proxy                                                   
0                                [No estudió fuera de España]
1           [Al menos parte de los estudios en el extranjero]
                                                                           extr_proxy_
                                                                                unique
EST_B23                                                                               
1.0      [No estudió fuera de España, Al menos parte de los estudios en el extranjero]
2.0                                  [Al menos parte de los estudios en el extranjero]
3.0                                  [Al menos parte de los estudios en el extranjero]
NaN      [No estudió fuera de España, Al menos parte de los estudios en el extranjero]


## 'EST_B24'  (redefinida como 'n_gsup')   
(Solo para quienes declaran haber terminado ciclos formativos de grado superior y/o estudios artísticos
superiores -respuesta 1 en variable 'EST_B11_5' (o 'g_sup') o  en 'EST_B11_4' (o 'est_art')-)  

**Número de ciclos formativos de grado superior o estudios artísticos superiores** que ha completado.

In [424]:
df['EST_B24'].unique()

array([' ', '1', '2', '3'], dtype=object)

In [425]:
traduce('EST_B24')

            aux_traduce
                 unique
EST_B24                
         [No aplicable]
1                 [Uno]
2                [Dos ]
3          [Tres o más]


In [426]:
# Se imputa 0 a no respuesta: 
df['EST_B24'].replace({' ':0},inplace=True)
df['EST_B24']=df['EST_B24'].astype('int')
df['EST_B24'].unique()

array([0, 1, 2, 3])

In [427]:
redefine('EST_B24','n_gsup')

In [428]:
df[['n_gsup','g_sup_','est_art_']].groupby(['g_sup_','est_art_'],dropna=False).agg(['unique']) # Inspección: 

n_gsup
                    unique
g_sup_ est_art_           
No     No              [0]
       Sí        [1, 2, 3]
Sí     No        [1, 2, 3]
       Sí        [2, 3, 1]

In [429]:
# Se establece 'otros_estudios' referido a ciclos formativos o estudios artísticos superiores: 
otros_estudios='n_gsup'

## 'EST_B25_FA1'  (redefinida como 'fam_gsup1')   
(Solo para quienes declaran haber terminado ciclos formativos de grado superior y/o estudios artísticos
superiores -respuesta 1 en variable 'EST_B11_5' (o 'g_sup') o  en 'EST_B11_4' (o 'est_art')-)  

**Familia de estudios** del primer ciclo formativo de grado superior (formación profesional, artes plásticas y diseño) o enseñanzas deportivas de grado superior mencionado.

In [430]:
df['EST_B25_FA1'].unique() # Missing values as '  ' or '99'

array(['  ', '88', '23', '19', '04', '21', '22', '25', '14', '24', '05',
       '20', '07', '10', '06', '99', '08', '18', '16', '26', '02', '01',
       '09', '12', '15', '17', '11', '03'], dtype=object)

In [431]:
leyenda('EST_B25_FA1')

01 	 Agraria
02 	 Marítimo-pesquera
03 	 Industrias alimentarias 
04 	 Química
05 	 Imagen personal 
06 	 Sanidad 
07 	 Seguridad y medio ambiente
08 	 Fabricación mecánica 
09 	 Instalación y mantenimiento
10 	 Electricidad y electrónica
11 	 Energía y agua
12 	 Transporte y mantenimiento de vehículos 
14 	 Edificación y obra civil 
15 	 Vidrio y cerámica 
16 	 Madera, mueble y corcho 
17 	 Textil, confección y piel
18 	 Artes gráficas 
19 	 Imagen y sonido
20 	 Informática y comunicaciones
21 	 Administración y gestión 
22 	 Comercio y marketing
23 	 Servicios socioculturales y a la comunidad 
24 	 Hostelería y turismo 
25 	 Actividades físicas y deportivas (incluidas enseñanzas deportivas de grado superior)
26 	 Artes y artesanías
88 	 Estudios artísticos superiores (artes plásticas, diseño, música y danza, arte dramático o conservación y restauración de bienes culturales)
99 	 NS/NC
   	 No aplicable


In [432]:
traduce('EST_B25_FA1',check=False)

In [433]:
re_otros_estudios('EST_B25_FA1','fam_gsup1',1,blank='  ',nsnc='99')

                                                                                                                                                                                                                                                                                                         fam_gsup1_
                                                                                                                                                                                                                                                                                                             unique
n_gsup                                                                                                                                                                                                                                                                                                             
0                                                                           

In [434]:
D_dict['fam_gsup1']=D_dict['EST_B25_FA1']
D_dict['fam_gsup1'][0]='No tiene'
traduce('fam_gsup1',check=False)

## 'EST_B26_1'  
(Solo para quienes declaran haber terminado ciclos formativos de grado superior y/o estudios artísticos
superiores -respuesta 1 en variable 'EST_B11_5' (o 'g_sup') o  en 'EST_B11_4' (o 'est_art')-)   

**Momento en que terminó** el primer ciclo formativo de grado superior (formación profesional, artes plásticas y diseño) o enseñanzas deportivas de grado superior mencionado.

In [435]:
df['EST_B26_1'].unique() # Missing values as ' ' or '9'

array([' ', '1', '3', '2', '9'], dtype=object)

In [436]:
traduce('EST_B26_1')

                                                                              aux_traduce
                                                                                   unique
EST_B26_1                                                                                
                                                                           [No aplicable]
1                  [Antes de comenzar los estudios de la titulación que finalizó en 2014]
2          [Durante la realización de los estudios de la titulación que finalizó en 2014]
3                                 [Después de obtener la titulación que finalizó en 2014]
9                                                                                 [NS/NC]


In [437]:
re_otros_estudios('EST_B26_1')

[ 1.  2.  3. nan]


## 'EST_B27_1'  
(Solo para quienes declaran haber terminado ciclos formativos de grado superior y/o estudios artísticos
superiores -respuesta 1 en variable 'EST_B11_5' (o 'g_sup') o  en 'EST_B11_4' (o 'est_art')-)   

**Dónde cursó** el primer ciclo formativo de grado superior (formación profesional, artes plásticas y diseño) o enseñanzas deportivas de grado superior mencionado.

In [438]:
df['EST_B27_1'].unique() # Missing values as ' ' or '9'

array([' ', '1', '2', '3', '9'], dtype=object)

In [439]:
traduce('EST_B27_1')

                                                   aux_traduce
                                                        unique
EST_B27_1                                                     
                                                [No aplicable]
1                                    [Completamente en España]
2          [Una parte en España y otra parte en el extranjero]
3                             [Completamente en el extranjero]
9                                                      [NS/NC]


In [440]:
re_otros_estudios('EST_B27_1')

[ 1.  2.  3. nan]


In [441]:
actualiza_extr_proxy('EST_B27_1')

                                                  aux_traduce
                                                       unique
extr_proxy                                                   
0                                [No estudió fuera de España]
1           [Al menos parte de los estudios en el extranjero]
                                                                             extr_proxy_
                                                                                  unique
EST_B27_1                                                                               
1.0        [Al menos parte de los estudios en el extranjero, No estudió fuera de España]
2.0                                    [Al menos parte de los estudios en el extranjero]
3.0                                    [Al menos parte de los estudios en el extranjero]
NaN        [No estudió fuera de España, Al menos parte de los estudios en el extranjero]


## 'EST_B25_FA2'  (redefinida como 'fam_gsup2')   
(Solo para quienes declaran haber terminado ciclos formativos de grado superior y/o estudios artísticos
superiores -respuesta 1 en variable 'EST_B11_5' (o 'g_sup') o  en 'EST_B11_4' (o 'est_art')-)  

**Familia de estudios** del segundo ciclo formativo de grado superior (formación profesional, artes plásticas y diseño) o enseñanzas deportivas de grado superior mencionado.

In [442]:
df['EST_B25_FA2'].unique() # Missing values as '  ' or '99'

array(['  ', '06', '22', '05', '21', '23', '88', '18', '14', '24', '20',
       '10', '25', '04', '19', '26', '17', '01', '07', '08', '99', '12',
       '09', '03', '02', '16'], dtype=object)

In [443]:
leyenda('EST_B25_FA2')

01 	 Agraria
02 	 Marítimo-pesquera
03 	 Industrias alimentarias 
04 	 Química
05 	 Imagen personal 
06 	 Sanidad 
07 	 Seguridad y medio ambiente
08 	 Fabricación mecánica 
09 	 Instalación y mantenimiento
10 	 Electricidad y electrónica
11 	 Energía y agua
12 	 Transporte y mantenimiento de vehículos 
14 	 Edificación y obra civil 
15 	 Vidrio y cerámica 
16 	 Madera, mueble y corcho 
17 	 Textil, confección y piel
18 	 Artes gráficas 
19 	 Imagen y sonido
20 	 Informática y comunicaciones
21 	 Administración y gestión 
22 	 Comercio y marketing
23 	 Servicios socioculturales y a la comunidad 
24 	 Hostelería y turismo 
25 	 Actividades físicas y deportivas (incluidas enseñanzas deportivas de grado superior)
26 	 Artes y artesanías
88 	 Estudios artísticos superiores (artes plásticas, diseño, música y danza, arte dramático o conservación y restauración de bienes culturales)
99 	 NS/NC
   	 No aplicable


In [444]:
traduce('EST_B25_FA2',check=False)

In [445]:
re_otros_estudios('EST_B25_FA2','fam_gsup2',2,blank='  ',nsnc='99')

                                                                                                                                                                                                                                                                                                         fam_gsup2_
                                                                                                                                                                                                                                                                                                             unique
n_gsup                                                                                                                                                                                                                                                                                                             
0                                                                           

In [446]:
D_dict['fam_gsup2']=D_dict['fam_gsup1']
traduce('fam_gsup2',check=False)

## 'EST_B26_2'  
(Solo para quienes declaran haber terminado ciclos formativos de grado superior y/o estudios artísticos
superiores -respuesta 1 en variable 'EST_B11_5' (o 'g_sup') o  en 'EST_B11_4' (o 'est_art')-)  

**Momento en que terminó** el segundo ciclo formativo de grado superior (formación profesional, artes plásticas y diseño) o enseñanzas deportivas de grado superior mencionado.

In [447]:
df['EST_B26_2'].unique() # Missing values as ' ' 

array([' ', '1', '3', '2'], dtype=object)

In [448]:
traduce('EST_B26_2')

                                                                              aux_traduce
                                                                                   unique
EST_B26_2                                                                                
                                                                           [No aplicable]
1                  [Antes de comenzar los estudios de la titulación que finalizó en 2014]
2          [Durante la realización de los estudios de la titulación que finalizó en 2014]
3                                 [Después de obtener la titulación que finalizó en 2014]


In [449]:
re_otros_estudios('EST_B26_2')

[ 1.  2.  3. nan]


## 'EST_B27_2'  
(Solo para quienes declaran haber terminado ciclos formativos de grado superior y/o estudios artísticos
superiores -respuesta 1 en variable 'EST_B11_5' (o 'g_sup') o  en 'EST_B11_4' (o 'est_art')-)  

**Dónde cursó** el segundo ciclo formativo de grado superior (formación profesional, artes plásticas y diseño) o enseñanzas deportivas de grado superior mencionado.

In [450]:
df['EST_B27_2'].unique() # Missing values as ' ' 

array([' ', '1', '3', '2'], dtype=object)

In [451]:
traduce('EST_B27_2')

                                                   aux_traduce
                                                        unique
EST_B27_2                                                     
                                                [No aplicable]
1                                    [Completamente en España]
2          [Una parte en España y otra parte en el extranjero]
3                             [Completamente en el extranjero]


In [452]:
re_otros_estudios('EST_B27_2')

[ 1.  2.  3. nan]


In [453]:
actualiza_extr_proxy('EST_B27_2')

                                                  aux_traduce
                                                       unique
extr_proxy                                                   
0                                [No estudió fuera de España]
1           [Al menos parte de los estudios en el extranjero]
                                                                             extr_proxy_
                                                                                  unique
EST_B27_2                                                                               
1.0        [No estudió fuera de España, Al menos parte de los estudios en el extranjero]
2.0                                    [Al menos parte de los estudios en el extranjero]
3.0                                    [Al menos parte de los estudios en el extranjero]
NaN        [No estudió fuera de España, Al menos parte de los estudios en el extranjero]


## 'EST_B25_FA3'  (redefinida como 'fam_gsup3')   
(Solo para quienes declaran haber terminado ciclos formativos de grado superior y/o estudios artísticos
superiores -respuesta 1 en variable 'EST_B11_5' (o 'g_sup') o  en 'EST_B11_4' (o 'est_art')-)   

**Familia de estudios** del tercer ciclo formativo de grado superior (formación profesional, artes plásticas y diseño) o enseñanzas deportivas de grado superior mencionado.

In [454]:
df['EST_B25_FA3'].unique() # Missing values as '  ' or '99'

array(['  ', '23', '19', '88', '14', '99', '06', '16', '10', '20', '09',
       '04', '03', '25', '22'], dtype=object)

In [455]:
leyenda('EST_B25_FA3')

01 	 Agraria
02 	 Marítimo-pesquera
03 	 Industrias alimentarias 
04 	 Química
05 	 Imagen personal 
06 	 Sanidad 
07 	 Seguridad y medio ambiente
08 	 Fabricación mecánica 
09 	 Instalación y mantenimiento
10 	 Electricidad y electrónica
11 	 Energía y agua
12 	 Transporte y mantenimiento de vehículos 
14 	 Edificación y obra civil 
15 	 Vidrio y cerámica 
16 	 Madera, mueble y corcho 
17 	 Textil, confección y piel
18 	 Artes gráficas 
19 	 Imagen y sonido
20 	 Informática y comunicaciones
21 	 Administración y gestión 
22 	 Comercio y marketing
23 	 Servicios socioculturales y a la comunidad 
24 	 Hostelería y turismo 
25 	 Actividades físicas y deportivas (incluidas enseñanzas deportivas de grado superior)
26 	 Artes y artesanías
88 	 Estudios artísticos superiores (artes plásticas, diseño, música y danza, arte dramático o conservación y restauración de bienes culturales)
99 	 NS/NC
   	 No aplicable


In [456]:
traduce('EST_B25_FA3',check=False)

In [457]:
re_otros_estudios('EST_B25_FA3','fam_gsup3',3,blank='  ',nsnc='99')

                                                                                                                                                                                                                                                                                                         fam_gsup3_
                                                                                                                                                                                                                                                                                                             unique
n_gsup                                                                                                                                                                                                                                                                                                             
0                                                                           

In [458]:
D_dict['fam_gsup3']=D_dict['fam_gsup1']
traduce('fam_gsup3',check=False)

## 'EST_B26_3'  
(Solo para quienes declaran haber terminado ciclos formativos de grado superior y/o estudios artísticos
superiores -respuesta 1 en variable 'EST_B11_5' (o 'g_sup') o  en 'EST_B11_4' (o 'est_art')-)    

**Momento en que terminó** el tercer ciclo formativo de grado superior (formación profesional, artes plásticas y diseño) o enseñanzas deportivas de grado superior mencionado.

In [459]:
df['EST_B26_3'].unique() # Missing values as ' ' 

array([' ', '1', '2', '3'], dtype=object)

In [460]:
traduce('EST_B26_3')

                                                                              aux_traduce
                                                                                   unique
EST_B26_3                                                                                
                                                                           [No aplicable]
1                  [Antes de comenzar los estudios de la titulación que finalizó en 2014]
2          [Durante la realización de los estudios de la titulación que finalizó en 2014]
3                                 [Después de obtener la titulación que finalizó en 2014]


In [461]:
re_otros_estudios('EST_B26_3')

[ 1.  2.  3. nan]


## 'EST_B27_3'  
(Solo para quienes declaran haber terminado ciclos formativos de grado superior y/o estudios artísticos
superiores -respuesta 1 en variable 'EST_B11_5' (o 'g_sup') o  en 'EST_B11_4' (o 'est_art')-)    

**Dónde cursó** el tercer ciclo formativo de grado superior (formación profesional, artes plásticas y diseño) o enseñanzas deportivas de grado superior mencionado.

In [462]:
df['EST_B27_3'].unique() # Missing values as ' ' 

array([' ', '1', '2'], dtype=object)

In [463]:
traduce('EST_B27_3')

                                                   aux_traduce
                                                        unique
EST_B27_3                                                     
                                                [No aplicable]
1                                    [Completamente en España]
2          [Una parte en España y otra parte en el extranjero]


In [464]:
re_otros_estudios('EST_B27_3')

[ 1.  2. nan]


In [465]:
actualiza_extr_proxy('EST_B27_3')

                                                  aux_traduce
                                                       unique
extr_proxy                                                   
0                                [No estudió fuera de España]
1           [Al menos parte de los estudios en el extranjero]
                                                                             extr_proxy_
                                                                                  unique
EST_B27_3                                                                               
1.0        [No estudió fuera de España, Al menos parte de los estudios en el extranjero]
2.0                                    [Al menos parte de los estudios en el extranjero]
NaN        [No estudió fuera de España, Al menos parte de los estudios en el extranjero]


In [466]:
for i in range(1,4):
    D_describe['fam_gsup'+str(i)]='Familia de estudios del '+L_ordinal[i-1]+ \
    ' titulo de ciclo formativo o enseñanzas deportivas de grado superior'

## 'EST_B28'  (redefinida como 'n_gmedio')   
(Solo para quienes declaran haber terminado ciclos formativos de grado medio -respuesta 1 en variable 'EST_B11_6' (o 'g_medio')-)  

**Número de ciclos formativos de grado medio** que ha completado.

In [467]:
df['EST_B28'].unique() # Missing values as ' '

array([' ', '1', '2', '3'], dtype=object)

In [468]:
traduce('EST_B28')

            aux_traduce
                 unique
EST_B28                
         [No aplicable]
1                 [Uno]
2                [Dos ]
3          [Tres o más]


In [469]:
redefine('EST_B28','n_gmedio')

In [470]:
df[['n_gmedio','g_medio']].groupby('g_medio',dropna=False).agg(['unique']) # Inspección: 

,n_gmedio
,unique
g_medio,
0,[ ]
1,"[1, 2, 3]"


In [471]:
# Se imputa valor 0 a quienes declararon no haber completado ciclos formativos de grado medio: 
df['n_gmedio'].mask(df['g_medio']==0,0,inplace=True)
df['n_gmedio']=df['n_gmedio'].astype('int')
df[['n_gmedio','g_medio']].groupby('g_medio',dropna=False).agg(['unique']) # Comprobación: 

,n_gmedio
,unique
g_medio,
0,[0]
1,"[1, 2, 3]"


In [472]:
# Se establece otros_estudios referido a ciclos formativos de grado medio: 
otros_estudios='n_gmedio'

## 'EST_B29_FA1'  (redefinida como 'fam_gmedio1')   
(Solo para quienes declaran haber terminado ciclos formativos de grado medio -respuesta 1 en variable 'EST_B11_6' (o 'g_medio')-) 

**Familia de estudios** del primer ciclo formativo de grado medio mencionado.

In [473]:
df['EST_B29_FA1'].sort_values().unique() # Missing values as '  ' 

array(['  ', '01', '02', '03', '04', '05', '06', '07', '08', '09', '10',
       '12', '13', '14', '16', '17', '18', '19', '20', '21', '22', '23',
       '24', '25', '88', '99'], dtype=object)

In [474]:
traduce('EST_B29_FA1')

                                                                                     aux_traduce
                                                                                          unique
EST_B29_FA1                                                                                     
                                                                                  [No aplicable]
01                                                                                     [Agraria]
02                                                                           [Marítimo-pesquera]
03                                                                    [Industrias alimentarias ]
04                                                                                     [Química]
05                                                                            [Imagen personal ]
06                                                                                    [Sanidad ]
07                            

In [475]:
re_otros_estudios('EST_B29_FA1','fam_gmedio1',1,blank='  ',nsnc='99')

                                                                                                                                                                                                                                                                                                         fam_gmedio1_
                                                                                                                                                                                                                                                                                                               unique
n_gmedio                                                                                                                                                                                                                                                                                                             
0                                                                     

In [476]:
D_dict['fam_gmedio1']=D_dict['EST_B29_FA1']
D_dict['fam_gmedio1'][0]='No tiene'
traduce('fam_gmedio1',check=False)

## 'EST_B30_1'  
(Solo para quienes declaran haber terminado ciclos formativos de grado medio -respuesta 1 en variable 'EST_B11_6' (o 'g_medio')-)  

**Momento en que terminó** el primer ciclo formativo de grado medio mencionado.

In [477]:
df['EST_B30_1'].unique() # Missing values as ' ' or '9'

array([' ', '1', '3', '2', '9'], dtype=object)

In [478]:
traduce('EST_B30_1')

                                                                              aux_traduce
                                                                                   unique
EST_B30_1                                                                                
                                                                           [No aplicable]
1                  [Antes de comenzar los estudios de la titulación que finalizó en 2014]
2          [Durante la realización de los estudios de la titulación que finalizó en 2014]
3                                 [Después de obtener la titulación que finalizó en 2014]
9                                                                                 [NS/NC]


In [479]:
re_otros_estudios('EST_B30_1')

[ 1.  2.  3. nan]


## 'EST_B31_1'  
(Solo para quienes declaran haber terminado ciclos formativos de grado medio -respuesta 1 en variable 'EST_B11_6' (o 'g_medio')-) 

**Dónde cursó** el primer ciclo formativo de grado medio mencionado.

In [480]:
df['EST_B31_1'].unique() # Missing values as ' ' or '9'

array([' ', '1', '3', '2', '9'], dtype=object)

In [481]:
traduce('EST_B31_1')

                                                   aux_traduce
                                                        unique
EST_B31_1                                                     
                                                [No aplicable]
1                                    [Completamente en España]
2          [Una parte en España y otra parte en el extranjero]
3                             [Completamente en el extranjero]
9                                                      [NS/NC]


In [482]:
re_otros_estudios('EST_B31_1')

[ 1.  2.  3. nan]


In [483]:
actualiza_extr_proxy('EST_B31_1')

                                                  aux_traduce
                                                       unique
extr_proxy                                                   
0                                [No estudió fuera de España]
1           [Al menos parte de los estudios en el extranjero]
                                                                             extr_proxy_
                                                                                  unique
EST_B31_1                                                                               
1.0        [No estudió fuera de España, Al menos parte de los estudios en el extranjero]
2.0                                    [Al menos parte de los estudios en el extranjero]
3.0                                    [Al menos parte de los estudios en el extranjero]
NaN        [No estudió fuera de España, Al menos parte de los estudios en el extranjero]


## 'EST_B29_FA2'  (redesignada como 'fam_gmedio2')   
(Solo para quienes declaran haber terminado ciclos formativos de grado medio -respuesta 1 en variable 'EST_B11_6' (o 'g_medio')-) 

**Familia de estudios** del segundo ciclo formativo de grado medio mencionado.

In [484]:
df['EST_B29_FA2'].sort_values().unique() # Missing values as '  '

array(['  ', '03', '04', '05', '06', '09', '10', '12', '14', '18', '19',
       '21', '22', '23', '24', '25', '88'], dtype=object)

In [485]:
traduce('EST_B29_FA2')

                                                                                     aux_traduce
                                                                                          unique
EST_B29_FA2                                                                                     
                                                                                  [No aplicable]
03                                                                    [Industrias alimentarias ]
04                                                                                     [Química]
05                                                                            [Imagen personal ]
06                                                                                    [Sanidad ]
09                                                                 [Instalación y mantenimiento]
10                                                                  [Electricidad y electrónica]
12                            

In [486]:
re_otros_estudios('EST_B29_FA2','fam_gmedio2',2,blank='  ')

                                                                                                                                                                                                                                                                                                         fam_gmedio2_
                                                                                                                                                                                                                                                                                                               unique
n_gmedio                                                                                                                                                                                                                                                                                                             
0                                                                     

In [487]:
D_dict['fam_gmedio2']=D_dict['fam_gmedio1']
traduce('fam_gmedio2',check=False)

## 'EST_B30_2'  
(Solo para quienes declaran haber terminado ciclos formativos de grado medio -respuesta 1 en variable 'EST_B11_6' (o 'g_medio')-)  

**Momento en que terminó** el segundo ciclo formativo de grado medio mencionado.

In [488]:
df['EST_B30_2'].unique() # Missing values as ' ' 

array([' ', '1', '3', '2'], dtype=object)

In [489]:
traduce('EST_B30_2')

                                                                              aux_traduce
                                                                                   unique
EST_B30_2                                                                                
                                                                           [No aplicable]
1                  [Antes de comenzar los estudios de la titulación que finalizó en 2014]
2          [Durante la realización de los estudios de la titulación que finalizó en 2014]
3                                 [Después de obtener la titulación que finalizó en 2014]


In [490]:
re_otros_estudios('EST_B30_2')

[ 1.  2.  3. nan]


## 'EST_31_2'  
(Solo para quienes declaran haber terminado ciclos formativos de grado medio -respuesta 1 en variable 'EST_B11_6' (o 'g_medio')-)  

**Dónde cursó** el segundo ciclo formativo de grado medio mencionado.

In [491]:
df['EST_B31_2'].unique() # Missing values as ' ' or '9'

array([' ', '1', '3', '9'], dtype=object)

In [492]:
traduce('EST_B31_2')

                                aux_traduce
                                     unique
EST_B31_2                                  
                             [No aplicable]
1                 [Completamente en España]
3          [Completamente en el extranjero]
9                                   [NS/NC]


In [493]:
re_otros_estudios('EST_B31_2')

[ 1.  3. nan]


In [494]:
actualiza_extr_proxy('EST_B31_2')

                                                  aux_traduce
                                                       unique
extr_proxy                                                   
0                                [No estudió fuera de España]
1           [Al menos parte de los estudios en el extranjero]
                                                                             extr_proxy_
                                                                                  unique
EST_B31_2                                                                               
1.0        [No estudió fuera de España, Al menos parte de los estudios en el extranjero]
3.0                                    [Al menos parte de los estudios en el extranjero]
NaN        [No estudió fuera de España, Al menos parte de los estudios en el extranjero]


## 'EST_B29_FA3'  (redefinida como 'fam_gmedio3')   
(Solo para quienes declaran haber terminado ciclos formativos de grado medio -respuesta 1 en variable 'EST_B11_6' (o 'g_medio')-) 

**Familia de estudios** del tercer ciclo formativo de grado medio mencionado.

In [495]:
df['EST_B29_FA3'].unique() # Missing values as '  '

array(['  ', '06', '23', '25'], dtype=object)

In [496]:
traduce('EST_B29_FA3')

                                                                                     aux_traduce
                                                                                          unique
EST_B29_FA3                                                                                     
                                                                                  [No aplicable]
06                                                                                    [Sanidad ]
23                                                 [Servicios socioculturales y a la comunidad ]
25           [Actividades físicas y deportivas (incluidas enseñanzas deportivas de grado medio)]


In [497]:
re_otros_estudios('EST_B29_FA3','fam_gmedio3',3,blank='  ')

                                                                                                                                        fam_gmedio3_
                                                                                                                                              unique
n_gmedio                                                                                                                                            
0                                                                                                                                              [nan]
1                                                                                                                                              [nan]
2                                                                                                                                              [nan]
3         [Sanidad , Servicios socioculturales y a la comunidad , Actividades físicas y deportivas (inclui

In [498]:
D_dict['fam_gmedio3']=D_dict['fam_gmedio1']
traduce('fam_gmedio3')

                                                                                     aux_traduce
                                                                                          unique
fam_gmedio3                                                                                     
0.0                                                                                   [No tiene]
6.0                                                                                   [Sanidad ]
23.0                                               [Servicios socioculturales y a la comunidad ]
25.0         [Actividades físicas y deportivas (incluidas enseñanzas deportivas de grado medio)]


## 'EST_B30_3'  
(Solo para quienes declaran haber terminado ciclos formativos de grado medio -respuesta 1 en variable 'EST_B11_6' (o 'g_medio')-) 

**Momento en que terminó** el tercer ciclo formativo de grado medio mencionado

In [499]:
df['EST_B30_3'].unique() # Missing values as ' ' 

array([' ', '3', '2', '1'], dtype=object)

In [500]:
traduce('EST_B30_3')

                                                                              aux_traduce
                                                                                   unique
EST_B30_3                                                                                
                                                                           [No aplicable]
1                  [Antes de comenzar los estudios de la titulación que finalizó en 2014]
2          [Durante la realización de los estudios de la titulación que finalizó en 2014]
3                                 [Después de obtener la titulación que finalizó en 2014]


In [501]:
re_otros_estudios('EST_B30_3')

[ 1.  2.  3. nan]


## 'EST_B31_3'  
(Solo para quienes declaran haber terminado ciclos formativos de grado medio -respuesta 1 en variable 'EST_B11_6' (o 'g_medio')-)  

**Dónde cursó** el tercer ciclo formativo de grado medio mencionado.

In [502]:
df['EST_B31_3'].unique() # Missing values as ' ' 

array([' ', '1'], dtype=object)

In [503]:
traduce('EST_B31_3')

                         aux_traduce
                              unique
EST_B31_3                           
                      [No aplicable]
1          [Completamente en España]


In [504]:
re_otros_estudios('EST_B31_3')

[ 1. nan]


In [505]:
actualiza_extr_proxy('EST_B31_3')

                                                  aux_traduce
                                                       unique
extr_proxy                                                   
0                                [No estudió fuera de España]
1           [Al menos parte de los estudios en el extranjero]
                                                                             extr_proxy_
                                                                                  unique
EST_B31_3                                                                               
1.0                                                         [No estudió fuera de España]
NaN        [No estudió fuera de España, Al menos parte de los estudios en el extranjero]


In [506]:
for i in range(1,4):
    D_describe['fam_gmedio'+str(i)]='Familia de estudios del '+L_ordinal[i-1]+ \
    ' titulo de ciclo formativo de grado medio'

## 'EST2_SA'   

¿Ha realizado o está realizando alguna **formación sanitaria especializada** (MIR, EIR, FIR, PIR, BIR, RFIR, QIR, etc.)?

In [507]:
df['EST2_SA'].unique()

array([' ', '4', '3', '2', '1', '9'], dtype=object)

In [508]:
traduce('EST2_SA')

                                                   aux_traduce
                                                        unique
EST2_SA                                                       
                      [No aplicable (Titulación no sanitaria)]
1                         [Sí, ya ha completado esa formación]
2        [Sí, esta realizando el MIR o EIR o FIR o PIR, … etc]
3               [No, pero está preparando el examen de acceso]
4                                                         [No]
9                                                      [NS/NC]


In [509]:
no_respuesta('EST2_SA')

[ 1.  2.  3.  4. nan]


## 'EST2_NC'   

**Estudios en curso**.

In [510]:
df['EST2_NC'].unique()

array([99,  9,  7,  4,  2,  1,  5,  3,  8,  6])

In [511]:
traduce('EST2_NC',check=False)

In [512]:
leyenda('EST2_NC')

01 	 Grado universitario de 180 créditos (3 años)
02 	 Grado universitario de 240 créditos (4 años)
03 	 Grado universitario de más de 240 créditos (5 años o más)
04 	 Máster universitario
05 	 Doctorado universitario
06 	 Estudios artísticos superiores (artes plásticas, diseño, música y danza, arte dramático o conservación y restauración de bienes culturales)
07 	 Ciclos Formativos de grado superior (formación profesional, artes plásticas y diseño) o Enseñanzas Deportivas de Grado Superior
08 	 Ciclos Formativos de grado medio (formación profesional, artes plásticas y diseño) o Enseñanzas Deportivas de Grado Medio
09 	 No estoy cursando ninguno de los estudios anteriores
99 	 NS/NC


In [513]:
no_respuesta('EST2_NC',nsnc=99, tipo='int')

[ 1.  2.  3.  4.  5.  6.  7.  8.  9. nan]


## 'IDI_MT1'   (redefinida como 'idioma_m1')

**Primer idioma materno**.

In [514]:
df['IDI_MT1'].unique() # No missing values

array([ 1,  2,  3,  6,  8,  5,  4, 10, 15, 12,  7,  9, 11, 14, 13])

In [515]:
traduce('IDI_MT1')

             aux_traduce
                  unique
IDI_MT1                 
1              [Español]
2              [Catalán]
3              [Euskera]
4              [Gallego]
5           [Valenciano]
6               [Inglés]
7              [Francés]
8               [Alemán]
9             [Italiano]
10           [Portugués]
11              [Rumano]
12               [Árabe]
13               [Chino]
14                [Ruso]
15       [Otros idiomas]


In [516]:
redefine('IDI_MT1','idioma_m1')

In [517]:
proporciones('IDI_MT1_')

Español          0.857603
Catalán          0.073047
Gallego          0.021390
Euskera          0.020694
Valenciano       0.016461
Otros idiomas    0.003096
Inglés           0.001643
Rumano           0.001137
Francés          0.000948
Árabe            0.000821
Alemán           0.000727
Portugués        0.000695
Ruso             0.000600
Chino            0.000600
Italiano         0.000537
Name: IDI_MT1_, dtype: float64

## 'IDI_MT2'   (redefinida como 'idioma_m2')

**Segundo idioma materno**.

In [518]:
df['IDI_MT2'].unique()

array(['  ', '02', '04', '01', '06', '15', '03', '05', '07', '12', '08',
       '09', '10', '14', '13', '11'], dtype=object)

In [519]:
traduce('IDI_MT2')

             aux_traduce
                  unique
IDI_MT2                 
          [No aplicable]
01             [Español]
02             [Catalán]
03             [Euskera]
04             [Gallego]
05          [Valenciano]
06              [Inglés]
07             [Francés]
08              [Alemán]
09            [Italiano]
10           [Portugués]
11              [Rumano]
12               [Árabe]
13               [Chino]
14                [Ruso]
15       [Otros idiomas]


In [520]:
no_respuesta('IDI_MT2',blank='  ')

[ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. nan]


In [521]:
redefine('IDI_MT2','idioma_m2')

## 'IDIOMAS'   (redefinida como 'n_idiomas')

**Número de idiomas** que habla sin contar los maternos.

Nota: En 'n_idiomas' se computan también los maternos declarados y se prescinde de la leyenda.

In [522]:
df['IDIOMAS'].unique()

array([1, 2, 0, 9, 4, 3, 5])

In [523]:
traduce('IDIOMAS')

        aux_traduce
             unique
IDIOMAS            
0         [Ninguno]
1               [1]
2               [2]
3               [3]
4               [4]
5         [5 o más]
9           [NS/NC]


In [524]:
no_respuesta('IDIOMAS',tipo='int')

[ 0.  1.  2.  3.  4.  5. nan]


In [525]:
D_describe['n_idiomas']='Número de idiomas que habla incluidos los maternos'
# Se computan los idiomas maternos (al menos uno) y los no maternos declarados: 
df['aux']=1 # Sin valores no informados en 'idioma_m1'
df['aux'].mask(df['idioma_m2'].notna(),2,inplace=True) # Segundo idioma materno cuando se declara alguno
df['n_idiomas']=df['aux']+df['IDIOMAS'].where(df['IDIOMAS'].notna(),0) # Idiomas no maternos si hay dato
a_lista('n_idiomas')
del df['aux']
df[['IDIOMAS_','n_idiomas']].groupby('IDIOMAS_',dropna=False).agg(['unique']) # Comprobación

,n_idiomas
,unique
IDIOMAS_,
1,"[2.0, 3.0]"
2,"[3.0, 4.0]"
3,"[5.0, 4.0]"
4,"[6.0, 5.0]"
5 o más,"[6.0, 7.0]"
Ninguno,"[1.0, 2.0]"
NaN,"[1.0, 2.0]"


In [526]:
proporciones('n_idiomas').sort_values()

7.0    0.001896
6.0    0.008341
5.0    0.035891
1.0    0.040125
4.0    0.146788
2.0    0.379388
3.0    0.387571
Name: n_idiomas, dtype: float64

## 'IDIOMA1'  (redefinida como 'idioma1')

(Solo para quienes declaran conocer algún idioma distinto de los maternos -respuesta distinta de 0 en 'IDIOMAS')

**Primer idioma distinto de su lengua materna** que mejor conoce.

In [527]:
df['IDIOMA1'].sort_values().unique()

array(['  ', '01', '02', '03', '04', '05', '06', '07', '08', '09', '10',
       '11', '12', '13', '14', '15', '99'], dtype=object)

In [528]:
def re_idiomas(col,nuevo=None,si_no=False,redef=True):
    """Procesa las variables referidas a los idiomas no maternos y el nivel de conocimiento de estos. 
    """
    traduce(col)
    reset(col)
    if '  ' in df[col].unique() or '99' in df[col].unique():
        no_respuesta(col,blank='  ',nsnc='99')
    else: 
        no_respuesta(col)
    if redef==True:
        if nuevo==None:
            nuevo=col.lower()
        redefine(col,nuevo,si_no=si_no)
    if str(col)[:-1]=='IDIOMA':
        print(df[[col,'IDIOMAS']].groupby('IDIOMAS').agg(['unique'])) # Inspección de exclusiones
    return

In [529]:
re_idiomas('IDIOMA1')

             aux_traduce
                  unique
IDIOMA1                 
          [No aplicable]
01             [Español]
02             [Catalán]
03             [Euskera]
04             [Gallego]
05          [Valenciano]
06              [Inglés]
07             [Francés]
08              [Alemán]
09            [Italiano]
10           [Portugués]
11              [Rumano]
12               [Árabe]
13               [Chino]
14                [Ruso]
15       [Otros idiomas]
99               [NS/NC]
[ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. nan]
                                                                                        IDIOMA1
                                                                                         unique
IDIOMAS                                                                                        
0.0                                                                                       [nan]
1.0            [6.0, 7.0, nan, 3.0, 15.0, 9.0, 1.0

In [530]:
proporciones('idioma1_')
# El inglés es el único idioma no materno más frecuente que no conocer ningún idioma extranjero. 

Inglés           0.781681
NaN              0.067202
Francés          0.037977
Español          0.031974
Catalán          0.022243
Valenciano       0.014944
Euskera          0.012764
Italiano         0.011437
Portugués        0.006603
Alemán           0.005182
Gallego          0.004929
Otros idiomas    0.002022
Árabe            0.000442
Ruso             0.000221
Chino            0.000190
Rumano           0.000190
Name: idioma1_, dtype: float64

## 'NIV_ID1'  (redefinida como 'idioma1_nivel')

(Solo para quienes declaran conocer algún idioma distinto de los maternos -respuesta distinta de 0 en 'IDIOMAS')

**Nivel de conocimiento del primer idioma no materno** que mejor conoce.

In [531]:
df['NIV_ID1'].unique()

array(['2', '1', ' ', '3', '9'], dtype=object)

In [532]:
re_idiomas('NIV_ID1','idioma1_nivel')

                                                                                              aux_traduce
                                                                                                   unique
NIV_ID1                                                                                                  
                                                                                           [No aplicable]
1        [Nivel alto (Entiende una gran variedad de textos complicados y usa el idioma con flexibilidad)]
2                    [Nivel medio (Entiende lo esencial en el lenguaje común y redacta textos sencillos)]
3                                              [Nivel bajo (Entiende y usa las expresiones más comunes )]
9                                                                                                 [NS/NC]
[ 1.  2.  3. nan]


In [533]:
proporciones('idioma1_nivel_')

Nivel alto (Entiende una gran variedad de textos complicados y usa el idioma con flexibilidad)    0.455341
Nivel medio (Entiende lo esencial en el lenguaje común y redacta textos sencillos)                0.402325
NaN                                                                                               0.074658
Nivel bajo (Entiende y usa las expresiones más comunes )                                          0.067676
Name: idioma1_nivel_, dtype: float64

## 'ACREIDI1'  

(Solo para quienes declaran conocer algún idioma distinto de los maternos -respuesta distinta de 0 en 'IDIOMAS')

¿Tiene **acreditación oficial** del primer idioma no materno?

In [534]:
df['ACREIDI1'].unique()

array(['1', '2', ' ', '9'], dtype=object)

In [535]:
re_idiomas('ACREIDI1',redef=False)

             aux_traduce
                  unique
ACREIDI1                
          [No aplicable]
1                   [Sí]
2                   [No]
9                [NS/NC]
[ 1.  2. nan]


## 'TIPOACREIDI1'  (redefinida como 'idioma1_acred')

(Solo para quienes declaran conocer algún idioma distintos de los maternos -respuesta distinta de 0 en 'IDIOMAS')

Máximo **nivel de acreditación oficial** del primer idioma no materno.

In [536]:
reset('TIPOACREIDI1')

In [537]:
df['TIPOACREIDI1'].sort_values().unique()

array([' ', '1', '2', '3', '4', '5', '6', '9'], dtype=object)

In [538]:
re_idiomas('TIPOACREIDI1','idioma1_acred')

                                       aux_traduce
                                            unique
TIPOACREIDI1                                      
                                    [No_aplicable]
1             [A1 (DELF A1, Fit in Deutsch 1, ….)]
2              [A2 (DELF A2, Fit in Deutsch 2, …)]
3                       [B1 (PET, DELF B1, ZD, …)]
4                     [B2 (FCE, DELF B2, ZDfB, …)]
5                      [C1 (CAE, DALF C1, PWD, …)]
6                      [C2 (CPE, DALF C2, ZOP, …)]
9                                          [NS/NC]
[ 1.  2.  3.  4.  5.  6. nan]


In [539]:
proporciones('idioma1_acred_',dropna=True)

B2 (FCE, DELF B2, ZDfB, …)            0.372167
B1 (PET, DELF B1, ZD, …)              0.264555
C1 (CAE, DALF C1, PWD, …)             0.258709
C2 (CPE, DALF C2, ZOP, …)             0.069136
A2 (DELF A2, Fit in Deutsch 2, …)     0.024875
A1 (DELF A1, Fit in Deutsch 1, ….)    0.010558
Name: idioma1_acred_, dtype: float64

## 'IDIOMA2'  (redefinida como 'idioma2')

(Solo para quienes declaran conocer dos idiomas distintos de los maternos -respuesta mayor que 1 en 'IDIOMAS')

**Segundo idioma distinto de su lengua materna** que mejor conoce.

In [540]:
df['IDIOMA2'].sort_values().unique()

array(['  ', '01', '02', '03', '04', '05', '06', '07', '08', '09', '10',
       '11', '12', '13', '14', '15', '99'], dtype=object)

In [541]:
re_idiomas('IDIOMA2')

             aux_traduce
                  unique
IDIOMA2                 
          [No aplicable]
01             [Español]
02             [Catalán]
03             [Euskera]
04             [Gallego]
05          [Valenciano]
06              [Inglés]
07             [Francés]
08              [Alemán]
09            [Italiano]
10           [Portugués]
11              [Rumano]
12               [Árabe]
13               [Chino]
14                [Ruso]
15       [Otros idiomas]
99               [NS/NC]
[ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. nan]
                                                                                        IDIOMA2
                                                                                         unique
IDIOMAS                                                                                        
0.0                                                                                       [nan]
1.0                                               

## 'NIV_ID2'  (redefinida como 'idioma2_nivel')

(Solo para quienes declaran conocer dos idiomas distintos de los maternos -respuesta mayor que 1 en 'IDIOMAS')

**Nivel de conocimiento del segundo idioma no materno** que mejor conoce.

In [542]:
df['NIV_ID2'].unique()

array([' ', '2', '3', '1', '9'], dtype=object)

In [543]:
re_idiomas('NIV_ID2','idioma2_nivel')

                                                                                              aux_traduce
                                                                                                   unique
NIV_ID2                                                                                                  
                                                                                           [No aplicable]
1        [Nivel alto (Entiende una gran variedad de textos complicados y usa el idioma con flexibilidad)]
2                    [Nivel medio (Entiende lo esencial en el lenguaje común y redacta textos sencillos)]
3                                              [Nivel bajo (Entiende y usa las expresiones más comunes )]
9                                                                                                 [NS/NC]
[ 1.  2.  3. nan]


## 'ACREIDI2'  

(Solo para quienes declaran conocer dos idiomas distintos de los maternos -respuesta mayor que 1 en 'IDIOMAS')

¿Tiene **acreditación oficial** del segundo idioma no materno?

In [544]:
df['ACREIDI2'].unique()

array([' ', '2', '1', '9'], dtype=object)

In [545]:
re_idiomas('ACREIDI2',redef=False)

             aux_traduce
                  unique
ACREIDI2                
          [No aplicable]
1                   [Sí]
2                   [No]
9                [NS/NC]
[ 1.  2. nan]


## 'TIPOACREIDI2'  (redefinida como 'idioma2_acred')

(Solo para quienes declaran conocer dos idiomas distintos de los maternos -respuesta mayor que 1 en 'IDIOMAS')

Máximo **nivel de acreditación oficial** del segundo idioma no materno.

In [546]:
df['TIPOACREIDI2'].unique()

array([' ', '3', '4', '5', '6', '2', '9', '1'], dtype=object)

In [547]:
re_idiomas('TIPOACREIDI2','idioma2_acred')

                                       aux_traduce
                                            unique
TIPOACREIDI2                                      
                                    [No_aplicable]
1             [A1 (DELF A1, Fit in Deutsch 1, ….)]
2              [A2 (DELF A2, Fit in Deutsch 2, …)]
3                       [B1 (PET, DELF B1, ZD, …)]
4                     [B2 (FCE, DELF B2, ZDfB, …)]
5                      [C1 (CAE, DALF C1, PWD, …)]
6                      [C2 (CPE, DALF C2, ZOP, …)]
9                                          [NS/NC]
[ 1.  2.  3.  4.  5.  6. nan]


### 'IDIOMA3'  (redefinida como 'idioma3')

(Solo para quienes declaran conocer tres idiomas distintos de los maternos -respuesta mayor que 2 en 'IDIOMAS')

**Tercer idioma distinto de su lengua materna** que mejor conoce.

In [548]:
df['IDIOMA3'].sort_values().unique()

array(['  ', '01', '02', '03', '04', '05', '06', '07', '08', '09', '10',
       '11', '12', '13', '14', '15', '99'], dtype=object)

In [549]:
re_idiomas('IDIOMA3')

             aux_traduce
                  unique
IDIOMA3                 
          [No aplicable]
01             [Español]
02             [Catalán]
03             [Euskera]
04             [Gallego]
05          [Valenciano]
06              [Inglés]
07             [Francés]
08              [Alemán]
09            [Italiano]
10           [Portugués]
11              [Rumano]
12               [Árabe]
13               [Chino]
14                [Ruso]
15       [Otros idiomas]
99               [NS/NC]
[ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. nan]
                                                                                        IDIOMA3
                                                                                         unique
IDIOMAS                                                                                        
0.0                                                                                       [nan]
1.0                                               

## 'NIV_ID3'  (redefinida como 'idioma3_nivel')

(Solo para quienes declaran conocer tres idiomas distintos de los maternos -respuesta mayor que 2 en 'IDIOMAS')

**Nivel de conocimiento del tercer idioma no materno** que mejor conoce.

In [550]:
df['NIV_ID3'].unique()

array([' ', '2', '3', '1', '9'], dtype=object)

In [551]:
re_idiomas('NIV_ID3','idioma3_nivel')

                                                                                              aux_traduce
                                                                                                   unique
NIV_ID3                                                                                                  
                                                                                           [No aplicable]
1        [Nivel alto (Entiende una gran variedad de textos complicados y usa el idioma con flexibilidad)]
2                    [Nivel medio (Entiende lo esencial en el lenguaje común y redacta textos sencillos)]
3                                              [Nivel bajo (Entiende y usa las expresiones más comunes )]
9                                                                                                 [NS/NC]
[ 1.  2.  3. nan]


## 'ACREIDI3'  

(Solo para quienes declaran conocer tres idiomas distintos de los maternos -respuesta mayor que 2 en 'IDIOMAS')

¿Tiene **acreditación oficial** del tercer idioma no materno?

In [552]:
df['ACREIDI3'].unique()

array([' ', '1', '2', '9'], dtype=object)

In [553]:
re_idiomas('ACREIDI3',redef=False)

             aux_traduce
                  unique
ACREIDI3                
          [No aplicable]
1                   [Sí]
2                   [No]
9                [NS/NC]
[ 1.  2. nan]


## 'TIPOACREIDI3'  (redefinida como 'idioma3_acred')

(Solo para quienes declaran conocer tres idiomas distintos de los maternos -respuesta mayor que 2 en 'IDIOMAS')

Máximo **nivel de acreditación oficial** del tercer idioma no materno.

In [554]:
df['TIPOACREIDI3'].unique()

array([' ', '4', '3', '2', '5', '1', '6', '9'], dtype=object)

In [555]:
re_idiomas('TIPOACREIDI3','idioma3_acred')

                                       aux_traduce
                                            unique
TIPOACREIDI3                                      
                                    [No_aplicable]
1             [A1 (DELF A1, Fit in Deutsch 1, ….)]
2              [A2 (DELF A2, Fit in Deutsch 2, …)]
3                       [B1 (PET, DELF B1, ZD, …)]
4                     [B2 (FCE, DELF B2, ZDfB, …)]
5                      [C1 (CAE, DALF C1, PWD, …)]
6                      [C2 (CPE, DALF C2, ZOP, …)]
9                                          [NS/NC]
[ 1.  2.  3.  4.  5.  6. nan]


## 'IDIOMA4'  (redefinida como 'idioma4')

(Solo para quienes declaran conocer cuatro idiomas distintos de los maternos -respuesta mayor que 3 en 'IDIOMAS')

**Cuarto idioma distinto de su lengua materna** que mejor conoce.

In [556]:
df['IDIOMA4'].sort_values().unique()

array(['  ', '01', '02', '03', '04', '05', '06', '07', '08', '09', '10',
       '11', '12', '13', '14', '15', '99'], dtype=object)

In [557]:
re_idiomas('IDIOMA4')

             aux_traduce
                  unique
IDIOMA4                 
          [No aplicable]
01             [Español]
02             [Catalán]
03             [Euskera]
04             [Gallego]
05          [Valenciano]
06              [Inglés]
07             [Francés]
08              [Alemán]
09            [Italiano]
10           [Portugués]
11              [Rumano]
12               [Árabe]
13               [Chino]
14                [Ruso]
15       [Otros idiomas]
99               [NS/NC]
[ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. nan]
                                                                                        IDIOMA4
                                                                                         unique
IDIOMAS                                                                                        
0.0                                                                                       [nan]
1.0                                               

## 'NIV_ID4'  (redefinida como 'idioma4_nivel')

(Solo para quienes declaran conocer cuatro idiomas distintos de los maternos -respuesta mayor que 3 en 'IDIOMAS')

**Nivel de conocimiento del cuarto idioma no materno** que mejor conoce.

In [558]:
df['NIV_ID4'].unique()

array([' ', '3', '2', '1', '9'], dtype=object)

In [559]:
re_idiomas('NIV_ID4','idioma4_nivel')

                                                                                              aux_traduce
                                                                                                   unique
NIV_ID4                                                                                                  
                                                                                           [No aplicable]
1        [Nivel alto (Entiende una gran variedad de textos complicados y usa el idioma con flexibilidad)]
2                    [Nivel medio (Entiende lo esencial en el lenguaje común y redacta textos sencillos)]
3                                              [Nivel bajo (Entiende y usa las expresiones más comunes )]
9                                                                                                 [NS/NC]
[ 1.  2.  3. nan]


## 'ACREIDI4'  

(Solo para quienes declaran conocer cuatro idiomas distintos de los maternos -respuesta mayor que 3 en 'IDIOMAS')

¿Tiene **acreditación oficial** del cuarto idioma no materno?

In [560]:
df['ACREIDI4'].unique()

array([' ', '1', '2', '9'], dtype=object)

In [561]:
re_idiomas('ACREIDI4',redef=False)

             aux_traduce
                  unique
ACREIDI4                
          [No aplicable]
1                   [Sí]
2                   [No]
9                [NS/NC]
[ 1.  2. nan]


## 'TIPOACREIDI4'  (redefinida como 'idioma4_acred')

(Solo para quienes declaran conocer cuatro idiomas distintos de los maternos -respuesta mayor que 3 en 'IDIOMAS')

Máximo **nivel de acreditación oficial** del cuarto idioma no materno.

In [562]:
df['TIPOACREIDI4'].unique()

array([' ', '2', '1', '3', '4', '5', '9', '6'], dtype=object)

In [563]:
re_idiomas('TIPOACREIDI4','idioma4_acred')

                                       aux_traduce
                                            unique
TIPOACREIDI4                                      
                                    [No_aplicable]
1             [A1 (DELF A1, Fit in Deutsch 1, ….)]
2              [A2 (DELF A2, Fit in Deutsch 2, …)]
3                       [B1 (PET, DELF B1, ZD, …)]
4                     [B2 (FCE, DELF B2, ZDfB, …)]
5                      [C1 (CAE, DALF C1, PWD, …)]
6                      [C2 (CPE, DALF C2, ZOP, …)]
9                                          [NS/NC]
[ 1.  2.  3.  4.  5.  6. nan]


In [564]:
for i in range(1,5):
    D_describe['idioma'+str(i)+'_nivel']='Nivel de conocimiento del '+L_ordinal[i-1]+\
    ' idioma no materno que mejor conoce'
    D_describe['idioma'+str(i)+'_acred']='Nivel máximo de acreditación oficial del '+L_ordinal[i-1]+\
    ' idioma no materno que mejor conoce'

## 'TITIDI'  

(Solo para quienes declaran tener acreditación oficial de algún idioma distinto de los maternos -respuesta Sí para alguna variable 'ACREIDIi' (i=1,2,...,4)-)

Tiene título de la **Escuela Oficial de Idiomas**. 

In [565]:
df['TITIDI'].unique()

array(['2', '1', ' ', '9'], dtype=object)

In [566]:
traduce('TITIDI')

           aux_traduce
                unique
TITIDI                
        [No aplicable]
1                 [Sí]
2                 [No]
9              [NS/NC]


In [567]:
df[['TITIDI_','ACREIDI1_','ACREIDI2_','ACREIDI3_','ACREIDI4_']]\
            .groupby(['ACREIDI1_','ACREIDI2_','ACREIDI3_','ACREIDI4_']).agg(['unique']) #Inspección

TITIDI_
                                                 unique
ACREIDI1_ ACREIDI2_ ACREIDI3_ ACREIDI4_                
No        No        No        No         [No aplicable]
                              Sí               [No, Sí]
                    Sí        No               [No, Sí]
                              Sí               [No, Sí]
          Sí        No        No               [Sí, No]
                              Sí               [No, Sí]
                    Sí        No               [No, Sí]
                              Sí               [Sí, No]
Sí        No        No        No               [No, Sí]
                              Sí               [No, Sí]
                    Sí        No               [No, Sí]
                              Sí               [Sí, No]
          Sí        No        No               [No, Sí]
                              Sí               [Sí, No]
                    Sí        No               [Sí, No]
                              Sí               [Sí, No]

## 'TIC'  

Capacidad para usar el ordenador u otros **dispositivos informáticos**.

In [568]:
df['TIC'].unique()

array([2, 3, 1, 9])

In [569]:
traduce('TIC')

                                                                                                                                                                                                         aux_traduce
                                                                                                                                                                                                              unique
TIC                                                                                                                                                                                                                 
1    [Usuario de nivel básico (navegar por internet, enviar correos electrónicos, copiar o mover archivos o carpetas, escribir un texto usando un procesador de textos, usar fórmulas simples en hojas de cálculo) ]
2                                   [Usuario de nivel intermedio (dar formato a textos, usar fórmulas más avanzadas y crear gráficos en hojas de cál

In [570]:
no_respuesta('TIC',tipo='int')

[ 1.  2.  3. nan]


In [571]:
# Diccionario abreviado para 'TIC':
for k,v in D_dict['TIC'].items():
    D_dict['TIC'][k]=v.split('(')[0]

In [572]:
redefine('TIC','ti')

In [573]:
proporciones('ti_')

Usuario de nivel intermedio     0.671069
Usuario de nivel avanzado       0.208398
Usuario de nivel básico         0.118985
NaN                             0.001548
Name: ti_, dtype: float64

## 'EST_B38_1'  

In [574]:
D_describe['EST_B38_1']

'Ha realizado cursos conducentes a Certificado de Profesionalidad Nivel 1'

In [575]:
df['EST_B38_1'].unique()

array([2, 1])

In [576]:
traduce('EST_B38_1')

          aux_traduce
               unique
EST_B38_1            
1                [Sí]
2                [No]


In [577]:
proporciones('EST_B38_1_')

No    0.965088
Sí    0.034912
Name: EST_B38_1_, dtype: float64

In [578]:
reset('EST_B38_1_')

## 'EST_B39_1'  

In [579]:
D_describe['EST_B39_1']

'Le ha sido útil para el trabajo los cursos conducentes a Certificado de Profesionalidad Nivel 1'

In [580]:
df['EST_B39_1'].unique()

array([' ', '1', '2', '9'], dtype=object)

In [581]:
traduce('EST_B39_1')

              aux_traduce
                   unique
EST_B39_1                
           [No aplicable]
1                    [Sí]
2                    [No]
9                 [NS/NC]


In [582]:
reset('EST_B39_1_')

## 'EST_B38_2'  

In [583]:
D_describe['EST_B38_2']

'Ha realizado cursos conducentes a Certificado de Profesionalidad Nivel 2'

In [584]:
df['EST_B38_2'].unique()

array([2, 1])

In [585]:
traduce('EST_B38_2')

          aux_traduce
               unique
EST_B38_2            
1                [Sí]
2                [No]


In [586]:
proporciones('EST_B38_2_')

No    0.980506
Sí    0.019494
Name: EST_B38_2_, dtype: float64

In [587]:
reset('EST_B38_2_')

## 'EST_B39_2'  

In [588]:
D_describe['EST_B39_2']

'Le ha sido útil para el trabajo los cursos conducentes a Certificado de Profesionalidad Nivel 2'

In [589]:
df['EST_B39_2'].unique()

array([' ', '1', '2', '9'], dtype=object)

In [590]:
traduce('EST_B39_2')

              aux_traduce
                   unique
EST_B39_2                
           [No aplicable]
1                    [Sí]
2                    [No]
9                 [NS/NC]


In [591]:
reset('EST_B39_2_')

### 'EST_B38_3'  

In [592]:
D_describe['EST_B38_3']

'Ha realizado cursos conducentes a Certificado de Profesionalidad Nivel 3'

In [593]:
df['EST_B38_3'].unique()

array([2, 1])

In [594]:
traduce('EST_B38_3')

          aux_traduce
               unique
EST_B38_3            
1                [Sí]
2                [No]


In [595]:
proporciones('EST_B38_3_')

No    0.967837
Sí    0.032163
Name: EST_B38_3_, dtype: float64

In [596]:
reset('EST_B38_3_')

## 'EST_B39_3'  

In [597]:
D_describe['EST_B39_3']

'Le ha sido útil para el trabajo los cursos conducentes a Certificado de Profesionalidad Nivel 3'

In [598]:
df['EST_B39_3'].unique()

array([' ', '2', '1', '9'], dtype=object)

In [599]:
traduce('EST_B39_3')

              aux_traduce
                   unique
EST_B39_3                
           [No aplicable]
1                    [Sí]
2                    [No]
9                 [NS/NC]


In [600]:
proporciones('EST_B39_3_')

No aplicable    0.967837
Sí              0.017788
No              0.013996
NS/NC           0.000379
Name: EST_B39_3_, dtype: float64

In [601]:
reset('EST_B39_3_')

## 'EST_B38_4'  

In [602]:
D_describe['EST_B38_4']

'Ha realizado otros cursos de Formación Ocupacional de los Servicios Públicos de Empleo'

In [603]:
df['EST_B38_4'].unique()

array([2, 1])

In [604]:
traduce('EST_B38_4')

          aux_traduce
               unique
EST_B38_4            
1                [Sí]
2                [No]


In [605]:
proporciones('EST_B38_4_')

No    0.888755
Sí    0.111245
Name: EST_B38_4_, dtype: float64

In [606]:
reset('EST_B38_4_')

## 'EST_B39_4'  

In [607]:
D_describe['EST_B39_4']

'Le ha sido útil para el trabajo los otros cursos de Formación Ocupacional de los Servicios Públicos de Empleo'

In [608]:
df['EST_B39_4'].unique()

array([' ', '2', '1', '9'], dtype=object)

In [609]:
traduce('EST_B39_4')

              aux_traduce
                   unique
EST_B39_4                
           [No aplicable]
1                    [Sí]
2                    [No]
9                 [NS/NC]


In [610]:
proporciones('EST_B39_4_')

No aplicable    0.888755
No              0.057692
Sí              0.052795
NS/NC           0.000758
Name: EST_B39_4_, dtype: float64

In [611]:
reset('EST_B39_4_')

## 'EST_B38_5'  

In [612]:
D_describe['EST_B38_5']

'Ha realizado cursos de idiomas'

In [613]:
df['EST_B38_5'].unique()

array([1, 2])

In [614]:
traduce('EST_B38_5')

          aux_traduce
               unique
EST_B38_5            
1                [Sí]
2                [No]


In [615]:
proporciones('EST_B38_5_')

No    0.536666
Sí    0.463334
Name: EST_B38_5_, dtype: float64

In [616]:
reset('EST_B38_5_')

## 'EST_B39_5'  

In [617]:
D_describe['EST_B39_5']

'Le ha sido útil para el trabajo los cursos de idiomas'

In [618]:
df['EST_B39_5'].unique()

array(['1', ' ', '2', '9'], dtype=object)

In [619]:
traduce('EST_B39_5')

              aux_traduce
                   unique
EST_B39_5                
           [No aplicable]
1                    [Sí]
2                    [No]
9                 [NS/NC]


In [620]:
proporciones('EST_B39_5_')

No aplicable    0.536666
Sí              0.263846
No              0.197940
NS/NC           0.001548
Name: EST_B39_5_, dtype: float64

In [621]:
reset('EST_B39_5_')

## 'EST_B38_6'  

In [622]:
D_describe['EST_B38_6']

'Ha realizado cursos de informática'

In [623]:
df['EST_B38_6'].unique()

array([2, 1])

In [624]:
traduce('EST_B38_6')

          aux_traduce
               unique
EST_B38_6            
1                [Sí]
2                [No]


In [625]:
proporciones('EST_B38_6_')

No    0.779375
Sí    0.220625
Name: EST_B38_6_, dtype: float64

In [626]:
reset('EST_B38_6_')

## 'EST_B39_6'  

In [627]:
D_describe['EST_B39_6']

'Le ha sido útil para el trabajo los cursos de informática'

In [628]:
df['EST_B39_6'].unique()

array([' ', '2', '1', '9'], dtype=object)

In [629]:
traduce('EST_B39_6')

              aux_traduce
                   unique
EST_B39_6                
           [No aplicable]
1                    [Sí]
2                    [No]
9                 [NS/NC]


In [630]:
proporciones('EST_B39_6_')

No aplicable    0.779375
Sí              0.131591
No              0.087770
NS/NC           0.001264
Name: EST_B39_6_, dtype: float64

In [631]:
reset('EST_B39_6_')

## 'EST_B38_7'  

In [632]:
D_describe['EST_B38_7']

'Ha realizado otros estudios no reglados'

In [633]:
df['EST_B38_7'].unique()

array([2, 1])

In [634]:
traduce('EST_B38_7')

          aux_traduce
               unique
EST_B38_7            
1                [Sí]
2                [No]


In [635]:
proporciones('EST_B38_7_')

No    0.560741
Sí    0.439259
Name: EST_B38_7_, dtype: float64

In [636]:
reset('EST_B38_7_')

## 'EST_B39_7'  

In [637]:
D_describe['EST_B39_7']

'Le ha sido útil para el trabajo los otros estudios no reglados'

In [638]:
df['EST_B39_7'].unique()

array([' ', '1', '2', '9'], dtype=object)

In [639]:
traduce('EST_B39_7')

              aux_traduce
                   unique
EST_B39_7                
           [No aplicable]
1                    [Sí]
2                    [No]
9                 [NS/NC]


In [640]:
proporciones('EST_B39_7_')

No aplicable    0.560741
Sí              0.261698
No              0.169284
NS/NC           0.008278
Name: EST_B39_7_, dtype: float64

In [641]:
reset('EST_B39_7_')

# BLOQUE C. MOVILIDAD

Se omite. 

# BLOQUE D. SITUACIÓN ACTUAL DEL GRADUADO UNIVERSITARIO

## 'TRBPRN1'  (redefinida como 'sit_lab')

**Situación laboral** actual.

In [642]:
df['TRBPRN1'].unique() # No missing values

array([1, 3, 2])

In [643]:
traduce('TRBPRN1')

            aux_traduce
                 unique
TRBPRN1                
1          [Trabajando]
2        [En desempleo]
3            [Inactivo]


In [644]:
no_respuesta('TRBPRN1')

[1. 2. 3.]


In [645]:
redefine('TRBPRN1','sit_lab')

In [646]:
proporciones('sit_lab_')

Trabajando      0.856971
En desempleo    0.077028
Inactivo        0.066001
Name: sit_lab_, dtype: float64

## 'TRBPRN2'  (redefinida como 'sit_lab_detalle')

(Solo para quienes declaran **situación de inactividad** -respuesta 3 en 'TRBPRN1' (o 'sit_lab')-)

Tipo de **situación de inactividad**.  

Nota: 'sit_lab_detalle se define para todas las situaciones laborales. Se añadirá el detalle disponible para ocupados y para desempleados.

In [647]:
df['TRBPRN2'].unique() # Missing values as ' ' or '9'

array([' ', '1', '4', '5', '9', '2', '3'], dtype=object)

In [648]:
traduce('TRBPRN2')

                                                 aux_traduce
                                                      unique
TRBPRN2                                                     
                                              [No aplicable]
1        [Estudiante (estudiante o preparando oposiciones) ]
2                                                 [Jubilado]
3                               [Incapacitado para trabajar]
4                         [Dedicado a las labores del hogar]
5                                           [Otra situación]
9                                                    [NS/NC]


In [649]:
no_respuesta('TRBPRN2')

[ 1.  2.  3.  4.  5. nan]


In [650]:
# Se imputa None a respuesta en blanco y NS/NC:
redefine('TRBPRN2','sit_lab_detalle')
D_describe['sit_lab_detalle']='Tipo de situación laboral (detalle)'

In [651]:
df[['sit_lab_','sit_lab_detalle_']].groupby('sit_lab_').agg(['unique']) # Comprobación: 

,sit_lab_detalle_
,unique
sit_lab_,
En desempleo,[nan]
Inactivo,"[Estudiante (estudiante o preparando oposiciones) , Dedicado a las labores del hogar, Otra situación, nan, Jubilado, Incapacitado para trabajar]"
Trabajando,[nan]


In [652]:
proporciones('sit_lab_detalle_',dropna=True)

Estudiante (estudiante o preparando oposiciones)     0.680610
Otra situación                                       0.126476
Dedicado a las labores del hogar                     0.105315
Jubilado                                             0.052657
Incapacitado para trabajar                           0.034941
Name: sit_lab_detalle_, dtype: float64

## 'TRINAC_D4'  (redefinida como 'busqueda_tr')

(Solo para quienes declaran **situación de inactividad** -respuesta 3 en 'TRBPRN1' (o 'sit_lab')-)

¿Ha **buscado trabajo** desde que se tituló?  

Nota: 'busqueda_tr' se redefine incluyendo esta información para todas las situaciones laborales cuando esté disponible. 

In [653]:
df['TRINAC_D4'].unique() # Missing values as ' ' or '9'

array([' ', '1', '2', '9'], dtype=object)

In [654]:
traduce('TRINAC_D4')

              aux_traduce
                   unique
TRINAC_D4                
           [No aplicable]
1                    [Sí]
2                    [No]
9                 [NS/NC]


In [655]:
df[['sit_lab_','TRINAC_D4']].groupby('sit_lab_',dropna=False).agg(['unique']) # Inspección

,TRINAC_D4
,unique
sit_lab_,
En desempleo,[ ]
Inactivo,"[1, 2, 9]"
Trabajando,[ ]


In [656]:
no_respuesta('TRINAC_D4')

[ 1.  2. nan]


In [657]:
redefine('TRINAC_D4','busqueda_tr',si_no=True)

            aux_traduce
                 unique
busqueda_tr            
0.0                [No]
1.0                [Sí]


In [658]:
proporciones('busqueda_tr_',dropna=True)

Sí    0.781085
No    0.218915
Name: busqueda_tr_, dtype: float64

In [659]:
D_describe['busqueda_tr']='¿Ha buscado empleo alguna vez desde que se tituló?'

## 'TRINAC_D5'  

(Solo para quienes declaran **situación de inactividad** -respuesta 3 en 'TRBPRN1' (o 'sit_lab')-)

**Momento en el que empezó a buscar trabajo**.

In [660]:
df['TRINAC_D5'].unique() # Missing values as ' ' or '9'

array([' ', '2', '1', '9'], dtype=object)

In [661]:
traduce('TRINAC_D5')

                                                                      aux_traduce
                                                                           unique
TRINAC_D5                                                                        
                                                                   [No aplicable]
1            [Antes de acabar los estudios de la titulación que finalizó en 2014]
2          [Después de acabar los estudios de la titulación que finalizó en 2014]
9                                                                         [NS/NC]


In [662]:
no_respuesta('TRINAC_D5')

[ 1.  2. nan]


## 'TRINAC_D6'  

(Solo para quienes declaran **situación de inactividad** -respuesta 3 en 'TRBPRN1' (o 'sit_lab')-)

**Tiempo que ha estado buscando trabajo**.

In [663]:
df['TRINAC_D6'].unique() # Missing values as ' ' or '9'

array([' ', '3', '6', '1', '4', '9', '5', '2'], dtype=object)

In [664]:
traduce('TRINAC_D6')

                             aux_traduce
                                  unique
TRINAC_D6                               
                          [No aplicable]
1                  [Menos de tres meses]
2                       [De 3 a 6 meses]
3                   [De 6 meses a 1 año]
4             [De 1 año a año y medio  ]
5          [De 1 año y medio a  2 años ]
6                         [2 ó más años]
9                                [NS/NC]


In [665]:
no_respuesta('TRINAC_D6')

[ 1.  2.  3.  4.  5.  6. nan]


## 'TRPARA_D5'  (complementa la variable  'busqueda_tr)

(Solo para quienes declaran **situación de desempleo** -respuesta 2 en 'TRBPRN1' (o 'sit_lab')-)

**Momento en el que empezó a buscar trabajo**.

Nota: Se usa esta información para completar la recogida en la variable 'busqueda_tr' (definida inicialmente para graduados en situación de inactividad). 

In [666]:
df['TRPARA_D5'].unique() # Missing values as ' ' or '9'

array([' ', '2', '1', '9'], dtype=object)

In [667]:
traduce('TRPARA_D5')

                                                                      aux_traduce
                                                                           unique
TRPARA_D5                                                                        
                                                                   [No aplicable]
1            [Antes de acabar los estudios de la titulación que finalizó en 2014]
2          [Después de acabar los estudios de la titulación que finalizó en 2014]
9                                                                         [NS/NC]


In [668]:
df[['sit_lab_','TRPARA_D5_']].groupby('sit_lab_',dropna=False).agg(['unique']) # Inspección

,TRPARA_D5_
,unique
sit_lab_,
En desempleo,"[Después de acabar los estudios de la titulación que finalizó en 2014, Antes de acabar los estudios de la titulación que finalizó en 2014, NS/NC]"
Inactivo,[No aplicable]
Trabajando,[No aplicable]


In [669]:
no_respuesta('TRPARA_D5')

[ 1.  2. nan]


In [670]:
leyenda('busqueda_tr')

0 	 No
1 	 Sí


In [671]:
# Se añaden a 'busqueda_tr' los desempleados que declaran expresamente haber buscado trabajo: 
# (Se procede así en contradicción con la definición formal de desempleado, que requiere búsqueda de empleo).
df['busqueda_tr'].mask((df['TRPARA_D5']==1)|(df['TRPARA_D5']==2),1,inplace=True)
traduce('busqueda_tr',check=False)
df[['busqueda_tr_','TRPARA_D5_']].groupby('TRPARA_D5_',dropna=False).agg(['unique']) # Comprobación

,busqueda_tr_
,unique
TRPARA_D5_,
Antes de acabar los estudios de la titulación que finalizó en 2014,[Sí]
Después de acabar los estudios de la titulación que finalizó en 2014,[Sí]
NaN,"[nan, Sí, No]"


## 'TRPARA_D6'  

(Solo para quienes declaran **situación de desempleo** -respuesta 2 en 'TRBPRN1' (o 'sit_lab')-)

**Tiempo que ha estado buscando trabajo**.

In [672]:
df['TRPARA_D6'].unique() # Missing values as ' ' or '9'

array([' ', '6', '1', '3', '5', '9', '4', '2'], dtype=object)

In [673]:
traduce('TRPARA_D6')

                             aux_traduce
                                  unique
TRPARA_D6                               
                          [No aplicable]
1                  [Menos de tres meses]
2                       [De 3 a 6 meses]
3                   [De 6 meses a 1 año]
4             [De 1 año a año y medio  ]
5          [De 1 año y medio a  2 años ]
6                         [2 ó más años]
9                                [NS/NC]


In [674]:
no_respuesta('TRPARA_D6')

[ 1.  2.  3.  4.  5.  6. nan]


## 'TRPARA_D7_1'  (redefinida como 'btr_internet')

(Solo para quienes declaran **situación de desempleo** -respuesta 2 en 'TRBPRN1' (o 'sit_lab')-)

(**Búsqueda de empleo**) ¿Busca o ha buscado empleo a través de **ofertas de trabajo en el periódico, internet**, etc.?

In [675]:
df['TRPARA_D7_1'].unique() # Missing values as ' ' or '9'

array([' ', '1', '2', '9'], dtype=object)

In [676]:
# Lista de métodos de búsqueda de empleo entre desempleados: 
L_btr=list()

In [677]:
def btr(col,nuevo):
    """Procesa columnas referidas a métodos de búsqueda de empleo utlizados por los desempleados.
    """
    traduce(col)
    no_respuesta(col)
    redefine(col,nuevo,si_no=True)
    D_describe[nuevo]=D_describe[col].split(':',1)[1].strip()
    L_btr.append(str(nuevo)+'_')
    return

In [678]:
btr('TRPARA_D7_1','btr_internet')

                aux_traduce
                     unique
TRPARA_D7_1                
             [No aplicable]
1                      [Sí]
2                      [No]
9                   [NS/NC]
[ 1.  2. nan]
             aux_traduce
                  unique
btr_internet            
0.0                 [No]
1.0                 [Sí]


## 'TRPARA_D7_2'  (redefinida como 'btr_spe')

(Solo para quienes declaran **situación de desempleo** -respuesta 2 en 'TRBPRN1' (o 'sit_lab')-)

(**Búsqueda de empleo**) ¿Busca o ha buscado empleo a través de **servicios públicos de empleo**?

In [679]:
df['TRPARA_D7_2'].unique() # Missing values as ' ' or '9'

array([' ', '2', '1', '9'], dtype=object)

In [680]:
btr('TRPARA_D7_2','btr_spe')

                aux_traduce
                     unique
TRPARA_D7_2                
             [No aplicable]
1                      [Sí]
2                      [No]
9                   [NS/NC]
[ 1.  2. nan]
        aux_traduce
             unique
btr_spe            
0.0            [No]
1.0            [Sí]


## 'TRPARA_D7_3'  (redefinida como 'btr_univ')

(Solo para quienes declaran **situación de desempleo** -respuesta 2 en 'TRBPRN1' (o 'sit_lab')-)

(**Búsqueda de empleo**) ¿Busca o ha buscado empleo a través de los **servicios de empleo de la universidad** (bolsa de trabajo, asociaciones de exalumnos, etc.)?

In [681]:
df['TRPARA_D7_3'].unique() # Missing values as ' ' or '9'

array([' ', '2', '1', '9'], dtype=object)

In [682]:
btr('TRPARA_D7_3','btr_univ')

                aux_traduce
                     unique
TRPARA_D7_3                
             [No aplicable]
1                      [Sí]
2                      [No]
9                   [NS/NC]
[ 1.  2. nan]
         aux_traduce
              unique
btr_univ            
0.0             [No]
1.0             [Sí]


## 'TRPARA_D7_4'  (redefinida como 'btr_ett')

(Solo para quienes declaran **situación de desempleo** -respuesta 2 en 'TRBPRN1' (o 'sit_lab')-)

(**Búsqueda de empleo**) ¿Busca o ha buscado empleo a través de **empresas de trabajo temporal**?

In [683]:
df['TRPARA_D7_4'].unique() # Missing values as ' ' or '9'

array([' ', '2', '1', '9'], dtype=object)

In [684]:
btr('TRPARA_D7_4','btr_ett')

                aux_traduce
                     unique
TRPARA_D7_4                
             [No aplicable]
1                      [Sí]
2                      [No]
9                   [NS/NC]
[ 1.  2. nan]
        aux_traduce
             unique
btr_ett            
0.0            [No]
1.0            [Sí]


## 'TRPARA_D7_5'  (redefinida como 'btr_bolsa')

(Solo para quienes declaran **situación de desempleo** -respuesta 2 en 'TRBPRN1' (o 'sit_lab')-)

(**Búsqueda de empleo**) ¿Busca o ha buscado empleo a través de una **bolsa de empleo**?

In [685]:
df['TRPARA_D7_5'].unique() # Missing values as ' ' or '9'

array([' ', '2', '1', '9'], dtype=object)

In [686]:
btr('TRPARA_D7_5','btr_bolsa')

                aux_traduce
                     unique
TRPARA_D7_5                
             [No aplicable]
1                      [Sí]
2                      [No]
9                   [NS/NC]
[ 1.  2. nan]
          aux_traduce
               unique
btr_bolsa            
0.0              [No]
1.0              [Sí]


## 'TRPARA_D7_6'  (redefinida como 'btr_contactos')

(Solo para quienes declaran **situación de desempleo** -respuesta 2 en 'TRBPRN1' (o 'sit_lab')-)

(**Búsqueda de empleo**) ¿Busca o ha buscado empleo buscando ayuda entre profesionales conocidos del sector o **contactos personales** (familia, amigos)?

In [687]:
df['TRPARA_D7_6'].unique() # Missing values as ' ' or '9'

array([' ', '1', '2', '9'], dtype=object)

In [688]:
btr('TRPARA_D7_6','btr_contactos')

                aux_traduce
                     unique
TRPARA_D7_6                
             [No aplicable]
1                      [Sí]
2                      [No]
9                   [NS/NC]
[ 1.  2. nan]
              aux_traduce
                   unique
btr_contactos            
0.0                  [No]
1.0                  [Sí]


## 'TRPARA_D7_7'  (redefinida como 'btr_empresas')

(Solo para quienes declaran **situación de desempleo** -respuesta 2 en 'TRBPRN1' (o 'sit_lab')-)

(**Búsqueda de empleo**) ¿Busca o ha buscado empleo **contactando directamente con las empresas**?

In [689]:
df['TRPARA_D7_7'].unique() # Missing values as ' ' or '9'

array([' ', '1', '2', '9'], dtype=object)

In [690]:
btr('TRPARA_D7_7','btr_empresas')

                aux_traduce
                     unique
TRPARA_D7_7                
             [No aplicable]
1                      [Sí]
2                      [No]
9                   [NS/NC]
[ 1.  2. nan]
             aux_traduce
                  unique
btr_empresas            
0.0                 [No]
1.0                 [Sí]


## 'TRPARA_D7_8'  (redefinida como 'btr_opo')

(Solo para quienes declaran **situación de desempleo** -respuesta 2 en 'TRBPRN1' (o 'sit_lab')-)

(**Búsqueda de empleo**) ¿Está preparando o preparó unas **oposiciones**?

In [691]:
df['TRPARA_D7_8'].unique() # Missing values as ' ' or '9'

array([' ', '1', '2', '9'], dtype=object)

In [692]:
btr('TRPARA_D7_8','btr_opo')

                aux_traduce
                     unique
TRPARA_D7_8                
             [No aplicable]
1                      [Sí]
2                      [No]
9                   [NS/NC]
[ 1.  2. nan]
        aux_traduce
             unique
btr_opo            
0.0            [No]
1.0            [Sí]


## 'TRPARA_D7_9'  (redefinida como 'btr_auto')

(Solo para quienes declaran **situación de desempleo** -respuesta 2 en 'TRBPRN1' (o 'sit_lab')-)

(**Búsqueda de empleo**) ¿Busca o ha buscado equipamiento para establecer un **negocio propio**?

In [693]:
df['TRPARA_D7_9'].unique() # Missing values as ' ' or '9'

array([' ', '2', '1', '9'], dtype=object)

In [694]:
btr('TRPARA_D7_9','btr_auto')

                aux_traduce
                     unique
TRPARA_D7_9                
             [No aplicable]
1                      [Sí]
2                      [No]
9                   [NS/NC]
[ 1.  2. nan]
         aux_traduce
              unique
btr_auto            
0.0             [No]
1.0             [Sí]


## 'TRPARA_D7_10'  (redefinida como 'btr_otro')

(Solo para quienes declaran **situación de desempleo** -respuesta 2 en 'TRBPRN1' (o 'sit_lab')-)

(**Búsqueda de empleo**) ¿Busca o ha buscado empleo de **otro modo**?

In [695]:
df['TRPARA_D7_10'].unique() # Missing values as ' ' or '9'

array([' ', '2', '9', '1'], dtype=object)

In [696]:
btr('TRPARA_D7_10','btr_otro')

                 aux_traduce
                      unique
TRPARA_D7_10                
              [No aplicable]
1                       [Sí]
2                       [No]
9                    [NS/NC]
[ 1.  2. nan]
         aux_traduce
              unique
btr_otro            
0.0             [No]
1.0             [Sí]


In [697]:
for col in L_btr:
    col=str(col).strip('_')
    print (col,'\t',D_describe[col])

btr_internet 	 Búsqueda de empleo: a través de ofertas de trabajo en el periódico,  internet…
btr_spe 	 Búsqueda de empleo: a través de los servicios públicos de empleo
btr_univ 	 Búsqueda de empleo: a través de los servicios de empleo de la universidad (bolsa de trabajo, asociaciones de exalumnos, ...)
btr_ett 	 Búsqueda de empleo: a través de empresas de trabajo temporal
btr_bolsa 	 Búsqueda de empleo: a través de una bolsa de empleo
btr_contactos 	 Búsqueda de empleo: buscando ayuda entre profesionales conocidos del sector o contactos personales (familia, amigos)
btr_empresas 	 Búsqueda de empleo: contactando directamente con las empresas
btr_opo 	 Búsqueda de empleo: esta preparando o preparó unas oposiciones
btr_auto 	 Búsqueda de empleo: buscando equipamiento para establecer su propio negocio
btr_otro 	 Búsqueda de empleo: otra forma


In [698]:
df[L_btr].apply(lambda x: x.value_counts(normalize=True))
# Las iniciativas de búsqueda de empleo más frecuentes entre los desempleados son, por este orden: ...
# ... anuncios de ofertas de trabajo, servicios públicos de empleo, contacto directo con empresas y ...
# ... contactos personales. 

,btr_internet_,btr_spe_,btr_univ_,btr_ett_,btr_bolsa_,btr_contactos_,btr_empresas_,btr_opo_,btr_auto_,btr_otro_
No,0.144638,0.394431,0.662926,0.631754,0.571488,0.4202,0.401081,0.594763,0.908146,0.988362
Sí,0.855362,0.605569,0.337074,0.368246,0.428512,0.5798,0.598919,0.405237,0.091854,0.011638


## 'TRPARA_D8_1'  (redefinida como 'dbtr_formacion')

(Solo para quienes declaran **situación de desempleo** -respuesta 2 en 'TRBPRN1' (o 'sit_lab')-)

(**Dificultades para encontrar empleo**) Importancia que atribuye a las **carencias en la formación universitaria recibida** en cuanto a sus dificultades para encontrar trabajo.

In [699]:
df['TRPARA_D8_1'].unique() # Missing values as ' ' or '9'

array([' ', '4', '2', '3', '1', '9'], dtype=object)

In [700]:
# Lista de factores que son una dificultad para encontrar empleo: 
L_dbtr=list()

In [701]:
def dbtr(col,nuevo):
    """Procesa columnas referidas a las dificultades en la búsqueda de trabajo para los desempleados.
    """
    traduce(col)
    no_respuesta(col)
    redefine(col,nuevo)
    D_describe[nuevo]=D_describe[col].split(':',1)[1].strip()
    L_dbtr.append(str(nuevo)+'_')
    return

In [702]:
dbtr('TRPARA_D8_1','dbtr_formacion')

                       aux_traduce
                            unique
TRPARA_D8_1                       
                    [No aplicable]
1                [Nada importante]
2                [Poco importante]
3            [Bastante importante]
4                 [Muy importante]
9                          [NS/NC]
[ 1.  2.  3.  4. nan]


## 'TRPARA_D8_2'  (redefinida como 'dbtr_personales')

(Solo para quienes declaran **situación de desempleo** -respuesta 2 en 'TRBPRN1' (o 'sit_lab')-)

(**Dificultades para encontrar empleo**) Importancia que atribuye a las **actividades personales** que impiden trabajar (seguir estudiando, familia, otras) en cuanto a sus dificultades para encontrar trabajo.

In [703]:
df['TRPARA_D8_2'].unique() # Missing values as ' ' or '9'

array([' ', '1', '2', '4', '3', '9'], dtype=object)

In [704]:
dbtr('TRPARA_D8_2','dbtr_personales')

                       aux_traduce
                            unique
TRPARA_D8_2                       
                    [No aplicable]
1                [Nada importante]
2                [Poco importante]
3            [Bastante importante]
4                 [Muy importante]
9                          [NS/NC]
[ 1.  2.  3.  4. nan]


## 'TRPARA_D8_3'  (redefinida como 'dbtr_experiencia')

(Solo para quienes declaran **situación de desempleo** -respuesta 2 en 'TRBPRN1' (o 'sit_lab')-)

(**Dificultades para encontrar empleo**) Importancia que atribuye a la **falta de experiencia profesional** en cuanto a sus dificultades para encontrar trabajo.

In [705]:
df['TRPARA_D8_3'].unique() # Missing values as ' ' or '9'

array([' ', '3', '2', '4', '1', '9'], dtype=object)

In [706]:
dbtr('TRPARA_D8_3','dbtr_experiencia')

                       aux_traduce
                            unique
TRPARA_D8_3                       
                    [No aplicable]
1                [Nada importante]
2                [Poco importante]
3            [Bastante importante]
4                 [Muy importante]
9                          [NS/NC]
[ 1.  2.  3.  4. nan]


## 'TRPARA_D8_4'  (redefinida como 'dbtr_expectativas')

(Solo para quienes declaran **situación de desempleo** -respuesta 2 en 'TRBPRN1' (o 'sit_lab')-)

(**Dificultades para encontrar empleo**) Importancia que atribuye a la **exigencia de tener un trabajo acorde con sus expectativas** económicas y profesionales en cuanto a sus dificultades para encontrar trabajo.

In [707]:
df['TRPARA_D8_4'].unique() # Missing values as ' ' or '9'

array([' ', '2', '4', '3', '1', '9'], dtype=object)

In [708]:
dbtr('TRPARA_D8_4','dbtr_expectativas')

                       aux_traduce
                            unique
TRPARA_D8_4                       
                    [No aplicable]
1                [Nada importante]
2                [Poco importante]
3            [Bastante importante]
4                 [Muy importante]
9                          [NS/NC]
[ 1.  2.  3.  4. nan]


## 'TRPARA_D8_5'  (redefinida como 'dbtr_idiomas')

(Solo para quienes declaran **situación de desempleo** -respuesta 2 en 'TRBPRN1' (o 'sit_lab')-)

(**Dificultades para encontrar empleo**) Importancia que atribuye a la **falta de conocimientos de idiomas** en cuanto a sus dificultades para encontrar trabajo.

In [709]:
df['TRPARA_D8_5'].unique() # Missing values as ' ' or '9'

array([' ', '4', '2', '3', '1', '9'], dtype=object)

In [710]:
dbtr('TRPARA_D8_5','dbtr_idiomas')

                       aux_traduce
                            unique
TRPARA_D8_5                       
                    [No aplicable]
1                [Nada importante]
2                [Poco importante]
3            [Bastante importante]
4                 [Muy importante]
9                          [NS/NC]
[ 1.  2.  3.  4. nan]


## 'TRPARA_D8_6'  (redefinida como 'dbtr_ti')

(Solo para quienes declaran **situación de desempleo** -respuesta 2 en 'TRBPRN1' (o 'sit_lab')-)

(**Dificultades para encontrar empleo**) Importancia que atribuye a la **falta de conocimientos informáticos** en cuanto a sus dificultades para encontrar trabajo.

In [711]:
df['TRPARA_D8_6'].unique() # Missing values as ' ' or '9'

array([' ', '1', '2', '4', '3', '9'], dtype=object)

In [712]:
dbtr('TRPARA_D8_6','dbtr_ti')

                       aux_traduce
                            unique
TRPARA_D8_6                       
                    [No aplicable]
1                [Nada importante]
2                [Poco importante]
3            [Bastante importante]
4                 [Muy importante]
9                          [NS/NC]
[ 1.  2.  3.  4. nan]


In [713]:
for col in L_dbtr:
    col=str(col).strip('_')
    print (col,'\t',D_describe[col])

dbtr_formacion 	 Dificultades para encontrar empleo: carencias en la formación universitaria recibida
dbtr_personales 	 Dificultades para encontrar empleo: actividades personales que impiden trabajar (seguir estudiando, familia, otras...)
dbtr_experiencia 	 Dificultades para encontrar empleo: falta de experiencia profesional
dbtr_expectativas 	 Dificultades para encontrar empleo: exigencia de tener un trabajo acorde con sus expectativas económicas, profesionales,...
dbtr_idiomas 	 Dificultades para encontrar empleo: falta de conocimientos de idiomas
dbtr_ti 	 Dificultades para encontrar empleo: falta de conocimientos informáticos


In [714]:
# Proporciones de respuestas sobre grado de importancia de cada dificultad en la búsqueda de trabajo:
df_dbtr=df[L_dbtr].apply(lambda x:x.value_counts(normalize=True))
# Para ordenar la salida según orden de importancia:
D_aux=dict()
for k,v in D_dict['dbtr_ti'].items():
    if k in range(1,5):
        D_aux[v]=k
df_dbtr['aux']=pd.Series(D_aux)

In [715]:
df_dbtr.sort_values('aux')
# La dificultad para encontrar empleo más mencionada como muy importante por los graduados desempleados es la ...
# ... falta de experiencia profesional. Le sigue las expectativas personales y profesionales, para la que 
# ... la valoración más frecuente es 'bastente importante'.

,dbtr_formacion_,dbtr_personales_,dbtr_experiencia_,dbtr_expectativas_,dbtr_idiomas_,dbtr_ti_,aux
Nada importante,0.267979,0.373932,0.094835,0.102171,0.250213,0.384384,1
Poco importante,0.332620,0.279915,0.151990,0.235419,0.342442,0.409266,2
Bastante importante,0.226884,0.203419,0.267146,0.344828,0.238258,0.140283,3
Muy importante,0.172517,0.142735,0.486029,0.317582,0.169086,0.066066,4


In [716]:
df_dbtr.rank(ascending=False).sort_values('aux',ascending=False)
# Ninguna otra dificultad tiene como valoración más frecuente 'bastante importante' o 'muy importante'.
# Las valoraciones de las carencias de formación, los idiomas y la informática tienen el ...
# ... mismo orden de frecuencias: la respuesta más frecuente es poco importante, seguido de nada, bastante y ...
# ... muy importante. 

,dbtr_formacion_,dbtr_personales_,dbtr_experiencia_,dbtr_expectativas_,dbtr_idiomas_,dbtr_ti_,aux
Nada importante,2.0,1.0,4.0,4.0,2.0,2.0,4.0
Poco importante,1.0,2.0,3.0,3.0,1.0,1.0,3.0
Bastante importante,3.0,3.0,2.0,1.0,3.0,3.0,2.0
Muy importante,4.0,4.0,1.0,2.0,4.0,4.0,1.0


## 'TRAC_D9'  

(Solo para quienes declaran **situación de ocupación** -respuesta 1 en 'TRBPRN1' (o 'sit_lab')-)  

Actualmente trabaja en **más de un empleo**.

In [717]:
df['TRAC_D9'].unique() # Missing values as ' ' or '9'

array(['2', ' ', '1', '9'], dtype=object)

In [718]:
traduce('TRAC_D9')

            aux_traduce
                 unique
TRAC_D9                
         [No aplicable]
1                  [Sí]
2                  [No]
9               [NS/NC]


In [719]:
no_respuesta('TRAC_D9')

[ 1.  2. nan]


In [720]:
proporciones('TRAC_D9_',dropna=True)

No    0.909014
Sí    0.090986
Name: TRAC_D9_, dtype: float64

## 'LUG_TRAB_PAIS'  (redefinida como 'pais_tr')

(Solo para quienes declaran situación de ocupación -respuesta 1 en 'TRBPRN1' (o 'sit_lab')-)  

**País de trabajo** actual.

In [721]:
df['LUG_TRAB_PAIS'].unique() # Missing values as '  ' or '99'

array(['01', '  ', '05', '07', '02', '08', '03', '10', '11', '06', '04',
       '09', '99'], dtype=object)

In [722]:
traduce('LUG_TRAB_PAIS')

                                                  aux_traduce
                                                       unique
LUG_TRAB_PAIS                                                
                                               [No aplicable]
01                                                   [España]
02                                              [Reino Unido]
03                                                 [Alemania]
04                                                  [Francia]
05                            [Otro país de la Unión Europea]
06                       [Otro país europeo (fuera de la UE)]
07                             [Un país de América del Norte]
08             [Un país de América Central o América del Sur]
09                                          [Un país de Asia]
10                                        [Un país de África]
11                                       [Un país de Oceanía]
99                                                    [NS/NC]


In [723]:
no_respuesta('LUG_TRAB_PAIS',blank='  ',nsnc='99')

[ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. nan]


In [724]:
redefine('LUG_TRAB_PAIS','pais_tr')

In [725]:
proporciones('pais_tr_',dropna=True)

España                                          0.939086
Reino Unido                                     0.017478
Otro país de la Unión Europea                   0.015155
Alemania                                        0.007190
Francia                                         0.004830
Un país de América del Norte                    0.004720
Otro país europeo (fuera de la UE)              0.004388
Un país de América Central o América del Sur    0.003024
Un país de Asia                                 0.001991
Un país de Oceanía                              0.001586
Un país de África                               0.000553
Name: pais_tr_, dtype: float64

## 'SIT_PRO'  

(Solo para quienes declaran **situación de ocupación** -respuesta 1 en 'TRBPRN1' (o 'sit_lab')-)  

**Situación profesional** actual.

In [726]:
df['SIT_PRO'].unique() # Missing values as ' ' 

array(['2', '3', ' ', '1', '4', '5', '6'], dtype=object)

In [727]:
traduce('SIT_PRO')

                                                                              aux_traduce
                                                                                   unique
SIT_PRO                                                                                  
                                                                           [No aplicable]
1               [Trabajador en prácticas, formación (incluido MIR, EIR, FIR,…) o becario]
2        [Asalariado con trabajo permanente o contrato de trabajo de duración indefinida]
3         [Asalariado con trabajo temporal o contrato de trabajo de duración determinada]
4                                                            [Empresario con asalariados]
5                                 [Trabajador independiente o empresario sin asalariados]
6                                                [Ayuda en la empresa o negocio familiar]


In [728]:
no_respuesta('SIT_PRO')

[ 1.  2.  3.  4.  5.  6. nan]


In [729]:
proporciones('SIT_PRO_',dropna=True)

Asalariado con trabajo permanente o contrato de trabajo de duración indefinida    0.573588
Asalariado con trabajo temporal o contrato de trabajo de duración determinada     0.279162
Trabajador independiente o empresario sin asalariados                             0.067542
Trabajador en prácticas, formación (incluido MIR, EIR, FIR,…) o becario           0.055597
Empresario con asalariados                                                        0.018065
Ayuda en la empresa o negocio familiar                                            0.006046
Name: SIT_PRO_, dtype: float64

In [730]:
leyenda('sit_lab')

1 	 Trabajando
2 	 En desempleo
3 	 Inactivo


In [731]:
# Se actualiza la variable 'sit_lab_detalle_':
df['sit_lab_detalle_'].mask(df['sit_lab']==1,df['SIT_PRO_'],inplace=True)
df['sit_lab_detalle_'].mask(df['sit_lab']==2,'Desempleado',inplace=True)
df[['sit_lab_detalle_','sit_lab_']].groupby('sit_lab_',dropna=False).agg(['unique']) # Comprobación

,sit_lab_detalle_
,unique
sit_lab_,
En desempleo,[Desempleado]
Inactivo,"[Estudiante (estudiante o preparando oposiciones) , Dedicado a las labores del hogar, Otra situación, nan, Jubilado, Incapacitado para trabajar]"
Trabajando,"[Asalariado con trabajo permanente o contrato de trabajo de duración indefinida, Asalariado con trabajo temporal o contrato de trabajo de duración determinada, Trabajador en prácticas, formación (incluido MIR, EIR, FIR,…) o becario, Empresario con asalariados, Trabajador independiente o empresar..."


In [732]:
# Se actualiza el diccionario asociado a la variable 'sit_lab_detalle':
D_dict['sit_lab_detalle']=dict()
dict_inv_aux=dict()
for n, valor in enumerate(df['sit_lab_detalle_'][df['sit_lab_detalle_'].notna()].unique()):
    dict_inv_aux[valor]=n+1 # Diccionario inverso para reconstruir la variable categórica
    D_dict['sit_lab_detalle'][n+1]=valor # Nuevo diccionario para la variable
df['sit_lab_detalle']=df['sit_lab_detalle_'].replace(dict_inv_aux)
# Comprobación
df[['sit_lab_detalle','sit_lab_detalle_','sit_lab_']].groupby(['sit_lab_','sit_lab_detalle'],
                                                             dropna=False).agg(['unique']) 

sit_lab_detalle_
                                                                                                        unique
sit_lab_     sit_lab_detalle                                                                                  
En desempleo 4.0                                                                                 [Desempleado]
Inactivo     3.0                                           [Estudiante (estudiante o preparando oposiciones) ]
             6.0                                                            [Dedicado a las labores del hogar]
             7.0                                                                              [Otra situación]
             11.0                                                                                   [Jubilado]
             12.0                                                                 [Incapacitado para trabajar]
             NaN                                                                                         [nan]
Trabajando   1.0              [Asalariado con trabajo permanente o contrato de trabajo de duración indefinida]
             2.0               [Asalariado con trabajo temporal o contrato de trabajo de duración determinada]
             5.0                     [Trabajador en prácticas, formación (incluido MIR, EIR, FIR,…) o becario]
             8.0                                                                  [Empresario con asalariados]
             9.0                                       [Trabajador independiente o empresario sin asalariados]
             10.0                                                     [Ayuda en la empresa o negocio familiar]

In [733]:
leyenda('sit_lab_detalle') # Comprobación

1 	 Asalariado con trabajo permanente o contrato de trabajo de duración indefinida
2 	 Asalariado con trabajo temporal o contrato de trabajo de duración determinada
3 	 Estudiante (estudiante o preparando oposiciones) 
4 	 Desempleado
5 	 Trabajador en prácticas, formación (incluido MIR, EIR, FIR,…) o becario
6 	 Dedicado a las labores del hogar
7 	 Otra situación
8 	 Empresario con asalariados
9 	 Trabajador independiente o empresario sin asalariados
10 	 Ayuda en la empresa o negocio familiar
11 	 Jubilado
12 	 Incapacitado para trabajar


## 'JORNADA'  (redefinida como 'jornada')

(Solo para quienes declaran **situación de ocupación** -respuesta 1 en 'TRBPRN1' (o 'sit_lab')-)  

Tipo de **jornada laboral** actual.

In [734]:
df['JORNADA'].unique() # Missing values as ' ' 

array(['2', ' ', '1'], dtype=object)

In [735]:
traduce('JORNADA')

                 aux_traduce
                      unique
JORNADA                     
                       [nan]
1         [A tiempo parcial]
2        [A tiempo completo]


In [736]:
no_respuesta('JORNADA')

[ 1.  2. nan]


In [737]:
redefine('JORNADA')

In [738]:
proporciones('jornada_',dropna=True)

A tiempo completo    0.861341
A tiempo parcial     0.138659
Name: jornada_, dtype: float64

## 'TRABOC' (redefinida como 'ocupacion')

(Solo para quienes declaran **situación de ocupación** -respuesta 1 en 'TRBPRN1' (o 'sit_lab')-)  

**Ocupación** que desempeña en el puesto de trabajo actual.

In [739]:
df['TRABOC'].nunique() # Missing values as ' ' or '99'

64

In [740]:
leyenda('TRABOC')

11 	 Miembros del poder ejecutivo y de los cuerpos legislativos; directivos de la Administración Pública y organizaciones de interés social; directores ejecutivos
12 	 Directores de departamentos administrativos y comerciales (directores financieros, directores de recursos humanos, directores comerciales, directores de publicidad, directores de desarrollo,…)
13 	 Directores de producción y operaciones: Directores de producción de explotaciones agropecuarias, forestales y pesqueras, y de industrias manufactureras, de minería, construcción,…; Directores de servicios de TIC y de empresas de servicios profesionales
14 	 Directores y gerentes de empresas de alojamiento, restauración y comercio
15 	 Directores y gerentes de otras empresas de servicios no clasificados bajo otros epígrafes: Directores y gerentes de empresas de actividades recreativas, culturales y deportivas; Directores y gerentes de empresas de gestión de residuos y de otros servicios no clasificados bajo otros epígrafes
21 	

In [741]:
traduce('TRABOC',check=False)

In [742]:
no_respuesta('TRABOC',blank='  ',nsnc='99')

[ 0. 11. 12. 13. 14. 15. 21. 22. 23. 24. 25. 26. 27. 28. 29. 31. 32. 33.
 34. 35. 36. 37. 38. 41. 42. 43. 44. 45. 50. 51. 52. 53. 54. 55. 56. 57.
 58. 59. 61. 62. 63. 64. 71. 72. 73. 74. 75. 76. 77. 78. 81. 82. 83. 84.
 91. 92. 93. 94. 95. 96. 97. 98. nan]


In [743]:
redefine('TRABOC','ocupacion')

In [744]:
import re

In [745]:
# Diccionario abreviado para 'ocupacion':
for k,v in D_dict['ocupacion'].items():
    v_aux=re.split('[:.(;]',v)[0]
    D_dict['ocupacion'][k]=v_aux
traduce('ocupacion')

                                                                                                     aux_traduce
                                                                                                          unique
ocupacion                                                                                                       
0.0                                                                                      [Ocupaciones militares]
11.0                                                [Miembros del poder ejecutivo y de los cuerpos legislativos]
12.0                                                [Directores de departamentos administrativos y comerciales ]
13.0                                                                    [Directores de producción y operaciones]
14.0                                 [Directores y gerentes de empresas de alojamiento, restauración y comercio]
...                                                                                             

In [746]:
proporciones('ocupacion_',dropna=True).head(10)

Profesionales de la enseñanza infantil, primaria, secundaria y postsecundaria                              0.155641
Profesionales de la salud                                                                                  0.124594
Profesionales de la ciencias físicas, químicas, matemáticas y de las ingenierías                           0.107317
Especialistas en organización de la Administración Pública y de las empresas y en la comercialización      0.063977
Técnicos de las ciencias y de las ingenierías                                                              0.038799
Profesionales en ciencias sociales                                                                         0.038541
Otros empleados administrativos sin tareas de atención al público                                          0.036991
Profesionales de las tecnologías de la información                                                         0.034997
Empleados en servicios contables, financieros, y de servicios de apoyo a

## 'TR_CNAE' (redefinida como 'cnae')

(Solo para quienes declaran **situación de ocupación** -respuesta 1 en 'TRBPRN1' (o 'sit_lab')-)  

**Actividad principal de la empresa** donde trabaja actualmente.

In [747]:
df['TR_CNAE'].unique() # Missing values as '  ' or '99'

array(['PP', '  ', 'S1', 'QQ', 'N3', 'N6', 'M1', 'AA', 'II', 'OO', 'M2',
       'UU', 'GG', 'N5', 'RR', 'N2', 'CC', 'HH', 'LL', 'M3', 'DD', 'KK',
       'JJ', 'N1', '99', 'S3', 'FF', 'TT', 'S4', 'N4', 'S2', 'EE', 'BB'],
      dtype=object)

In [748]:
leyenda('TR_CNAE')

AA 	 Agricultura, ganadería, silvicultura y pesca
BB 	 Industrias extractivas
CC 	 Industria manufacturera
DD 	 Suministro de energía eléctrica, gas, vapor y aire acondicionado
EE 	 Suministro de agua, actividades de saneamiento, gestión de residuos y descontaminación
FF 	 Construcción e ingeniería civil
GG 	 Comercio al por mayor y al por menor; reparación de vehículos de motor y motocicletas
HH 	 Transporte, almacenamiento, actividades postales y de correos
II 	 Hostelería
JJ 	 Información y comunicaciones (actividades de edición, cinematográficas, de vídeo, de sonido, programas de televisión, telecomunicaciones, programación informática y servicios de información) 
KK 	 Actividades financieras y de seguros
LL 	 Actividades inmobiliarias
M1 	 Actividades profesionales, científicas y técnicas
M2 	 Actividades de fotografía
M3 	 Actividades veterinarias
N1 	 Actividades de alquiler
N2 	 Actividades relacionadas con el empleo
N3 	 Actividades de agencias de viajes, operadores turísticos

In [749]:
traduce('TR_CNAE',check=False)

In [750]:
no_respuesta('TR_CNAE',blank='  ',nsnc='99')

['AA' 'BB' 'CC' 'DD' 'EE' 'FF' 'GG' 'HH' 'II' 'JJ' 'KK' 'LL' 'M1' 'M2'
 'M3' 'N1' 'N2' 'N3' 'N4' 'N5' 'N6' 'OO' 'PP' 'QQ' 'RR' 'S1' 'S2' 'S3'
 'S4' 'TT' 'UU' None]


In [751]:
redefine('TR_CNAE','cnae')

In [752]:
proporciones('cnae_',dropna=True).head(10)

Educación                                                                                                                                                                                          0.197237
Actividades sanitarias y de servicios sociales                                                                                                                                                     0.148438
Actividades profesionales, científicas y técnicas                                                                                                                                                  0.129238
Administración Pública y defensa; Seguridad Social obligatoria                                                                                                                                     0.074721
Comercio al por mayor y al por menor; reparación de vehículos de motor y motocicletas                                                                                                   

## 'TR_TAM' (redefinida como 'tamaño_em')

(Solo para quienes declaran **situación de ocupación** -respuesta 1 en 'TRBPRN1' (o 'sit_lab')-   

**Número de trabajadores** en la empresa en la que trabaja actualmente. 

In [753]:
df['TR_TAM'].unique()

array(['3', '5', '4', ' ', '1', '2', '0', '9'], dtype=object)

In [754]:
traduce('TR_TAM')

                                                      aux_traduce
                                                           unique
TR_TAM                                                           
                                                  [No aplicable ]
0       [Trabajadores independientes o autónomos sin asalariados]
1                                             [De 1 a 9 personas]
2                                        [Entre 10 y 19 personas]
3                                        [Entre 20 y 49 personas]
4                                       [Entre 50 y 249 personas]
5                                            [250 o más personas]
9                                                         [NS/NC]


In [755]:
no_respuesta('TR_TAM')

[ 0.  1.  2.  3.  4.  5. nan]


In [756]:
redefine('TR_TAM','tamaño_em')

In [757]:
proporciones('tamaño_em_',dropna=True)

250 o más personas                                         0.403880
Entre 50 y 249 personas                                    0.191980
De 1 a 9 personas                                          0.139619
Entre 20 y 49 personas                                     0.124011
Entre 10 y 19 personas                                     0.072429
Trabajadores independientes o autónomos sin asalariados    0.068081
Name: tamaño_em_, dtype: float64

## 'TR_ANIO' 

(Solo para quienes declaran **situación de ocupación** -respuesta 1 en 'TRBPRN1' (o 'sit_lab')-   

**Año** en el que empezó a trabajar en su empleo actual.

In [758]:
df['TR_ANIO'].sort_values().unique()

array(['    ', '1970', '1975', '1977', '1978', '1979', '1980', '1981',
       '1982', '1983', '1984', '1985', '1986', '1987', '1988', '1989',
       '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997',
       '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005',
       '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013',
       '2014', '2015', '2016', '2017', '2018', '2019', '9999'],
      dtype=object)

In [759]:
no_respuesta('TR_ANIO',blank='    ',nsnc='9999')

[1970. 1975. 1977. 1978. 1979. 1980. 1981. 1982. 1983. 1984. 1985. 1986.
 1987. 1988. 1989. 1990. 1991. 1992. 1993. 1994. 1995. 1996. 1997. 1998.
 1999. 2000. 2001. 2002. 2003. 2004. 2005. 2006. 2007. 2008. 2009. 2010.
 2011. 2012. 2013. 2014. 2015. 2016. 2017. 2018. 2019.   nan]


## 'TR_SUELDO'   (redefinida como 'sueldo')

(Solo para quienes declaran **situación de ocupación** -respuesta 1 en 'TRBPRN1' (o 'sit_lab')-   

**Sueldo mensual neto** actual.

In [760]:
df['TR_SUELDO'].unique()

array(['3', '5', '4', ' ', '2', '1', '9', '6', '7'], dtype=object)

In [761]:
traduce('TR_SUELDO')

                            aux_traduce
                                 unique
TR_SUELDO                              
                         [No aplicable]
1                  [Menos de 700 euros]
2                  [De 700 a 999 euros]
3              [De 1.000 a 1.499 euros]
4              [De 1.500 a 1.999 euros]
5              [De 2.000 a 2.499 euros]
6              [De 2.500 a 2.999 euros]
7          [De 3.000 euros en adelante]
9                               [NS/NC]


In [762]:
no_respuesta('TR_SUELDO')

[ 1.  2.  3.  4.  5.  6.  7. nan]


In [763]:
redefine('TR_SUELDO','sueldo')

In [764]:
proporciones('sueldo_',dropna=True)

De 1.000 a 1.499 euros        0.318931
De 1.500 a 1.999 euros        0.298920
De 2.000 a 2.499 euros        0.130453
De 700 a 999 euros            0.095660
Menos de 700 euros            0.066553
De 2.500 a 2.999 euros        0.045935
De 3.000 euros en adelante    0.043547
Name: sueldo_, dtype: float64

In [765]:
dist_condicionada('sueldo_','fem_',dropna=True)
# Inspección rápida (ejem)

In [766]:
df['fem'].mean()

0.5704717070550693

In [767]:
dist_condicionada('sueldo_','fem_',rev=True,dropna=True)
# A pesar de que hay menos hombres que mujeres entre los graduados, los hombres están más representados en ...
# ... todos los grupos salariales por encima de 2000 €.

In [768]:
dist_condicionada('sueldo_','disca_',dropna=True)
# La distribución de sueldos es algo más favorable para los graduados con discapacidad reconocida.

## 'TR_D19'     

(Solo para quienes declaran **situación de ocupación** -respuesta 1 en 'TRBPRN1' (o 'sit_lab')-   

**Nivel de formación más apropiado** para realizar el trabajo actual.

In [769]:
df['TR_D19'].unique()

array(['3', ' ', '2', '7', '5', '4', '9', '6', '1'], dtype=object)

In [770]:
traduce('TR_D19')

                                                  aux_traduce
                                                       unique
TR_D19                                                       
                                               [No aplicable]
1                                                 [Doctorado]
2                                                    [Máster]
3         [Título universitario (excepto doctorado o máster)]
4                   [Formación profesional de grado superior]
5       [Formación profesional de grado medio/ bachillerato ]
6                              [Formación profesional básica]
7                                            [ESO, EGB, etc.]
9                                                     [NS/NC]


In [771]:
no_respuesta('TR_D19')

[ 1.  2.  3.  4.  5.  6.  7. nan]


In [772]:
proporciones('TR_D19_',dropna=True)

Título universitario (excepto doctorado o máster)      0.566296
Máster                                                 0.194747
Formación profesional de grado superior                0.101163
Formación profesional de grado medio/ bachillerato     0.052049
ESO, EGB, etc.                                         0.039529
Doctorado                                              0.025783
Formación profesional básica                           0.020433
Name: TR_D19_, dtype: float64

## 'TR_D20'       

(Solo para quienes declaran **situación de ocupación** -respuesta 1 en 'TRBPRN1' (o 'sit_lab')-   

**Área de estudio maś apropiada** para el trabajo actual.

In [773]:
df['TR_D20'].unique()

array(['2', ' ', '4', '3', '1', '9'], dtype=object)

In [774]:
traduce('TR_D20')

                                        aux_traduce
                                             unique
TR_D20                                             
                                    [No aplicable ]
1       [Exclusivamente su propia área de estudios]
2             [Su propia área o alguna relacionada]
3                    [Un área totalmente diferente]
4                       [Ningún área en particular]
9                                           [NS/NC]


In [775]:
no_respuesta('TR_D20')

[ 1.  2.  3.  4. nan]


In [776]:
proporciones('TR_D20_',dropna=True)

Su propia área o alguna relacionada          0.498680
Exclusivamente su propia área de estudios    0.239210
Un área totalmente diferente                 0.152559
Ningún área en particular                    0.109550
Name: TR_D20_, dtype: float64

## 'TR_D21'       

(Solo para quienes declaran **situación de ocupación** -respuesta 1 en 'TRBPRN1' (o 'sit_lab')-   

¿Se hace **uso** en el trabajo actual de los **conocimientos y habilidades** adquiridos en estos estudios?

In [777]:
df['TR_D21'].unique()

array(['1', ' ', '2', '9'], dtype=object)

In [778]:
traduce('TR_D21')

           aux_traduce
                unique
TR_D21                
        [No aplicable]
1                 [Sí]
2                 [No]
9              [NS/NC]


In [779]:
no_respuesta('TR_D21')

[ 1.  2. nan]


In [780]:
proporciones('TR_D21')

1.0    0.642823
2.0    0.206881
NaN    0.150295
Name: TR_D21, dtype: float64

# 'TR_D22_1'  (redefinida como 'vbtr_teoricos')     

(Solo para quienes declaran **situación de ocupación** -respuesta 1 en 'TRBPRN1' (o 'sit_lab')-   

Influencia de los **conocimientos teóricos** para conseguir su trabajo actual. 

In [781]:
df['TR_D22_1'].unique()

array(['1', '5', '2', ' ', '3', '4', '9'], dtype=object)

In [782]:
L_vbtr=list()

In [783]:
def vbtr(col,nuevo):
    """Procesa columnas referidas a influencia de distintos factores en el éxito en la búsqueda de empleo.
    """
    traduce(col)
    no_respuesta(col)
    redefine(col,nuevo)
    L_vbtr.append(str(nuevo)+'_')
    return

In [784]:
vbtr('TR_D22_1','vbtr_teoricos')

             aux_traduce
                  unique
TR_D22_1                
          [No aplicable]
1                 [Nada]
2                 [Poco]
3                 [Algo]
4             [Bastante]
5                [Mucho]
9                [NS/NC]
[ 1.  2.  3.  4.  5. nan]


# 'TR_D22_2'  (redefinida como 'vbtr_practicos')     

(Solo para quienes declaran **situación de ocupación** -respuesta 1 en 'TRBPRN1' (o 'sit_lab')-   

Influencia de las **habilidades prácticas** para conseguir su trabajo actual. 

In [785]:
df['TR_D22_2'].unique()

array(['4', '5', ' ', '2', '1', '3', '9'], dtype=object)

In [786]:
vbtr('TR_D22_2','vbtr_practicos')

             aux_traduce
                  unique
TR_D22_2                
          [No aplicable]
1                 [Nada]
2                 [Poco]
3                 [Algo]
4             [Bastante]
5                [Mucho]
9                [NS/NC]
[ 1.  2.  3.  4.  5. nan]


# 'TR_D22_3'  (redefinida como 'vbtr_idiomas')     

(Solo para quienes declaran **situación de ocupación** -respuesta 1 en 'TRBPRN1' (o 'sit_lab')-   

Influencia de **conocer idiomas** para conseguir su trabajo actual. 

In [787]:
df['TR_D22_3'].unique()

array(['5', '3', ' ', '4', '1', '2', '9'], dtype=object)

In [788]:
vbtr('TR_D22_3','vbtr_idiomas')

             aux_traduce
                  unique
TR_D22_3                
          [No aplicable]
1                 [Nada]
2                 [Poco]
3                 [Algo]
4             [Bastante]
5                [Mucho]
9                [NS/NC]
[ 1.  2.  3.  4.  5. nan]


# 'TR_D22_4'  (redefinida como 'vbtr_ti')     

(Solo para quienes declaran **situación de ocupación** -respuesta 1 en 'TRBPRN1' (o 'sit_lab')-   

Influencia de la formación o el dominio de la **informática** y de las **tecnologías de la información y la comunicación** para conseguir su trabajo actual. 

In [789]:
df['TR_D22_4'].unique()

array(['3', '4', '5', ' ', '2', '1', '9'], dtype=object)

In [790]:
vbtr('TR_D22_4','vbtr_ti')

             aux_traduce
                  unique
TR_D22_4                
          [No aplicable]
1                 [Nada]
2                 [Poco]
3                 [Algo]
4             [Bastante]
5                [Mucho]
9                [NS/NC]
[ 1.  2.  3.  4.  5. nan]


# 'TR_D22_4'  (redefinida como 'vbtr_personales')     

(Solo para quienes declaran **situación de ocupación** -respuesta 1 en 'TRBPRN1' (o 'sit_lab')-   

Influencia de las **competencias personales y sociales** -personalidad, habilidades sociales, comunicación, capacidad de trabajar en grupo- para conseguir su trabajo actual. 

In [791]:
df['TR_D22_4'].unique()

array([ 3.,  4.,  5., nan,  2.,  1.])

In [792]:
vbtr('TR_D22_4','vbtr_personales')

         aux_traduce
              unique
TR_D22_4            
1.0           [Nada]
2.0           [Poco]
3.0           [Algo]
4.0       [Bastante]
5.0          [Mucho]
[ 1.  2.  3.  4.  5. nan]


# 'TR_D22_5'  (redefinida como 'vbtr_gestion')     

(Solo para quienes declaran **situación de ocupación** -respuesta 1 en 'TRBPRN1' (o 'sit_lab')-   

Influencia de la **capacidad de gestión, planificación y emprendimiento** para conseguir su trabajo actual. 

In [793]:
df['TR_D22_5'].unique()

array(['4', '5', '3', ' ', '1', '9', '2'], dtype=object)

In [794]:
vbtr('TR_D22_5','vbtr_gestion')

             aux_traduce
                  unique
TR_D22_5                
          [No aplicable]
1                 [Nada]
2                 [Poco]
3                 [Algo]
4             [Bastante]
5                [Mucho]
9                [NS/NC]
[ 1.  2.  3.  4.  5. nan]


In [795]:
df[L_vbtr].apply(lambda x:x.value_counts(normalize=True)).sort_values('vbtr_teoricos_')

,vbtr_teoricos_,vbtr_practicos_,vbtr_idiomas_,vbtr_ti_,vbtr_personales_,vbtr_gestion_
Nada,0.115563,0.088698,0.232414,0.126360,0.126360,0.049300
Poco,0.122228,0.080960,0.182464,0.139754,0.139754,0.041433
Algo,0.175875,0.124877,0.176643,0.223541,0.223541,0.099217
Bastante,0.282635,0.287125,0.174084,0.291955,0.291955,0.312068
Mucho,0.303699,0.418340,0.234396,0.218389,0.218389,0.497982


In [796]:
df[L_vbtr].apply(lambda x:x.value_counts(normalize=True)).rank(ascending=False).sort_values('vbtr_teoricos_')

,vbtr_teoricos_,vbtr_practicos_,vbtr_idiomas_,vbtr_ti_,vbtr_personales_,vbtr_gestion_
Mucho,1.0,1.0,1.0,3.0,3.0,1.0
Bastante,2.0,2.0,5.0,1.0,1.0,2.0
Algo,3.0,3.0,4.0,2.0,2.0,3.0
Poco,4.0,5.0,3.0,4.0,4.0,5.0
Nada,5.0,4.0,2.0,5.0,5.0,4.0


# 'TR_D23'  (completa la variable 'busqueda_tr')

(Solo para quienes declaran **situación de ocupación** -respuesta 1 en 'TRBPRN1' (o 'sit_lab')-   

¿Ha **intentado conseguir otro trabajo remunerado** en las últimas 4 semanas?

Nota: Se usa esta información para actualizar la variable busqueda_tr'.

In [797]:
df['TR_D23'].unique()

array(['3', ' ', '1', '2', '9'], dtype=object)

In [798]:
traduce('TR_D23')

                                                                                                     aux_traduce
                                                                                                          unique
TR_D23                                                                                                          
                                                                                                  [No aplicable]
1                                                                                                           [Sí]
2       [No, pero está esperando respuesta a solicitudes anteriores de empleo o los resultados de una oposición]
3                                                                                                           [No]
9                                                                                                        [NS/NC]


In [799]:
no_respuesta('TR_D23')

[ 1.  2.  3. nan]


In [800]:
# Se actualiza la variable 'busqueda_tr':
df['busqueda_tr'].mask((df['TR_D23']==1)|(df['TR_D23']==2),1,inplace=True)
traduce('busqueda_tr')
df['busqueda_tr_'][(df['TR_D23']==1)|(df['TR_D23']==2)].unique() # Comprobación

            aux_traduce
                 unique
busqueda_tr            
0.0                [No]
1.0                [Sí]


array(['Sí'], dtype=object)

# BLOQUE E. HISTORIAL LABORAL DEL GRADUADO UNIVERSITARIO

# 'HL_E1'  

¿Realizó **prácticas en empresas**, instituciones o similares?

In [801]:
df['HL_E1' ].unique()

array([3, 1, 4, 2])

In [802]:
traduce('HL_E1' )

                                 aux_traduce
                                      unique
HL_E1                                       
1      [Sí, como parte del plan de estudios]
2           [Sí, fuera del plan de estudios]
3             [Sí, ambos tipos de prácticas]
4                                       [No]


# 'HL_E2'  

(Solo para respuestas 1, 2 o 3 en variable 'HL_E1')

Duración en meses de las **prácticas en empresas**, instituciones o similares.

In [803]:
df['HL_E2'].unique()

array(['3', '2', '1', ' ', '5', '4', '9'], dtype=object)

In [804]:
traduce('HL_E2')

              aux_traduce
                   unique
HL_E2                    
           [No aplicable]
1        [De 1 a 3 meses]
2        [De 4 a 6 meses]
3        [De 7 a 9 meses]
4      [De 10 a 12 meses]
5         [Más de un año]
9                 [NS/NC]


In [805]:
no_respuesta('HL_E2')

[ 1.  2.  3.  4.  5. nan]


# 'HL_E3'  

¿Ha tenido algún **trabajo remunerado mientras realizaba los estudios** (incluido el trabajo por cuenta propia, trabajos de formación, contrato en prácticas, puestos de becario, etc.)?

In [806]:
df['HL_E3'].unique()

array([2, 1])

In [807]:
traduce('HL_E3')

      aux_traduce
           unique
HL_E3            
1            [Sí]
2            [No]


# 'HL_E5'  

¿**Continuó en ese trabajo** durante al menos 6 meses después de acabar estos estudios?

In [808]:
df['HL_E5'].unique()

array([' ', '2', '1'], dtype=object)

In [809]:
traduce('HL_E5')

          aux_traduce
               unique
HL_E5                
       [No aplicable]
1                [Sí]
2                [No]


In [810]:
no_respuesta('HL_E5')

[ 1.  2. nan]


# 'HL_JOR'  

**Tipo de jornada** en ese trabajo.

In [811]:
df['HL_JOR'].unique()

array([' ', '1', '2', '9'], dtype=object)

In [812]:
traduce('HL_JOR')

                aux_traduce
                     unique
HL_JOR                     
                      [nan]
1        [A tiempo parcial]
2       [A tiempo completo]
9                   [NS/NC]


In [813]:
no_respuesta('HL_JOR')

[ 1.  2. nan]


# 'HL_E7'  

Una vez completados estos estudios, ¿**mejoraron las condiciones del trabajo** en el que continuó después de acabarlos?

In [814]:
df['HL_E7'].unique()

array([' ', '1', '2', '9'], dtype=object)

In [815]:
traduce('HL_E7')

          aux_traduce
               unique
HL_E7                
       [No aplicable]
1                [Sí]
2                [No]
9             [NS/NC]


In [816]:
no_respuesta('HL_E7')

[ 1.  2. nan]


# 'HL_E8'  

Mientras tenía este empleo, ¿**buscó un trabajo mejor**?

In [817]:
df['HL_E8'].unique()

array([' ', '2', '1', '9'], dtype=object)

In [818]:
traduce('HL_E8')

          aux_traduce
               unique
HL_E8                
       [No aplicable]
1                [Sí]
2                [No]
9             [NS/NC]


In [819]:
no_respuesta('HL_E8')

[ 1.  2. nan]


# 'HA_TRAB'  (redefinida como 'ha_trab')

Ha tenido **algún trabajo remunerado desde que terminó sus estudios** (distinto del que haya realizado mientras estudiaba), incluyendo trabajo por cuenta propia, trabajos de formación, en prácticas, puestos de becario, etc.

Nota: La variable 'HA_TRAB' se ha empleado como filtro para algunas de las variables definidas a continuación, formuladas solo a quienes responden Sí o a quienes responden No a esta pregunta. 

In [820]:
df['HA_TRAB'].unique()

array([1, 2])

In [821]:
traduce('HA_TRAB')

        aux_traduce
             unique
HA_TRAB            
1              [Sí]
2              [No]


In [822]:
df[['HA_TRAB_','sit_lab_']].groupby(['sit_lab_']).agg(['unique']) # Inspección

,HA_TRAB_
,unique
sit_lab_,
En desempleo,"[Sí, No]"
Inactivo,"[Sí, No]"
Trabajando,[Sí]


In [823]:
redefine('HA_TRAB',si_no=True)

        aux_traduce
             unique
ha_trab            
0              [No]
1              [Sí]


# 'HATR_E11'  

(Solo para quienes **declaran haber trabajado** después de haber terminado sus estudios -respuesta 1 en 'HA_TRAB' o 'ha_trab'-)

**Tiempo** transcurrido desde que finalizó estos estudios **hasta que empezó a trabajar**.

In [824]:
df['HATR_E11'].unique()

array(['3', '1', '0', '5', '4', '2', '6', ' ', '9'], dtype=object)

In [825]:
traduce('HATR_E11')

                                                                         aux_traduce
                                                                              unique
HATR_E11                                                                            
                                                                      [No aplicable]
0         [Continuó al menos 6 meses más en el trabajo que tenía mientras estudiaba]
1                                                              [Menos de tres meses]
2                                                                   [De 3 a 6 meses]
3                                                               [De 6 meses a 1 año]
4                                                        [De 1 año a  año y medio  ]
5                                                      [De 1 año y medio a  2 años ]
6                                                                     [2 ó más años]
9                                                                

In [826]:
df[['HATR_E11','ha_trab_']].groupby('ha_trab_').agg(['unique']) # Inspección

,HATR_E11
,unique
ha_trab_,
No,[ ]
Sí,"[3, 1, 0, 5, 4, 2, 6, 9]"


In [827]:
no_respuesta('HATR_E11')

[ 0.  1.  2.  3.  4.  5.  6. nan]


## 'HATR_E12'  

(Solo para quienes **declaran haber trabajado** después de haber terminado sus estudios -respuesta 1 en 'HA_TRAB' o 'ha_trab'-)

Momento en que empezó a buscar trabajo.

In [828]:
df['HATR_E12'].unique()

array(['2', '1', '0', ' ', '9'], dtype=object)

In [829]:
traduce('HATR_E12')

                                                                         aux_traduce
                                                                              unique
HATR_E12                                                                            
                                                                      [No aplicable]
0         [Continuó al menos 6 meses más en el trabajo que tenía mientras estudiaba]
1             [Antes de terminar los estudios de la titulación que finalizó en 2014]
2           [Después de terminar los estudios de la titulación que finalizó en 2014]
9                                                                            [NS/NC]


In [830]:
df[['HA_TRAB_','HATR_E12_','sit_lab_']].groupby('HA_TRAB_').agg(['unique']) # Inspección
# Solo se ha preguntado a quien ha trabajado después de terminar sus estudios, con independencia de su ...
# ... situación laboral actual.

,HATR_E12_,sit_lab_
,unique,unique
HA_TRAB_,,
No,[No aplicable],"[Inactivo, En desempleo]"
Sí,"[Después de terminar los estudios de la titulación que finalizó en 2014, Antes de terminar los estudios de la titulación que finalizó en 2014, Continuó al menos 6 meses más en el trabajo que tenía mientras estudiaba, NS/NC]","[Trabajando, Inactivo, En desempleo]"


In [831]:
no_respuesta('HATR_E12')

[ 0.  1.  2. nan]


## 'HATR_BSQ1'  (redefinida como 'etr_internet', actualiza 'btr_internet')

(Solo para quienes **declaran haber trabajado** después de haber terminado sus estudios -respuesta 1 en 'HA_TRAB' o 'ha_trab'-)

¿**Encontró trabajo** a través de ofertas de trabajo en el **periódico, internet**, etc.?  

Nota: Esta es la primera de una serie de preguntas sobre estrategias de búsqueda de empleo entre quienes han tenido alguno desde que terminaron sus estudios. La variable 'HATR_BSQ1' es distinta de 'TRPARA_D7_1' (o 'btr_internet'); 'btr_internet' se refiere a la búsqueda de trabajo a través de anuncios en prensa o internet y se formula solo a desempleados, mientras que 'TRPARA_D7_1' se refiere al **éxito de esa estrategia para encontrar trabajo** y se formula a todos los que han trabajado alguna vez desde que terminaron sus estudios.   
No obstante, esta y otras variables de esta serie permiten actualizar algunas de las variables con prefijo 'btr_' definidas originalmente solo para desempleados. 

In [832]:
df['HATR_BSQ1'].unique()

array(['1', '2', ' ', '9'], dtype=object)

In [833]:
traduce('HATR_BSQ1')

              aux_traduce
                   unique
HATR_BSQ1                
           [No aplicable]
1                    [Sí]
2                    [No]
9                 [NS/NC]


In [834]:
df[['HATR_BSQ1_','btr_internet_','HA_TRAB_','sit_lab_']].groupby(['sit_lab_','HA_TRAB_']).agg(['unique'])
# Solo se ha preguntado a quien ha trabajado desde que terminó sus estudios (a diferencia de 'btr_internet', ...
# ... que se preguntó a los desempleados)

HATR_BSQ1_  btr_internet_
                                unique         unique
sit_lab_     HA_TRAB_                                
En desempleo No         [No aplicable]  [Sí, No, nan]
             Sí        [No, Sí, NS/NC]  [Sí, No, nan]
Inactivo     No         [No aplicable]          [nan]
             Sí        [Sí, No, NS/NC]          [nan]
Trabajando   Sí        [Sí, No, NS/NC]          [nan]

In [835]:
leyenda('sit_lab')

1 	 Trabajando
2 	 En desempleo
3 	 Inactivo


In [836]:
df['HATR_BSQ1_'][df['sit_lab']==2].equals(df['btr_internet_'][df['sit_lab']==2])
# 'btr_internet' y 'HATR_BSQ1_' son variables distintas (responden a preguntas distintas)

False

In [837]:
df[['HATR_BSQ1_','btr_internet_']][df['sit_lab']==2].sample(5)

,HATR_BSQ1_,btr_internet_
2187,No,No
9435,No,Sí
29144,No,Sí
1532,Sí,Sí
5795,No,Sí


In [838]:
# Lista de estrategias de búsqueda de empleo de quienes lo encontraron después de terminar sus estudios.
L_etr=list()

In [839]:
def etr(col,nuevo):
    """Procesa las columnas referidas a las estrategias de búsqueda de empleo de quienes tuvieron alguno después
    de terminar sus estudios.
    """
    traduce(col)
    no_respuesta(col)
    redefine(col,nuevo,si_no=True)
    D_describe[nuevo]='Encontró empleo:'+ D_describe[col].split(':')[1]
    if str(nuevo)+'_' not in L_etr:
        L_etr.append(str(nuevo)+'_')
    return

In [840]:
etr('HATR_BSQ1','etr_internet')

              aux_traduce
                   unique
HATR_BSQ1                
           [No aplicable]
1                    [Sí]
2                    [No]
9                 [NS/NC]
[ 1.  2. nan]
             aux_traduce
                  unique
etr_internet            
0.0                 [No]
1.0                 [Sí]


In [841]:
def actualiza_btr(col,btr):
    """ Actualiza las variables con prefijo btr_ definidas para las distintas estrategias de búsqueda de empleo, 
    añadiendo las respuestas declaradas por quienes han trabajado después de terminar sus estudios: registra la 
    estrategia definida en la variable 'col' como utilizada por el encuestado cuando este declara que le sirvió 
    para encontrar empleo. No se infiere que no la haya usado cuando el encuestado no le atribuye éxito. 
    """
    df[btr].mask(df[col]==1,1,inplace=True)
    traduce(btr)
    print (df[[str(col)+'_',str(btr)+'_']][df['HA_TRAB']==1].groupby([str(col)+'_'],
                                                                        dropna=False).agg(['unique']))
    return

In [842]:
actualiza_btr('HATR_BSQ1','btr_internet')

             aux_traduce
                  unique
btr_internet            
0.0                 [No]
1.0                 [Sí]
            btr_internet_
                   unique
HATR_BSQ1_               
No          [nan, Sí, No]
Sí                   [Sí]
NaN         [nan, No, Sí]


## 'HATR_BSQ2'  (redefinida como 'etr_spe', actualiza 'btr_spe')  

(Solo para quienes **declaran haber trabajado** después de haber terminado sus estudios -respuesta 1 en 'HA_TRAB' o 'ha_trab'-)  

¿**Encontró trabajo** a través de los **servicios públicos de empleo**?

In [843]:
df['HATR_BSQ2'].unique()

array(['1', '2', ' ', '9'], dtype=object)

In [844]:
etr('HATR_BSQ2','etr_spe')

              aux_traduce
                   unique
HATR_BSQ2                
           [No aplicable]
1                    [Sí]
2                    [No]
9                 [NS/NC]
[ 1.  2. nan]
        aux_traduce
             unique
etr_spe            
0.0            [No]
1.0            [Sí]


In [845]:
actualiza_btr('HATR_BSQ2','btr_spe')

        aux_traduce
             unique
btr_spe            
0.0            [No]
1.0            [Sí]
                 btr_spe_
                   unique
HATR_BSQ2_               
No          [nan, No, Sí]
Sí                   [Sí]
NaN         [nan, Sí, No]


## 'HATR_BSQ3'  (redefinida como 'etr_univ', actualiza 'btr_univ')    

(Solo para quienes **declaran haber trabajado** después de haber terminado sus estudios -respuesta 1 en 'HA_TRAB' o 'ha_trab'-)  

¿**Encontró trabajo** a través de los **servicios de empleo de la universidad** (bolsa de trabajo, asociaciones de exalumnos, etc.)?

In [846]:
df['HATR_BSQ3'].unique()

array(['1', '2', ' ', '9'], dtype=object)

In [847]:
etr('HATR_BSQ3','etr_univ')

              aux_traduce
                   unique
HATR_BSQ3                
           [No aplicable]
1                    [Sí]
2                    [No]
9                 [NS/NC]
[ 1.  2. nan]
         aux_traduce
              unique
etr_univ            
0.0             [No]
1.0             [Sí]


In [848]:
actualiza_btr('HATR_BSQ3','btr_univ')

         aux_traduce
              unique
btr_univ            
0.0             [No]
1.0             [Sí]
                btr_univ_
                   unique
HATR_BSQ3_               
No          [nan, No, Sí]
Sí                   [Sí]
NaN         [nan, No, Sí]


## 'HATR_BSQ4'  (redefinida como 'etr_ett', actualiza 'btr_ett')  

(Solo para quienes **declaran haber trabajado** después de haber terminado sus estudios -respuesta 1 en 'HA_TRAB' o 'ha_trab'-)  

¿**Encontró trabajo** a través de **empresas de trabajo temporal**?

In [849]:
df['HATR_BSQ4'].unique()

array(['2', '1', ' ', '9'], dtype=object)

In [850]:
etr('HATR_BSQ4','etr_ett')

              aux_traduce
                   unique
HATR_BSQ4                
           [No aplicable]
1                    [Sí]
2                    [No]
9                 [NS/NC]
[ 1.  2. nan]
        aux_traduce
             unique
etr_ett            
0.0            [No]
1.0            [Sí]


In [851]:
actualiza_btr('HATR_BSQ4','btr_ett')

        aux_traduce
             unique
btr_ett            
0.0            [No]
1.0            [Sí]
                 btr_ett_
                   unique
HATR_BSQ4_               
No          [nan, No, Sí]
Sí                   [Sí]
NaN         [nan, No, Sí]


## 'HATR_BSQ5'  (redefinida como 'etr_bolsa', actualiza 'btr_bolsa')  

(Solo para quienes **declaran haber trabajado** después de haber terminado sus estudios -respuesta 1 en 'HA_TRAB' o 'ha_trab'-)  

¿**Encontró trabajo** a través de una **bolsa de empleo**?

In [852]:
df['HATR_BSQ5'].unique()

array(['2', '1', ' ', '9'], dtype=object)

In [853]:
etr('HATR_BSQ5','etr_bolsa')

              aux_traduce
                   unique
HATR_BSQ5                
           [No aplicable]
1                    [Sí]
2                    [No]
9                 [NS/NC]
[ 1.  2. nan]
          aux_traduce
               unique
etr_bolsa            
0.0              [No]
1.0              [Sí]


In [854]:
actualiza_btr('HATR_BSQ5','btr_bolsa')

          aux_traduce
               unique
btr_bolsa            
0.0              [No]
1.0              [Sí]
               btr_bolsa_
                   unique
HATR_BSQ5_               
No          [nan, No, Sí]
Sí                   [Sí]
NaN         [nan, No, Sí]


## 'HATR_BSQ6'  (redefinida como 'etr_contactos', actualiza 'btr_contactos')  

(Solo para quienes **declaran haber trabajado** después de haber terminado sus estudios -respuesta 1 en 'HA_TRAB' o 'ha_trab'-)  

¿**Encontró trabajo** contactando con el empleador por iniciativa propia o utilizando **contactos personales** (familia, amigos)

In [855]:
df['HATR_BSQ6'].unique()

array(['2', '1', ' ', '9'], dtype=object)

In [856]:
etr('HATR_BSQ6','etr_contactos')

              aux_traduce
                   unique
HATR_BSQ6                
           [No aplicable]
1                    [Sí]
2                    [No]
9                 [NS/NC]
[ 1.  2. nan]
              aux_traduce
                   unique
etr_contactos            
0.0                  [No]
1.0                  [Sí]


In [857]:
actualiza_btr('HATR_BSQ6','btr_contactos')

              aux_traduce
                   unique
btr_contactos            
0.0                  [No]
1.0                  [Sí]
           btr_contactos_
                   unique
HATR_BSQ6_               
No          [nan, No, Sí]
Sí                   [Sí]
NaN         [nan, No, Sí]


## 'HATR_BSQ7'  (redefinida como 'etr_empleador', actualiza 'btr_otro')  

(Solo para quienes **declaran haber trabajado** después de haber terminado sus estudios -respuesta 1 en 'HA_TRAB' o 'ha_trab'-)  

¿**Encontró trabajo** gracias a que el **empleador se puso en contacto** con usted?

In [858]:
df['HATR_BSQ7'].unique()

array(['2', '1', ' ', '9'], dtype=object)

In [859]:
etr('HATR_BSQ7','etr_empleador')

              aux_traduce
                   unique
HATR_BSQ7                
           [No aplicable]
1                    [Sí]
2                    [No]
9                 [NS/NC]
[ 1.  2. nan]
              aux_traduce
                   unique
etr_empleador            
0.0                  [No]
1.0                  [Sí]


In [860]:
actualiza_btr('HATR_BSQ7','btr_otro')

         aux_traduce
              unique
btr_otro            
0.0             [No]
1.0             [Sí]
                btr_otro_
                   unique
HATR_BSQ7_               
No          [nan, No, Sí]
Sí                   [Sí]
NaN             [nan, No]


## 'HATR_BSQ8'  (redefinida como 'etr_practicas', actualiza 'btr_otro')  

(Solo para quienes **declaran haber trabajado** después de haber terminado sus estudios -respuesta 1 en 'HA_TRAB' o 'ha_trab'-)  

¿Continuó con las **prácticas en empresas** o instituciones que realizó durante la carrera?

In [861]:
df['HATR_BSQ8'].unique()

array(['2', '1', ' ', '9'], dtype=object)

In [862]:
etr('HATR_BSQ8','etr_practicas')

              aux_traduce
                   unique
HATR_BSQ8                
           [No aplicable]
1                    [Sí]
2                    [No]
9                 [NS/NC]
[ 1.  2. nan]
              aux_traduce
                   unique
etr_practicas            
0.0                  [No]
1.0                  [Sí]


In [863]:
actualiza_btr('HATR_BSQ8','btr_otro')

         aux_traduce
              unique
btr_otro            
0.0             [No]
1.0             [Sí]
                btr_otro_
                   unique
HATR_BSQ8_               
No          [nan, Sí, No]
Sí                   [Sí]
NaN             [nan, No]


## 'HATR_BSQ9'  (redefinida como 'etr_opo', actualiza 'btr_opo')  

(Solo para quienes **declaran haber trabajado** después de haber terminado sus estudios -respuesta 1 en 'HA_TRAB' o 'ha_trab'-)  

¿**Encontró trabajo** preparando unas **oposiciones**?

In [864]:
df['HATR_BSQ9'].unique()

array(['2', '1', ' ', '9'], dtype=object)

In [865]:
etr('HATR_BSQ9','etr_opo')

              aux_traduce
                   unique
HATR_BSQ9                
           [No aplicable]
1                    [Sí]
2                    [No]
9                 [NS/NC]
[ 1.  2. nan]
        aux_traduce
             unique
etr_opo            
0.0            [No]
1.0            [Sí]


In [866]:
actualiza_btr('HATR_BSQ9','btr_opo')

        aux_traduce
             unique
btr_opo            
0.0            [No]
1.0            [Sí]
                 btr_opo_
                   unique
HATR_BSQ9_               
No          [nan, Sí, No]
Sí                   [Sí]
NaN         [nan, No, Sí]


## 'HATR_BSQ10'  (redefinida como 'etr_auto', actualiza 'btr_auto)  

(Solo para quienes **declaran haber trabajado** después de haber terminado sus estudios -respuesta 1 en 'HA_TRAB' o 'ha_trab'-)  

¿**Encontró trabajo** montando un **negocio propio**?

In [867]:
df['HATR_BSQ10'].unique()

array(['2', ' ', '1', '9'], dtype=object)

In [868]:
etr('HATR_BSQ10','etr_auto')

               aux_traduce
                    unique
HATR_BSQ10                
            [No aplicable]
1                     [Sí]
2                     [No]
9                  [NS/NC]
[ 1.  2. nan]
         aux_traduce
              unique
etr_auto            
0.0             [No]
1.0             [Sí]


In [869]:
actualiza_btr('HATR_BSQ10','btr_auto')

         aux_traduce
              unique
btr_auto            
0.0             [No]
1.0             [Sí]
                 btr_auto_
                    unique
HATR_BSQ10_               
No           [nan, No, Sí]
Sí                    [Sí]
NaN          [nan, No, Sí]


## 'HATR_BSQ11'  (redefinida como 'etr_otro', actualiza 'btr_otro')  

(Solo para quienes **declaran haber trabajado** después de haber terminado sus estudios -respuesta 1 en 'HA_TRAB' o 'ha_trab'-) 

¿**Encontró trabajo de otra forma**?

In [870]:
df['HATR_BSQ11'].unique()

array(['2', '1', ' ', '9'], dtype=object)

In [871]:
etr('HATR_BSQ11','etr_otro')

               aux_traduce
                    unique
HATR_BSQ11                
            [No aplicable]
1                     [Sí]
2                     [No]
9                  [NS/NC]
[ 1.  2. nan]
         aux_traduce
              unique
etr_otro            
0.0             [No]
1.0             [Sí]


In [872]:
actualiza_btr('HATR_BSQ11','btr_otro')

         aux_traduce
              unique
btr_otro            
0.0             [No]
1.0             [Sí]
                 btr_otro_
                    unique
HATR_BSQ11_               
No           [nan, Sí, No]
Sí                    [Sí]
NaN              [nan, No]


In [873]:
for col in L_etr:
    col=str(col).strip('_')
    print (col,'\t',D_describe[col])

etr_internet 	 Encontró empleo: a través de ofertas de trabajo en el periódico, internet,...
etr_spe 	 Encontró empleo: a través de servicios de empleo públicos
etr_univ 	 Encontró empleo: a través de los servicios de empleo de la universidad (bolsa de trabajo, asociaciones de exalumnos, …)
etr_ett 	 Encontró empleo: a través de empresas de trabajo temporal
etr_bolsa 	 Encontró empleo: a través de una bolsa de empleo
etr_contactos 	 Encontró empleo: contactó con el empleador por iniciativa propia o utilizó contactos personales (familia, amigos)
etr_empleador 	 Encontró empleo: el empleador se puso en contacto con él
etr_practicas 	 Encontró empleo: continuó con las prácticas en empresas/ instituciones que realizó durante la carrera
etr_opo 	 Encontró empleo: preparó unas oposiciones
etr_auto 	 Encontró empleo: montó su propio negocio
etr_otro 	 Encontró empleo: de otra forma


In [874]:
# Promedios del éxito atribuído a cada estrategia de búsqueda de empleo por quienes han trabajado después de ...
# ... terminar sus estudios: 
df[L_etr].apply(lambda x: x.value_counts(normalize=True))

,etr_internet_,etr_spe_,etr_univ_,etr_ett_,etr_bolsa_,etr_contactos_,etr_empleador_,etr_practicas_,etr_opo_,etr_auto_,etr_otro_
No,0.698214,0.899437,0.898949,0.947832,0.904901,0.634241,0.80616,0.914853,0.851107,0.951475,0.974632
Sí,0.301786,0.100563,0.101051,0.052168,0.095099,0.365759,0.19384,0.085147,0.148893,0.048525,0.025368


## 'P_CONTI'  

(Solo para quienes **declaran haber trabajado** después de haber terminado sus estudios -respuesta 1 en 'HA_TRAB' o 'ha_trab'-) 

¿**Continua en su primer empleo**?

In [875]:
df['P_CONTI'].unique()

array(['2', '1', ' '], dtype=object)

In [876]:
traduce('P_CONTI')

            aux_traduce
                 unique
P_CONTI                
         [No aplicable]
1                  [Sí]
2                  [No]


In [877]:
df[['P_CONTI','ha_trab_']].groupby('ha_trab_').agg(['unique']) # Inspección

,P_CONTI
,unique
ha_trab_,
No,[ ]
Sí,"[2, 1]"


In [878]:
no_respuesta('P_CONTI')

[ 1.  2. nan]


In [879]:
proporciones('P_CONTI_')

No     0.682253
Sí     0.296294
NaN    0.021453
Name: P_CONTI_, dtype: float64

## 'PR_PAIS'  

(Solo para quienes **declaran haber trabajado** después de haber terminado sus estudios -respuesta 1 en 'HA_TRAB' o 'ha_trab'-) 

**País** en el que desempeñaba su **primer empleo**.

In [880]:
df['PR_PAIS'].unique()

array(['01', '  ', '02', '05', '03', '06', '07', '08', '04', '99', '09',
       '10', '11'], dtype=object)

In [881]:
traduce('PR_PAIS')

                                            aux_traduce
                                                 unique
PR_PAIS                                                
                                         [No aplicable]
01                                             [España]
02                                        [Reino Unido]
03                                           [Alemania]
04                                            [Francia]
05                      [Otro país de la Unión Europea]
06                 [Otro país europeo (fuera de la UE)]
07                       [Un país de América del Norte]
08       [Un país de América Central o América del Sur]
09                                    [Un país de Asia]
10                                  [Un país de África]
11                                 [Un país de Oceanía]
99                                              [NS/NC]


In [882]:
df[['ha_trab_','PR_PAIS']].groupby(['ha_trab_']).agg(['unique']) # Inspección

,PR_PAIS
,unique
ha_trab_,
No,[ ]
Sí,"[01, 02, 05, 03, 06, 07, 08, 04, 99, 09, 10, 11]"


In [883]:
no_respuesta('PR_PAIS',blank='  ',nsnc='99')

[ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. nan]


## 'PR_OCU'  

(Solo para quienes **declaran haber trabajado** después de haber terminado sus estudios -respuesta 1 en 'HA_TRAB' o 'ha_trab'-) 

**Ocupación** que desempeñaba en su **primer empleo**.

In [884]:
df['PR_OCU'].unique()

array(['23', '22', '37', '51', '52', '45', '57', '55', '26', '33', '  ',
       '42', '28', '58', '56', '59', '14', '91', '13', '31', '36', '21',
       '44', '43', '92', '35', '54', '96', '41', '12', '24', '98', '15',
       '97', '84', '93', '75', '95', '11', '29', '72', '27', '94', '73',
       '78', '71', '74', '77', '00', '53', '81', '83', '99', '38', '61',
       '82', '25', '32', '34', '76', '64', '50', '62', '63'], dtype=object)

In [885]:
D_dict['PR_OCU']=D_dict['ocupacion'] # Diccionario abreviado de ocupaciones
traduce('PR_OCU')

                                                         aux_traduce
                                                              unique
PR_OCU                                                              
                                                      [No aplicable]
00                                           [Ocupaciones militares]
11      [Miembros del poder ejecutivo y de los cuerpos legislativos]
12      [Directores de departamentos administrativos y comerciales ]
13                          [Directores de producción y operaciones]
...                                                              ...
95                       [Peones agrarios, forestales y de la pesca]
96                       [Peones de la construcción y de la minería]
97                         [Peones de las industrias manufactureras]
98              [Peones del transporte, descargadores y reponedores]
99                                                           [NS/NC]

[64 rows x 1 columns]


In [886]:
df[['PR_OCU','ha_trab_']].groupby('ha_trab_').agg(['unique']) # Inspección

,PR_OCU
,unique
ha_trab_,
No,[ ]
Sí,"[23, 22, 37, 51, 52, 45, 57, 55, 26, 33, 42, 28, 58, 56, 59, 14, 91, 13, 31, 36, 21, 44, 43, 92, 35, 54, 96, 41, 12, 24, 98, 15, 97, 84, 93, 75, 95, 11, 29, 72, 27, 94, 73, 78, 71, 74, 77, 00, 53, 81, 83, 99, 38, 61, 82, 25, 32, 34, 76, 64, 50, 62, 63]"


In [887]:
no_respuesta('PR_OCU',blank='  ',nsnc='99')

[ 0. 11. 12. 13. 14. 15. 21. 22. 23. 24. 25. 26. 27. 28. 29. 31. 32. 33.
 34. 35. 36. 37. 38. 41. 42. 43. 44. 45. 50. 51. 52. 53. 54. 55. 56. 57.
 58. 59. 61. 62. 63. 64. 71. 72. 73. 74. 75. 76. 77. 78. 81. 82. 83. 84.
 91. 92. 93. 94. 95. 96. 97. 98. nan]


## 'PR_SIT_PR'  

(Solo para quienes **declaran haber trabajado** después de haber terminado sus estudios -respuesta 1 en 'HA_TRAB' o 'ha_trab'-) 

**Situación profesional** en su **primer empleo**.

In [888]:
df['PR_SIT_PR'].unique()

array(['1', '3', '2', '5', ' ', '6', '4'], dtype=object)

In [889]:
traduce('PR_SIT_PR')

                                                                                aux_traduce
                                                                                     unique
PR_SIT_PR                                                                                  
                                                                             [No aplicable]
1                 [Trabajador en prácticas, formación (incluido MIR, EIR, FIR,…) o becario]
2          [Asalariado con trabajo permanente o contrato de trabajo de duración indefinida]
3           [Asalariado con trabajo temporal o contrato de trabajo de duración determinada]
4                                                              [Empresario con asalariados]
5                                   [Trabajador independiente o empresario sin asalariados]
6                                                  [Ayuda en la empresa o negocio familiar]


In [890]:
df[['PR_SIT_PR_','ha_trab_']].groupby('ha_trab_').agg(['unique']) # Inspección

,PR_SIT_PR_
,unique
ha_trab_,
No,[No aplicable]
Sí,"[Trabajador en prácticas, formación (incluido MIR, EIR, FIR,…) o becario, Asalariado con trabajo temporal o contrato de trabajo de duración determinada, Asalariado con trabajo permanente o contrato de trabajo de duración indefinida, Trabajador independiente o empresario sin asalariados, Ayuda en..."


In [891]:
no_respuesta('PR_SIT_PR')

[ 1.  2.  3.  4.  5.  6. nan]


## 'PR_JOR'  

(Solo para quienes **declaran haber trabajado** después de haber terminado sus estudios -respuesta 1 en 'HA_TRAB' o 'ha_trab'-) 

Tipo de **jornada de trabajo** en su **primer empleo**.

In [892]:
df['PR_JOR'].unique()

array(['1', '2', ' '], dtype=object)

In [893]:
traduce('PR_JOR')

                aux_traduce
                     unique
PR_JOR                     
                      [nan]
1        [A tiempo parcial]
2       [A tiempo completo]


In [894]:
df[['PR_JOR_','ha_trab_']].groupby('ha_trab_').agg(['unique']) # Inspección

,PR_JOR_
,unique
ha_trab_,
No,[nan]
Sí,"[A tiempo parcial, A tiempo completo]"


In [895]:
no_respuesta('PR_JOR')

[ 1.  2. nan]


## 'PR_NIVEL'  

(Solo para quienes **declaran haber trabajado** después de haber terminado sus estudios -respuesta 1 en 'HA_TRAB' o 'ha_trab'-) 

**Nivel de formación** más apropiado para su **primer empleo**.

In [896]:
df['PR_NIVEL'].unique()
reset('PR_NIVEL')

In [897]:
traduce('PR_NIVEL')

                                                    aux_traduce
                                                         unique
PR_NIVEL                                                       
                                                 [No aplicable]
1                                                   [Doctorado]
2                                                      [Máster]
3           [Título universitario (excepto doctorado o máster)]
4                     [Formación profesional de grado superior]
5         [Formación profesional de grado medio/ bachillerato ]
6                                [Formación profesional básica]
7                                              [ESO, EGB, etc.]
9                                                       [NS/NC]


In [898]:
df[['PR_NIVEL','ha_trab_','sit_lab_']].groupby(['ha_trab_','sit_lab_']).agg(['unique']) # Inspección

PR_NIVEL
                                         unique
ha_trab_ sit_lab_                              
No       En desempleo                       [ ]
         Inactivo                           [ ]
Sí       En desempleo  [7, 3, 5, 4, 6, 2, 9, 1]
         Inactivo      [3, 4, 5, 6, 7, 9, 2, 1]
         Trabajando    [3, 4, 7, 2, 5, 6, 9, 1]

In [899]:
no_respuesta('PR_NIVEL')

[ 1.  2.  3.  4.  5.  6.  7. nan]


## 'PR_AREA'  

(Solo para quienes **declaran haber trabajado** después de haber terminado sus estudios -respuesta 1 en 'HA_TRAB' o 'ha_trab'-) 

**Área de estudio** más apropiada para su **primer empleo**.

In [900]:
df['PR_AREA'].unique()

array(['2', '3', '4', '1', ' ', '9'], dtype=object)

In [901]:
traduce('PR_AREA')

                                         aux_traduce
                                              unique
PR_AREA                                             
                                     [No aplicable ]
1        [Exclusivamente su propia área de estudios]
2              [Su propia área o alguna relacionada]
3                     [Un área totalmente diferente]
4                        [Ningún área en particular]
9                                            [NS/NC]


In [902]:
df[['PR_AREA_','ha_trab_']].groupby('ha_trab_').agg(['unique']) # Inspección

,PR_AREA_
,unique
ha_trab_,
No,[No aplicable ]
Sí,"[Su propia área o alguna relacionada, Un área totalmente diferente, Ningún área en particular, Exclusivamente su propia área de estudios, NS/NC]"


In [903]:
no_respuesta('PR_AREA')

[ 1.  2.  3.  4. nan]


## 'PR_CONOC'  

(Solo para quienes **declaran haber trabajado** después de haber terminado sus estudios -respuesta 1 en 'HA_TRAB' o 'ha_trab'-) 

¿Se hacía uso en su **primer empleo** de los **conocimientos y habilidades** adquiridos en estos estudios?

In [904]:
df['PR_CONOC'].unique()

array(['1', '2', ' ', '9'], dtype=object)

In [905]:
traduce('PR_CONOC')

             aux_traduce
                  unique
PR_CONOC                
          [No aplicable]
1                   [Sí]
2                   [No]
9                [NS/NC]


In [906]:
df[['PR_CONOC_','ha_trab_']].groupby('ha_trab_').agg(['unique']) # Inspección

,PR_CONOC_
,unique
ha_trab_,
No,[No aplicable]
Sí,"[Sí, No, NS/NC]"


In [907]:
no_respuesta('PR_CONOC')

[ 1.  2. nan]


## 'PR_SUELDO'  (redefinida como 'pr_sueldo')

(Solo para quienes **declaran haber trabajado** después de haber terminado sus estudios -respuesta 1 en 'HA_TRAB' o 'ha_trab'-) 

**Sueldo mensual neto del primer trabajo** o de su empleo actual en el momento en el que empezó.

In [908]:
df['PR_SUELDO'].unique()

array(['3', '2', '1', '4', ' ', '9', '5', '7', '6'], dtype=object)

In [909]:
traduce('PR_SUELDO')

                            aux_traduce
                                 unique
PR_SUELDO                              
                         [No aplicable]
1                  [Menos de 700 euros]
2                  [De 700 a 999 euros]
3              [De 1.000 a 1.499 euros]
4              [De 1.500 a 1.999 euros]
5              [De 2.000 a 2.499 euros]
6              [De 2.500 a 2.999 euros]
7          [De 3.000 euros en adelante]
9                               [NS/NC]


In [910]:
df[['PR_SUELDO','ha_trab_']].groupby('ha_trab_').agg(['unique']) # Inspección

,PR_SUELDO
,unique
ha_trab_,
No,[ ]
Sí,"[3, 2, 1, 4, 9, 5, 7, 6]"


In [911]:
no_respuesta('PR_SUELDO')

[ 1.  2.  3.  4.  5.  6.  7. nan]


In [912]:
redefine('PR_SUELDO','pr_sueldo')

In [913]:
proporciones('pr_sueldo_',dropna=True)

Menos de 700 euros            0.326339
De 1.000 a 1.499 euros        0.268203
De 700 a 999 euros            0.240678
De 1.500 a 1.999 euros        0.121797
De 2.000 a 2.499 euros        0.028915
De 2.500 a 2.999 euros        0.007051
De 3.000 euros en adelante    0.007017
Name: pr_sueldo_, dtype: float64

In [914]:
dist_condicionada('pr_sueldo_','fem',dropna=True)

In [915]:
dist_condicionada('pr_sueldo_','disca',dropna=True)

## 'HATR_TIEMP'  

(Solo para quienes **declaran haber trabajado** después de haber terminado sus estudios -respuesta 1 en 'HA_TRAB' o 'ha_trab'-) 

**Tiempo que ha estado trabajando** desde que terminó sus estudios. 

In [916]:
df['HATR_TIEMP'].unique()

array(['5', '3', '4', '2', '1', ' ', '9'], dtype=object)

In [917]:
traduce('HATR_TIEMP')

                            aux_traduce
                                 unique
HATR_TIEMP                             
                         [No aplicable]
1                    [Menos de 6 meses]
2                  [De 6 meses a 1 año]
3              [De 1 año a año y medio]
4           [De 1 año y medio a 2 años]
5                        [2 o más años]
9                               [NS/NC]


In [918]:
df[['HATR_TIEMP','ha_trab_']].groupby('ha_trab_').agg(['unique']) # Inspección

,HATR_TIEMP
,unique
ha_trab_,
No,[ ]
Sí,"[5, 3, 4, 2, 1, 9]"


In [919]:
no_respuesta('HATR_TIEMP')

[ 1.  2.  3.  4.  5. nan]


## 'HATR_NUM'  

(Solo para quienes **declaran haber trabajado** después de haber terminado sus estudios -respuesta 1 en 'HA_TRAB' o 'ha_trab'-) 

**Número de empleadores distintos** que ha tenido.

In [920]:
df['HATR_NUM'].unique()

array(['03', '06', '08', '02', '07', '05', '04', '09', '11', '01', '  ',
       '00', '10', '99'], dtype=object)

In [921]:
traduce('HATR_NUM')

             aux_traduce
                  unique
HATR_NUM                
          [No aplicable]
00                  [00]
01                  [01]
02                  [02]
03                  [03]
04                  [04]
05                  [05]
06                  [06]
07                  [07]
08                  [08]
09                  [09]
10                  [10]
11            [11 o más]
99               [NS/NC]


In [922]:
df[['HATR_NUM','ha_trab_']].groupby('ha_trab_').agg(['unique']) # Inspección

,HATR_NUM
,unique
ha_trab_,
No,[ ]
Sí,"[03, 06, 08, 02, 07, 05, 04, 09, 11, 01, 00, 10, 99]"


In [923]:
no_respuesta('HATR_NUM',blank='  ',nsnc='99')

[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. nan]


## 'HATR_E27'  (redefinida como 'utilidad')

(Solo para quienes **declaran haber trabajado** después de haber terminado sus estudios -respuesta 1 en 'HA_TRAB' o 'ha_trab'-) 

¿Cree que el título universitario le ha servido para encontrar trabajo?

In [924]:
df['HATR_E27'].unique()

array(['1', '2', ' ', '9'], dtype=object)

In [925]:
traduce('HATR_E27')

             aux_traduce
                  unique
HATR_E27                
          [No aplicable]
1                   [Sí]
2                   [No]
9                [NS/NC]


In [926]:
df[['HATR_E27_','ha_trab_']].groupby('ha_trab_').agg(['unique']) # Inspección

,HATR_E27_
,unique
ha_trab_,
No,[No aplicable]
Sí,"[Sí, No, NS/NC]"


In [927]:
no_respuesta('HATR_E27')

[ 1.  2. nan]


In [928]:
redefine('HATR_E27','utilidad',si_no=True)

         aux_traduce
              unique
utilidad            
0.0             [No]
1.0             [Sí]


In [929]:
proporciones('utilidad_')

Sí     0.695176
No     0.275157
NaN    0.029667
Name: utilidad_, dtype: float64

## 'NOTRAB_BUS'  (completa la variable 'busqueda_tr')

(Solo para quienes **declaran no haber trabajado** después de haber terminado sus estudios -respuesta 2 en 'HA_TRAB' o 0 en 'ha_trab'-) 

¿**Ha buscado empleo** alguna vez?

Nota: Se usa esta información para completar la recogida en la variable 'busqueda_tr'.

In [930]:
df['NOTRAB_BUS'].unique()
reset('NOTRAB_BUS')

In [931]:
traduce('NOTRAB_BUS')

               aux_traduce
                    unique
NOTRAB_BUS                
            [No aplicable]
1                     [Sí]
2                     [No]
9                  [NS/NC]


In [932]:
df[['NOTRAB_BUS_','ha_trab_']].groupby('ha_trab_').agg(['unique']) # Inspección

,NOTRAB_BUS_
,unique
ha_trab_,
No,"[Sí, No, NS/NC]"
Sí,[No aplicable]


In [933]:
df[['NOTRAB_BUS_','busqueda_tr_','sit_lab']][df['sit_lab']==3].groupby(['busqueda_tr_','sit_lab'],
                                                                       dropna=False).agg(['unique'])
# No contradice la información facilitada por desempleados en la variable 'TRINAC_D4' (recogida en 'busqueda_tr')

,,NOTRAB_BUS_
,,unique
busqueda_tr_,sit_lab,
No,3.0,"[No, No aplicable]"
Sí,3.0,"[No aplicable, Sí]"
NaN,3.0,"[No aplicable, NS/NC]"


In [934]:
df[['NOTRAB_BUS_','sit_lab_']].groupby('sit_lab_',dropna=False).agg(['unique'])
# Sin información para quienes han trabajado después de terminar los estudios (incluye todos los ocupados y ...
# ... los desempleados que sí han trabajado).

,NOTRAB_BUS_
,unique
sit_lab_,
En desempleo,"[No aplicable, Sí]"
Inactivo,"[No aplicable, Sí, No, NS/NC]"
Trabajando,[No aplicable]


In [935]:
no_respuesta('NOTRAB_BUS')

[ 1.  2. nan]


In [936]:
# Se usa esta información para actualizar la variable 'busqueda_tr':
df['busqueda_tr'].mask(df['NOTRAB_BUS']==1,1,inplace=True)
df['busqueda_tr'].mask(df['NOTRAB_BUS']==2,0,inplace=True)
traduce('busqueda_tr')
df['busqueda_tr'][df['NOTRAB_BUS']==1].unique() # Comprobación

            aux_traduce
                 unique
busqueda_tr            
0.0                [No]
1.0                [Sí]


array([1.])

In [937]:
df['busqueda_tr'][df['NOTRAB_BUS']==2].unique() # Comprobación 

array([0.])

## 'NOTRAB_BUSM'  

(Solo para quienes **declaran haber buscado empleo** -respuesta 1 en 'NOTRAB_BUS'-. Implica que no han trabajado después de haber terminado los estudios) 

Momento en que empezó a buscar trabajo. 

In [938]:
df['NOTRAB_BUSM'].unique()

array([' ', '2', '1', '9'], dtype=object)

In [939]:
traduce('NOTRAB_BUSM')

                                                                        aux_traduce
                                                                             unique
NOTRAB_BUSM                                                                        
                                                                     [No aplicable]
1              [Antes de acabar los estudios de la titulación que finalizó en 2014]
2            [Después de acabar los estudios de la titulación que finalizó en 2014]
9                                                                           [NS/NC]


In [940]:
df[['NOTRAB_BUSM_','NOTRAB_BUS_','sit_lab_']].groupby(['NOTRAB_BUS_','sit_lab_'],
                                                      dropna=False).agg(['unique']) # Inspección
# Sin información para quienes declaran haber trabajado después de terminar sus estudios.

NOTRAB_BUSM_
                                                                                                                                                                     unique
NOTRAB_BUS_ sit_lab_                                                                                                                                                       
No          Inactivo                                                                                                                                         [No aplicable]
Sí          En desempleo  [Antes de acabar los estudios de la titulación que finalizó en 2014, Después de acabar los estudios de la titulación que finalizó en 2014, NS/NC]
            Inactivo             [Después de acabar los estudios de la titulación que finalizó en 2014, Antes de acabar los estudios de la titulación que finalizó en 2014]
NaN         En desempleo                                                                                                                                     [No aplicable]
            Inactivo                                                                                                                                         [No aplicable]
            Trabajando                                                                                                                                       [No aplicable]

In [941]:
no_respuesta('NOTRAB_BUSM')

[ 1.  2. nan]


## 'NOTRAB_BUST'  

(Solo para quienes **declaran haber buscado empleo** -respuesta 1 en 'NOTRAB_BUS'-. Implica que no han trabajado después de terminar los estudios) 

**Tiempo que lleva buscando trabajo**. 

In [942]:
df['NOTRAB_BUST'].unique()

array([' ', '6', '4', '3', '5', '1', '9', '2'], dtype=object)

In [943]:
traduce('NOTRAB_BUST')

                               aux_traduce
                                    unique
NOTRAB_BUST                               
                            [No aplicable]
1                    [Menos de tres meses]
2                         [De 3 a 6 meses]
3                     [De 6 meses a 1 año]
4               [De 1 año a año y medio  ]
5            [De 1 año y medio a  2 años ]
6                           [2 ó más años]
9                                  [NS/NC]


In [944]:
df[['NOTRAB_BUST_','NOTRAB_BUS_','sit_lab_']].groupby(['NOTRAB_BUS_','sit_lab_'],
                                                      dropna=False).agg(['unique']) # Inspección

NOTRAB_BUST_
                                                                                                                                                         unique
NOTRAB_BUS_ sit_lab_                                                                                                                                           
No          Inactivo                                                                                                                             [No aplicable]
Sí          En desempleo  [2 ó más años, De 1 año a año y medio  , Menos de tres meses, De 3 a 6 meses, De 6 meses a 1 año, De 1 año y medio a  2 años , NS/NC]
            Inactivo      [2 ó más años, De 1 año a año y medio  , De 6 meses a 1 año, De 1 año y medio a  2 años , NS/NC, De 3 a 6 meses, Menos de tres meses]
NaN         En desempleo                                                                                                                         [No aplicable]
            Inactivo                                                                                                                             [No aplicable]
            Trabajando                                                                                                                           [No aplicable]

In [945]:
no_respuesta('NOTRAB_BUST')

[ 1.  2.  3.  4.  5.  6. nan]


## 'NOTRAB_BSQ1'  (completa la información de la variable 'btr_internet')

(Solo para quienes **declaran haber buscado empleo** -respuesta 1 en 'NOTRAB_BUS'-. Implica que no han trabajado después de terminar los estudios) 

¿Ha buscado trabajo a través de anuncios en periódicos, **internet**, etc.?

Nota: 'NOTRAB_BSQ1' recoge la misma información que la variable 'TRPARA_D7_1' definida para desempleados (redefinida como 'btr_internet'). 'NOTRAB_BSQ1' está definida para quienes no han trabajado y sí han buscado empleo. Esto incluye algunos desempleados. Se ha comprobado que la información de estas dos variables no es contradictoria. 

In [946]:
df['NOTRAB_BSQ1'].unique()

array([' ', '1', '2', '9'], dtype=object)

In [947]:
traduce('NOTRAB_BSQ1')

                aux_traduce
                     unique
NOTRAB_BSQ1                
             [No aplicable]
1                      [Sí]
2                      [No]
9                   [NS/NC]


In [948]:
df[['NOTRAB_BSQ1_','NOTRAB_BUS_','ha_trab_']].groupby('NOTRAB_BUS_',dropna=False).agg(['unique']) # Inspección
# Solo se ha preguntado a quienes declaran no haber trabajado después de terminar sus estudios y haber ...
# ... buscado empleo. 

,NOTRAB_BSQ1_,ha_trab_
,unique,unique
NOTRAB_BUS_,,
No,[No aplicable],[No]
Sí,"[Sí, No, NS/NC]",[No]
NaN,[No aplicable],"[Sí, No]"


In [949]:
df[['NOTRAB_BSQ1','btr_internet_','sit_lab_']][df['sit_lab']==2].groupby(['NOTRAB_BSQ1','sit_lab_'],
                                                                         dropna=False).agg(['unique']) 
# No contradice lo declarado por los desempleados en la variable 'TRPARA_D7_1' (redefinida como 'btr_internet')

,,btr_internet_
,,unique
NOTRAB_BSQ1,sit_lab_,
,En desempleo,"[Sí, No, nan]"
1,En desempleo,[Sí]
2,En desempleo,[No]
9,En desempleo,[nan]


In [950]:
L_btr=list()

In [951]:
def actualiza_btr(col,btr):
    """ Actualiza las variables con prefijo btr_ definidas para las distintas estrategias de búsqueda de empleo, 
    añadiendo las respuestas declaradas por quienes no han trabajado y sí han buscado empleo.
    """
    no_respuesta(col)
    df[btr].mask(df[col]==1,1,inplace=True)
    df[btr].mask(df[col]==2,0,inplace=True)
    traduce(btr)
    print (df[[str(col)+'_',str(btr)+'_']][df['NOTRAB_BUS']==1].groupby([str(col)+'_'],
                                                                        dropna=False).agg(['unique']))
    if str(btr)+'_' not in L_btr:
        L_btr.append(str(btr)+'_')
    return

In [952]:
actualiza_btr('NOTRAB_BSQ1','btr_internet')

[ 1.  2. nan]
             aux_traduce
                  unique
btr_internet            
0.0                 [No]
1.0                 [Sí]
             btr_internet_
                    unique
NOTRAB_BSQ1_              
No                    [No]
Sí                    [Sí]
NaN                  [nan]


## 'NOTRAB_BSQ2'  (completa la información de la variable 'btr_spe')

(Solo para quienes **declaran haber buscado empleo** -respuesta 1 en 'NOTRAB_BUS'-. Implica que no han trabajado después de terminar los estudios) 

¿Ha buscado trabajo a través de los **servicios públicos de empleo**?

Nota: 'NOTRAB_BSQ2' recoge la misma información que la variable 'TRPARA_D7_2' definida para desempleados (redefinida como 'btr_sep'). 

In [953]:
df['NOTRAB_BSQ2'].unique()

array([' ', '1', '2', '9'], dtype=object)

In [954]:
traduce('NOTRAB_BSQ2')

                aux_traduce
                     unique
NOTRAB_BSQ2                
             [No aplicable]
1                      [Sí]
2                      [No]
9                   [NS/NC]


In [955]:
actualiza_btr('NOTRAB_BSQ2','btr_spe')

[ 1.  2. nan]
        aux_traduce
             unique
btr_spe            
0.0            [No]
1.0            [Sí]
             btr_spe_
               unique
NOTRAB_BSQ2_         
No               [No]
Sí               [Sí]
NaN             [nan]


## 'NOTRAB_BSQ3'  (completa la información de la variable 'btr_univ')

(Solo para quienes **declaran haber buscado empleo** -respuesta 1 en 'NOTRAB_BUS'-. Implica que no han trabajado después de terminar los estudios) 

¿Ha buscado trabajo a través de los **servicios de empleo de la universidad** (bolsa de trabajo, asociaciones de exalumnos, etc.)?

Nota: 'NOTRAB_BSQ3' recoge la misma información que la variable 'TRPARA_D7_3' definida para desempleados (redefinida como 'btr_univ'). 

In [956]:
df['NOTRAB_BSQ3'].unique()

array([' ', '2', '1', '9'], dtype=object)

In [957]:
traduce('NOTRAB_BSQ3')

                aux_traduce
                     unique
NOTRAB_BSQ3                
             [No aplicable]
1                      [Sí]
2                      [No]
9                   [NS/NC]


In [958]:
actualiza_btr('NOTRAB_BSQ3','btr_univ')

[ 1.  2. nan]
         aux_traduce
              unique
btr_univ            
0.0             [No]
1.0             [Sí]
             btr_univ_
                unique
NOTRAB_BSQ3_          
No                [No]
Sí                [Sí]
NaN              [nan]


## 'NOTRAB_BSQ4'  (completa la información de la variable 'btr_ett')

(Solo para quienes **declaran haber buscado empleo** -respuesta 1 en 'NOTRAB_BUS'-. Implica que no han trabajado después de terminar los estudios) 

¿Ha buscado trabajo a través de **empresas de trabajo temporal**?

Nota: 'NOTRAB_BSQ4' recoge la misma información que la variable 'TRPARA_D7_4' definida para desempleados (redefinida como 'btr_ett'). 

In [959]:
df['NOTRAB_BSQ4'].unique()

array([' ', '2', '1', '9'], dtype=object)

In [960]:
traduce('NOTRAB_BSQ4')

                aux_traduce
                     unique
NOTRAB_BSQ4                
             [No aplicable]
1                      [Sí]
2                      [No]
9                   [NS/NC]


In [961]:
actualiza_btr('NOTRAB_BSQ4','btr_ett')

[ 1.  2. nan]
        aux_traduce
             unique
btr_ett            
0.0            [No]
1.0            [Sí]
             btr_ett_
               unique
NOTRAB_BSQ4_         
No               [No]
Sí               [Sí]
NaN             [nan]


## 'NOTRAB_BSQ5'  (completa la información de la variable 'btr_bolsa')

(Solo para quienes **declaran haber buscado empleo** -respuesta 1 en 'NOTRAB_BUS'-. Implica que no han trabajado después de terminar los estudios) 

¿Ha buscado trabajo a través de una **bolsa de empleo**?

Nota: 'NOTRAB_BSQ5' recoge la misma información que la variable 'TRPARA_D7_5' definida para desempleados (redefinida como 'btr_bolsa'). 

In [962]:
df['NOTRAB_BSQ5'].unique()

array([' ', '2', '1', '9'], dtype=object)

In [963]:
traduce('NOTRAB_BSQ5')

                aux_traduce
                     unique
NOTRAB_BSQ5                
             [No aplicable]
1                      [Sí]
2                      [No]
9                   [NS/NC]


In [964]:
actualiza_btr('NOTRAB_BSQ5','btr_bolsa')

[ 1.  2. nan]
          aux_traduce
               unique
btr_bolsa            
0.0              [No]
1.0              [Sí]
             btr_bolsa_
                 unique
NOTRAB_BSQ5_           
No                 [No]
Sí                 [Sí]
NaN               [nan]


## 'NOTRAB_BSQ6'  (completa la información de la variable 'btr_contactos')

(Solo para quienes **declaran haber buscado empleo** -respuesta 1 en 'NOTRAB_BUS'-. Implica que no han trabajado después de terminar los estudios) 

¿Ha buscado trabajo buscando ayuda entre profesionales conocidos del sector o **contactos personales** (familia, amigos)?

Nota: 'NOTRAB_BSQ6' recoge la misma información que la variable 'TRPARA_D7_6' definida para desempleados (redefinida como 'btr_contactos'). 

In [965]:
df['NOTRAB_BSQ6'].unique()

array([' ', '1', '2', '9'], dtype=object)

In [966]:
traduce('NOTRAB_BSQ6')

                aux_traduce
                     unique
NOTRAB_BSQ6                
             [No aplicable]
1                      [Sí]
2                      [No]
9                   [NS/NC]


In [967]:
actualiza_btr('NOTRAB_BSQ6','btr_contactos')

[ 1.  2. nan]
              aux_traduce
                   unique
btr_contactos            
0.0                  [No]
1.0                  [Sí]
             btr_contactos_
                     unique
NOTRAB_BSQ6_               
No                     [No]
Sí                     [Sí]
NaN                   [nan]


## 'NOTRAB_BSQ7'  (completa la información de la variable 'btr_empresas')

(Solo para quienes **declaran haber buscado empleo** -respuesta 1 en 'NOTRAB_BUS'-. Implica que no han trabajado después de terminar los estudios) 

¿Ha buscado trabajo **contactando directamente con las empresas**?  

Nota: 'NOTRAB_BSQ7' recoge la misma información que la variable 'TRPARA_D7_7' definida para desempleados (redefinida como 'btr_empresas'). 

In [968]:
df['NOTRAB_BSQ7'].unique()

array([' ', '1', '2', '9'], dtype=object)

In [969]:
traduce('NOTRAB_BSQ7')

                aux_traduce
                     unique
NOTRAB_BSQ7                
             [No aplicable]
1                      [Sí]
2                      [No]
9                   [NS/NC]


In [970]:
actualiza_btr('NOTRAB_BSQ7','btr_empresas')

[ 1.  2. nan]
             aux_traduce
                  unique
btr_empresas            
0.0                 [No]
1.0                 [Sí]
             btr_empresas_
                    unique
NOTRAB_BSQ7_              
No                    [No]
Sí                    [Sí]
NaN                  [nan]


## 'NOTRAB_BSQ8'  (completa la información de la variable 'btr_opo')

(Solo para quienes **declaran haber buscado empleo** -respuesta 1 en 'NOTRAB_BUS'-. Implica que no han trabajado después de terminar los estudios) 

¿Ha buscado trabajo **preparando unas oposiciones**?  

Nota: 'NOTRAB_BSQ8' recoge la misma información que la variable 'TRPARA_D7_8' definida para desempleados (redefinida como 'btr_opo'). 

In [971]:
df['NOTRAB_BSQ8'].unique()

array([' ', '2', '1', '9'], dtype=object)

In [972]:
traduce('NOTRAB_BSQ8')

                aux_traduce
                     unique
NOTRAB_BSQ8                
             [No aplicable]
1                      [Sí]
2                      [No]
9                   [NS/NC]


In [973]:
actualiza_btr('NOTRAB_BSQ8','btr_opo')

[ 1.  2. nan]
        aux_traduce
             unique
btr_opo            
0.0            [No]
1.0            [Sí]
             btr_opo_
               unique
NOTRAB_BSQ8_         
No               [No]
Sí               [Sí]
NaN             [nan]


## 'NOTRAB_BSQ9'  (completa la información de la variable 'btr_auto')

(Solo para quienes **declaran haber buscado empleo** -respuesta 1 en 'NOTRAB_BUS'-. Implica que no han trabajado después de terminar los estudios) 

¿Ha buscado trabajo buscando equipamiento para establecer un **negocio propio**?  

Nota: 'NOTRAB_BSQ9' recoge la misma información que la variable 'TRPARA_D7_9' definida para desempleados (redefinida como 'btr_auto'). 

In [974]:
df['NOTRAB_BSQ9'].unique()

array([' ', '2', '1', '9'], dtype=object)

In [975]:
traduce('NOTRAB_BSQ9')

                aux_traduce
                     unique
NOTRAB_BSQ9                
             [No aplicable]
1                      [Sí]
2                      [No]
9                   [NS/NC]


In [976]:
actualiza_btr('NOTRAB_BSQ9','btr_auto')

[ 1.  2. nan]
         aux_traduce
              unique
btr_auto            
0.0             [No]
1.0             [Sí]
             btr_auto_
                unique
NOTRAB_BSQ9_          
No                [No]
Sí                [Sí]
NaN              [nan]


## 'NOTRAB_BSQ10'  (completa la información de la variable 'btr_otro')

(Solo para quienes **declaran haber buscado empleo** -respuesta 1 en 'NOTRAB_BUS'-. Implica que no han trabajado después de terminar los estudios) 

¿Ha buscado trabajo de **otro modo**?  

Nota: 'NOTRAB_BSQ10' recoge la misma información que la variable 'TRPARA_D7_10' definida para desempleados (redefinida como 'btr_otro'). 

In [977]:
df['NOTRAB_BSQ10'].unique()

array([' ', '2', '1', '9'], dtype=object)

In [978]:
traduce('NOTRAB_BSQ10')

                 aux_traduce
                      unique
NOTRAB_BSQ10                
              [No aplicable]
1                       [Sí]
2                       [No]
9                    [NS/NC]


In [979]:
actualiza_btr('NOTRAB_BSQ10','btr_otro')

[ 1.  2. nan]
         aux_traduce
              unique
btr_otro            
0.0             [No]
1.0             [Sí]
              btr_otro_
                 unique
NOTRAB_BSQ10_          
No                 [No]
Sí                 [Sí]
NaN               [nan]


In [980]:
# Distribución de esstrategias de búsqueda de empleo entre quienes no han trabajado: 
df[L_btr].apply(lambda x: x.value_counts(normalize=True))

,btr_internet_,btr_spe_,btr_univ_,btr_ett_,btr_bolsa_,btr_contactos_,btr_empresas_,btr_opo_,btr_auto_,btr_otro_
No,0.034062,0.188106,0.304149,0.410629,0.277693,0.067747,0.421093,0.21448,0.580045,0.189658
Sí,0.965938,0.811894,0.695851,0.589371,0.722307,0.932253,0.578907,0.78552,0.419955,0.810342


## 'NOTRAB_DIF1'  (completa la información de la variable 'dbtr_formacion')

(Solo para quienes **declaran haber buscado empleo** -respuesta 1 en 'NOTRAB_BUS'-. Implica que no han trabajado después de terminar los estudios) 

(**Dificultades para encontrar empleo**) Importancia que atribuye a las **carencias en la formación universitaria recibida** en cuanto a sus dificultades para encontrar trabajo.

Nota: 'NOTRAB_DIF1' recoge la misma información que la variable 'TRPARA_D8_1' definida para desempleados (redefinida como 'dbtr_formacion'). 

In [981]:
df['NOTRAB_DIF1'].unique()

array([' ', '2', '1', '4', '9', '3'], dtype=object)

In [982]:
traduce('NOTRAB_DIF1')

                       aux_traduce
                            unique
NOTRAB_DIF1                       
                    [No aplicable]
1                [Nada importante]
2                [Poco importante]
3            [Bastante importante]
4                 [Muy importante]
9                          [NS/NC]


In [983]:
df[['NOTRAB_DIF1_','NOTRAB_BUS_']].groupby('NOTRAB_BUS_',dropna=False).agg(['unique']) # Inspección

,NOTRAB_DIF1_
,unique
NOTRAB_BUS_,
No,[No aplicable]
Sí,"[Poco importante, Nada importante, Muy importante, NS/NC, Bastante importante]"
NaN,[No aplicable]


In [984]:
df[['NOTRAB_DIF1_','dbtr_formacion_','sit_lab_']][df['sit_lab']==2].groupby('NOTRAB_DIF1_',
                                                                            dropna=False).agg(['unique']) 
# No hay contradicción con lo declarado en 'TRPARA_D8_1' (o 'dbtr_formacion') por los desempleados.

,dbtr_formacion_,sit_lab_
,unique,unique
NOTRAB_DIF1_,,
Bastante importante,[Bastante importante],[En desempleo]
Muy importante,[Muy importante],[En desempleo]
NS/NC,[nan],[En desempleo]
Nada importante,[Nada importante],[En desempleo]
No aplicable,"[Muy importante, Poco importante, Bastante importante, Nada importante, nan]",[En desempleo]
Poco importante,[Poco importante],[En desempleo]


In [985]:
L_dbtr=list()

In [986]:
def actualiza_dbtr(col,dbtr):
    """Actualiza las variables con prefijo 'dbtr_' (definidas originalmente para desempleados), añadiendo la 
    información que facilitan quienes declaran no haber trabajado desde que terminaron sus estudios y haber 
    buscado trabajo. 
    """
    no_respuesta(col)
    cond=(df[col]==1)|(df[col]==2)|(df[col]==3)|(df[col]==4)
    df[dbtr].mask(cond,df[col],inplace=True)
    traduce(dbtr)
    print (df[[str(col)+'_',str(dbtr)+'_']][df['NOTRAB_BUS']==1].groupby(str(col)+'_',
                                                                         dropna=False).agg(['unique']))
    if str(dbtr)+'_' not in L_dbtr: 
        L_dbtr.append(str(dbtr)+'_')
    return

In [987]:
actualiza_dbtr('NOTRAB_DIF1','dbtr_formacion')

[ 1.  2.  3.  4. nan]
                          aux_traduce
                               unique
dbtr_formacion                       
1.0                 [Nada importante]
2.0                 [Poco importante]
3.0             [Bastante importante]
4.0                  [Muy importante]
                           dbtr_formacion_
                                    unique
NOTRAB_DIF1_                              
Bastante importante  [Bastante importante]
Muy importante            [Muy importante]
Nada importante          [Nada importante]
Poco importante          [Poco importante]
NaN                                  [nan]


## 'NOTRAB_DIF2'  (completa la información de la variable 'dbtr_personales')

(Solo para quienes **declaran haber buscado empleo** -respuesta 1 en 'NOTRAB_BUS'-. Implica que no han trabajado después de terminar los estudios) 

(**Dificultades para encontrar empleo**) Importancia que atribuye a las **actividades personales** que impiden trabajar (seguir estudiando, familia, otras) en cuanto a sus dificultades para encontrar trabajo.

Nota: 'NOTRAB_DIF2' recoge la misma información que la variable 'TRPARA_D8_2' definida para desempleados (redefinida como 'dbtr_personales'). 

In [988]:
df['NOTRAB_DIF2'].unique()

array([' ', '2', '1', '4', '3', '9'], dtype=object)

In [989]:
traduce('NOTRAB_DIF2')

                       aux_traduce
                            unique
NOTRAB_DIF2                       
                    [No aplicable]
1                [Nada importante]
2                [Poco importante]
3            [Bastante importante]
4                 [Muy importante]
9                          [NS/NC]


In [990]:
actualiza_dbtr('NOTRAB_DIF2','dbtr_personales')

[ 1.  2.  3.  4. nan]
                           aux_traduce
                                unique
dbtr_personales                       
1.0                  [Nada importante]
2.0                  [Poco importante]
3.0              [Bastante importante]
4.0                   [Muy importante]
                          dbtr_personales_
                                    unique
NOTRAB_DIF2_                              
Bastante importante  [Bastante importante]
Muy importante            [Muy importante]
Nada importante          [Nada importante]
Poco importante          [Poco importante]
NaN                                  [nan]


## 'NOTRAB_DIF3'  (completa la información de la variable 'dbtr_experiencia')

(Solo para quienes **declaran haber buscado empleo** -respuesta 1 en 'NOTRAB_BUS'-. Implica que no han trabajado después de terminar los estudios) 

(**Dificultades para encontrar empleo**) Importancia que atribuye a la **falta de experiencia profesional** en cuanto a sus dificultades para encontrar trabajo.

Nota: 'NOTRAB_DIF3' recoge la misma información que la variable 'TRPARA_D8_3' definida para desempleados (redefinida como 'dbtr_experiencia'). 

In [991]:
df['NOTRAB_DIF3'].unique()

array([' ', '4', '3', '9', '1', '2'], dtype=object)

In [992]:
traduce('NOTRAB_DIF3')

                       aux_traduce
                            unique
NOTRAB_DIF3                       
                    [No aplicable]
1                [Nada importante]
2                [Poco importante]
3            [Bastante importante]
4                 [Muy importante]
9                          [NS/NC]


In [993]:
actualiza_dbtr('NOTRAB_DIF3','dbtr_experiencia')

[ 1.  2.  3.  4. nan]
                            aux_traduce
                                 unique
dbtr_experiencia                       
1.0                   [Nada importante]
2.0                   [Poco importante]
3.0               [Bastante importante]
4.0                    [Muy importante]
                         dbtr_experiencia_
                                    unique
NOTRAB_DIF3_                              
Bastante importante  [Bastante importante]
Muy importante            [Muy importante]
Nada importante          [Nada importante]
Poco importante          [Poco importante]
NaN                                  [nan]


## 'NOTRAB_DIF4'  (completa la información de la variable 'dbtr_expectativas')

(Solo para quienes **declaran haber buscado empleo** -respuesta 1 en 'NOTRAB_BUS'-. Implica que no han trabajado después de terminar los estudios) 

(**Dificultades para encontrar empleo**) Importancia que atribuye a la **exigencia de tener un trabajo acorde con sus expectativas** económicas y profesionales en cuanto a sus dificultades para encontrar trabajo.

Nota: 'NOTRAB_DIF4' recoge la misma información que la variable 'TRPARA_D8_4' definida para desempleados (redefinida como 'dbtr_formación'). 

In [994]:
df['NOTRAB_DIF4'].unique()

array([' ', '2', '1', '4', '3', '9'], dtype=object)

In [995]:
traduce('NOTRAB_DIF4')

                       aux_traduce
                            unique
NOTRAB_DIF4                       
                    [No aplicable]
1                [Nada importante]
2                [Poco importante]
3            [Bastante importante]
4                 [Muy importante]
9                          [NS/NC]


In [996]:
actualiza_dbtr('NOTRAB_DIF4','dbtr_expectativas')

[ 1.  2.  3.  4. nan]
                             aux_traduce
                                  unique
dbtr_expectativas                       
1.0                    [Nada importante]
2.0                    [Poco importante]
3.0                [Bastante importante]
4.0                     [Muy importante]
                        dbtr_expectativas_
                                    unique
NOTRAB_DIF4_                              
Bastante importante  [Bastante importante]
Muy importante            [Muy importante]
Nada importante          [Nada importante]
Poco importante          [Poco importante]
NaN                                  [nan]


## 'NOTRAB_DIF5'  (completa la información de la variable 'dbtr_idiomas')

(Solo para quienes **declaran haber buscado empleo** -respuesta 1 en 'NOTRAB_BUS'-. Implica que no han trabajado después de terminar los estudios) 

(**Dificultades para encontrar empleo**) Importancia que atribuye a la **falta de conocimientos de idiomas** en cuanto a sus dificultades para encontrar trabajo.

Nota: 'NOTRAB_DIF5' recoge la misma información que la variable 'TRPARA_D8_5' definida para desempleados (redefinida como 'dbtr_idiomas'). 

In [997]:
df['NOTRAB_DIF5'].unique()

array([' ', '2', '1', '3', '4', '9'], dtype=object)

In [998]:
traduce('NOTRAB_DIF5')

                       aux_traduce
                            unique
NOTRAB_DIF5                       
                    [No aplicable]
1                [Nada importante]
2                [Poco importante]
3            [Bastante importante]
4                 [Muy importante]
9                          [NS/NC]


In [999]:
actualiza_dbtr('NOTRAB_DIF5','dbtr_idiomas')

[ 1.  2.  3.  4. nan]
                        aux_traduce
                             unique
dbtr_idiomas                       
1.0               [Nada importante]
2.0               [Poco importante]
3.0           [Bastante importante]
4.0                [Muy importante]
                             dbtr_idiomas_
                                    unique
NOTRAB_DIF5_                              
Bastante importante  [Bastante importante]
Muy importante            [Muy importante]
Nada importante          [Nada importante]
Poco importante          [Poco importante]
NaN                                  [nan]


## 'NOTRAB_DIF6'  (completa la información de la variable 'dbtr_ti')

(Solo para quienes **declaran haber buscado empleo** -respuesta 1 en 'NOTRAB_BUS'-. Implica que no han trabajado después de terminar los estudios) 

(**Dificultades para encontrar empleo**) Importancia que atribuye a la **falta de conocimientos informáticos** en cuanto a sus dificultades para encontrar trabajo.

Nota: 'NOTRAB_DIF6' recoge la misma información que la variable 'TRPARA_D8_6' definida para desempleados (redefinida como 'dbtr_ti'). 

In [1000]:
df['NOTRAB_DIF6'].unique()

array([' ', '3', '1', '2', '4', '9'], dtype=object)

In [1001]:
traduce('NOTRAB_DIF6')

                       aux_traduce
                            unique
NOTRAB_DIF6                       
                    [No aplicable]
1                [Nada importante]
2                [Poco importante]
3            [Bastante importante]
4                 [Muy importante]
9                          [NS/NC]


In [1002]:
actualiza_dbtr('NOTRAB_DIF6','dbtr_ti')

[ 1.  2.  3.  4. nan]
                   aux_traduce
                        unique
dbtr_ti                       
1.0          [Nada importante]
2.0          [Poco importante]
3.0      [Bastante importante]
4.0           [Muy importante]
                                  dbtr_ti_
                                    unique
NOTRAB_DIF6_                              
Bastante importante  [Bastante importante]
Muy importante            [Muy importante]
Nada importante          [Nada importante]
Poco importante          [Poco importante]
NaN                                  [nan]


In [1003]:
# Distribución de la importancia que atribuyen a cada dificultad para encontrar trabajo quienes no han ...
# ... trabajado después de terminar sus estudios y han buscado trabajo:
df_dbtr=df[L_dbtr][df['NOTRAB_BUS']==1].apply(lambda x: x.value_counts(normalize=True))
# Para ordenar la salida por grado de importancia: 
df_dbtr.sort_values('dbtr_experiencia_')

,dbtr_formacion_,dbtr_personales_,dbtr_experiencia_,dbtr_expectativas_,dbtr_idiomas_,dbtr_ti_
Nada importante,0.276596,0.242105,0.057292,0.136364,0.223404,0.361930
Poco importante,0.340426,0.252632,0.070312,0.320856,0.401596,0.412869
Bastante importante,0.215426,0.242105,0.187500,0.296791,0.242021,0.168901
Muy importante,0.167553,0.263158,0.684896,0.245989,0.132979,0.056300


In [1004]:
df_dbtr.sort_values('dbtr_experiencia_').rank(ascending=False)
# La dificultad destacada como la principal para encontrar trabajo es la falta de experiencia profesional ...
# ... (la frecuencia de las respuestas disminuye a medida que reflejan menor importancia atribuida).
# Solo las circunstancias personales aparecen también como muy importantes con la frecuencia más alta, aunque ...
# con una distribución casi uniforme entre las cuatro respuestas posibles.  

,dbtr_formacion_,dbtr_personales_,dbtr_experiencia_,dbtr_expectativas_,dbtr_idiomas_,dbtr_ti_
Nada importante,2.0,3.5,4.0,4.0,3.0,2.0
Poco importante,1.0,2.0,3.0,1.0,1.0,1.0
Bastante importante,3.0,3.5,2.0,2.0,2.0,3.0
Muy importante,4.0,1.0,1.0,3.0,4.0,4.0


## 'RECHA_TR' (redefinida como 'rechazo')

¿Ha **rechazado algún trabajo** por considerarlo poco adecuado desde que se ha titulado?

In [1005]:
df['RECHA_TR'].unique() # No aparece la respuesta no aplicable: se pregunta a todos los encuestados. 

array([2, 1, 9])

In [1006]:
traduce('RECHA_TR')

         aux_traduce
              unique
RECHA_TR            
1               [Sí]
2               [No]
9            [NS/NC]


In [1007]:
no_respuesta('RECHA_TR',tipo='int')
redefine('RECHA_TR','rechazo',si_no=True)

[ 1.  2. nan]
        aux_traduce
             unique
rechazo            
0.0            [No]
1.0            [Sí]


In [1008]:
proporciones('rechazo_')

Sí     0.506840
No     0.491801
NaN    0.001359
Name: rechazo_, dtype: float64

## 'RECHA_M1'  (redefinida como 'rech_estudios')

(Solo para quienes **declaran haber rechazado una oferta de trabajo** -respuesta 1 en 'RECHA_TR' o 'rechazo'-)  

(**Motivo del rechazo**) ¿Ha rechazado una oferta de trabajo porque no le permitía **seguir estudiando**?

In [1009]:
df['RECHA_M1'].unique()

array([' ', '2', '1', '9'], dtype=object)

In [1010]:
traduce('RECHA_M1')

             aux_traduce
                  unique
RECHA_M1                
          [No aplicable]
1                   [Sí]
2                   [No]
9                [NS/NC]


In [1011]:
df[['rechazo_','RECHA_M1']].groupby('rechazo_',dropna=True).agg(['unique']) # Inspección

,RECHA_M1
,unique
rechazo_,
No,[ ]
Sí,"[2, 1, 9]"


In [1012]:
L_rechazo=list()

In [1013]:
def re_rechazo(col,nuevo):
    """ Procesa las columnas referidas a motivos de rechazo de una oferta de trabajo.
    """
    no_respuesta(col)
    redefine(col,nuevo,si_no=True)
    D_describe[nuevo]='Motivo para rechazar oferta de empleo:'+D_describe[col].split(':')[1]
    print(df[[str(col)+'_',str(nuevo)+'_']].groupby(str(col)+'_',dropna=True).agg(['unique']))
    if str(nuevo)+'_' not in L_rechazo:
        L_rechazo.append(str(nuevo)+'_')
    return

In [1014]:
re_rechazo('RECHA_M1','rech_estudios')

[ 1.  2. nan]
              aux_traduce
                   unique
rech_estudios            
0.0                  [No]
1.0                  [Sí]
          rech_estudios_
                  unique
RECHA_M1_               
No                  [No]
Sí                  [Sí]


## 'RECHA_M2'  (redefinida como 'rech_personales')

(Solo para quienes **declaran haber rechazado una oferta de trabajo** -respuesta 1 en 'RECHA_TR' o 'rechazo'-)  

(**Motivo del rechazo**) ¿Ha rechazado una oferta de trabajo porque el horario de trabajo era incompatible con **obligaciones personales o familiares**?

In [1015]:
df['RECHA_M2'].unique()

array([' ', '2', '1', '9'], dtype=object)

In [1016]:
traduce('RECHA_M2')

             aux_traduce
                  unique
RECHA_M2                
          [No aplicable]
1                   [Sí]
2                   [No]
9                [NS/NC]


In [1017]:
re_rechazo('RECHA_M2','rech_personales')

[ 1.  2. nan]
                aux_traduce
                     unique
rech_personales            
0.0                    [No]
1.0                    [Sí]
          rech_personales_
                    unique
RECHA_M2_                 
No                    [No]
Sí                    [Sí]


## 'RECHA_M3'  (redefinida como 'rech_residencia')

(Solo para quienes **declaran haber rechazado una oferta de trabajo** -respuesta 1 en 'RECHA_TR' o 'rechazo'-)  

(**Motivo del rechazo**) ¿Ha rechazado una oferta de trabajo porque aceptarlo implicaba **cambiar de lugar de residencia**?

In [1018]:
df['RECHA_M3'].unique()

array([' ', '2', '1', '9'], dtype=object)

In [1019]:
traduce('RECHA_M3')

             aux_traduce
                  unique
RECHA_M3                
          [No aplicable]
1                   [Sí]
2                   [No]
9                [NS/NC]


In [1020]:
re_rechazo('RECHA_M3','rech_residencia')

[ 1.  2. nan]
                aux_traduce
                     unique
rech_residencia            
0.0                    [No]
1.0                    [Sí]
          rech_residencia_
                    unique
RECHA_M3_                 
No                    [No]
Sí                    [Sí]


## 'RECHA_M4'  (redefinida como 'rech_distancia')

(Solo para quienes **declaran haber rechazado una oferta de trabajo** -respuesta 1 en 'RECHA_TR' o 'rechazo'-)  

(**Motivo del rechazo**) ¿Ha rechazado una oferta de trabajo porque el trayecto entre el domicilio y el puesto de trabajo no estaba a una **distancia** razonable?

In [1021]:
df['RECHA_M4'].unique()

array([' ', '2', '1', '9'], dtype=object)

In [1022]:
traduce('RECHA_M4')

             aux_traduce
                  unique
RECHA_M4                
          [No aplicable]
1                   [Sí]
2                   [No]
9                [NS/NC]


In [1023]:
re_rechazo('RECHA_M4','rech_distancia')

[ 1.  2. nan]
               aux_traduce
                    unique
rech_distancia            
0.0                   [No]
1.0                   [Sí]
          rech_distancia_
                   unique
RECHA_M4_                
No                   [No]
Sí                   [Sí]


## 'RECHA_M5'  (redefinida como 'rech_retribucion')

(Solo para quienes **declaran haber rechazado una oferta de trabajo** -respuesta 1 en 'RECHA_TR' o 'rechazo'-)  

(**Motivo del rechazo**) ¿Ha rechazado una oferta de trabajo porque consideraba que **no estaba adecuadamente remunerado**?

In [1024]:
df['RECHA_M5'].unique()

array([' ', '2', '1', '9'], dtype=object)

In [1025]:
traduce('RECHA_M5')

             aux_traduce
                  unique
RECHA_M5                
          [No aplicable]
1                   [Sí]
2                   [No]
9                [NS/NC]


In [1026]:
re_rechazo('RECHA_M5','rech_retribucion')

[ 1.  2. nan]
                 aux_traduce
                      unique
rech_retribucion            
0.0                     [No]
1.0                     [Sí]
          rech_retribucion_
                     unique
RECHA_M5_                  
No                     [No]
Sí                     [Sí]


## 'RECHA_M6'  (redefinida como 'rech_formacion')

(Solo para quienes **declaran haber rechazado una oferta de trabajo** -respuesta 1 en 'RECHA_TR' o 'rechazo'-)  

(**Motivo del rechazo**) ¿Ha rechazado una oferta de trabajo porque el nivel del puesto de trabajo **no estaba acorde con sus habilidades y conocimientos**?

In [1027]:
df['RECHA_M6'].unique()

array([' ', '2', '1', '9'], dtype=object)

In [1028]:
traduce('RECHA_M6')

             aux_traduce
                  unique
RECHA_M6                
          [No aplicable]
1                   [Sí]
2                   [No]
9                [NS/NC]


In [1029]:
re_rechazo('RECHA_M6','rech_formacion')

[ 1.  2. nan]
               aux_traduce
                    unique
rech_formacion            
0.0                   [No]
1.0                   [Sí]
          rech_formacion_
                   unique
RECHA_M6_                
No                   [No]
Sí                   [Sí]


## 'RECHA_M7'  (redefinida como 'rech_alternativa')

(Solo para quienes **declaran haber rechazado una oferta de trabajo** -respuesta 1 en 'RECHA_TR' o 'rechazo'-)  

(**Motivo del rechazo**) ¿Ha rechazado una oferta de trabajo porque tenía **otra oferta de trabajo o ya estaba trabajando**?

In [1030]:
df['RECHA_M7'].unique()

array([' ', '2', '1', '9'], dtype=object)

In [1031]:
traduce('RECHA_M7')

             aux_traduce
                  unique
RECHA_M7                
          [No aplicable]
1                   [Sí]
2                   [No]
9                [NS/NC]


In [1032]:
re_rechazo('RECHA_M7','rech_alternativa')

[ 1.  2. nan]
                 aux_traduce
                      unique
rech_alternativa            
0.0                     [No]
1.0                     [Sí]
          rech_alternativa_
                     unique
RECHA_M7_                  
No                     [No]
Sí                     [Sí]


## 'RECHA_M8'  (redefinida como 'rech_opo')

(Solo para quienes **declaran haber rechazado una oferta de trabajo** -respuesta 1 en 'RECHA_TR' o 'rechazo'-)  

(**Motivo del rechazo**) ¿Ha rechazado una oferta de trabajo porque **estaba preparando oposiciones**?

In [1033]:
df['RECHA_M8'].unique()

array([' ', '1', '2', '9'], dtype=object)

In [1034]:
traduce('RECHA_M8')

             aux_traduce
                  unique
RECHA_M8                
          [No aplicable]
1                   [Sí]
2                   [No]
9                [NS/NC]


In [1035]:
re_rechazo('RECHA_M8','rech_opo')

[ 1.  2. nan]
         aux_traduce
              unique
rech_opo            
0.0             [No]
1.0             [Sí]
          rech_opo_
             unique
RECHA_M8_          
No             [No]
Sí             [Sí]


## 'RECHA_M9'  (redefinida como 'rech_otro')

(Solo para quienes **declaran haber rechazado una oferta de trabajo** -respuesta 1 en 'RECHA_TR' o 'rechazo'-)  

(**Motivo del rechazo**) ¿Ha rechazado una oferta de trabajo por **otros motivos**?

In [1036]:
df['RECHA_M9'].unique()

array([' ', '2', '1', '9'], dtype=object)

In [1037]:
traduce('RECHA_M9')

             aux_traduce
                  unique
RECHA_M9                
          [No aplicable]
1                   [Sí]
2                   [No]
9                [NS/NC]


In [1038]:
re_rechazo('RECHA_M9','rech_otro')

[ 1.  2. nan]
          aux_traduce
               unique
rech_otro            
0.0              [No]
1.0              [Sí]
          rech_otro_
              unique
RECHA_M9_           
No              [No]
Sí              [Sí]


In [1039]:
for col in L_rechazo:
    col=str(col).strip('_')
    print (col,'\t',D_describe[col])

rech_estudios 	 Motivo para rechazar oferta de empleo: no le permitía continuar estudiando
rech_personales 	 Motivo para rechazar oferta de empleo: el horario de trabajo era incompatible con obligaciones personales o familiares
rech_residencia 	 Motivo para rechazar oferta de empleo: aceptarlo implicaba cambiar de lugar de residencia
rech_distancia 	 Motivo para rechazar oferta de empleo: el trayecto entre el domicilio y el puesto de trabajo no estaba a una distancia razonable
rech_retribucion 	 Motivo para rechazar oferta de empleo: consideraba que no estaba adecuadamente remunerado
rech_formacion 	 Motivo para rechazar oferta de empleo: el nivel del puesto de trabajo no estaba acorde con las habilidades y conocimientos
rech_alternativa 	 Motivo para rechazar oferta de empleo: tenía otra oferta de trabajo / ya estaba trabajando
rech_opo 	 Motivo para rechazar oferta de empleo: estaba preparando oposiciones
rech_otro 	 Motivo para rechazar oferta de empleo: otros motivos


In [1040]:
df[L_rechazo].apply(lambda x:x.value_counts(normalize=True))
# El motivo más frecuente para rechazar una oferta de trabajo es la remuneración insuficiente. Es el único ...
# ... identificado como razón para rechazar una oferta en más de la mitad de las respuestas. 

,rech_estudios_,rech_personales_,rech_residencia_,rech_distancia_,rech_retribucion_,rech_formacion_,rech_alternativa_,rech_opo_,rech_otro_
No,0.909732,0.840876,0.872776,0.862788,0.333292,0.65647,0.583307,0.915288,0.922779
Sí,0.090268,0.159124,0.127224,0.137212,0.666708,0.34353,0.416693,0.084712,0.077221


# BLOQUE F. INFORMACIÓN DEL HOGAR

## 'HO_F1'  (redefinida como 'e_civil')

**Estado civil**.

In [1041]:
df['HO_F1'].unique()

array([1, 2, 3, 5, 4, 9])

In [1042]:
traduce('HO_F1')

        aux_traduce
             unique
HO_F1              
1         [Soltero]
2          [Casado]
3           [Viudo]
4        [Separado]
5      [Divorciado]
9           [NS/NC]


In [1043]:
no_respuesta('HO_F1',tipo='int')
redefine('HO_F1','e_civil')

[ 1.  2.  3.  4.  5. nan]


In [1044]:
df['e_civil_'].unique()

array(['Soltero', 'Casado', 'Viudo', 'Divorciado', 'Separado', nan],
      dtype=object)

## 'HO_PAREJ'  (redefinida como 'pareja')

¿Convive en pareja?

In [1045]:
df['HO_PAREJ'].unique()

array([2, 1, 9])

In [1046]:
traduce('HO_PAREJ')

         aux_traduce
              unique
HO_PAREJ            
1               [Sí]
2               [No]
9            [NS/NC]


In [1047]:
no_respuesta('HO_PAREJ',tipo='int')
redefine('HO_PAREJ','pareja',si_no=True)

[ 1.  2. nan]
       aux_traduce
            unique
pareja            
0.0           [No]
1.0           [Sí]


## 'HO_NPER'  (redefinida como 'n_hogar')

**Número de personas que viven en el hogar**, incluyendo al entrevistado. 

In [1048]:
df['HO_NPER'].unique()

array([3, 1, 4, 2, 6, 5, 9])

In [1049]:
traduce('HO_NPER')

        aux_traduce
             unique
HO_NPER            
1               [1]
2               [2]
3               [3]
4               [4]
5               [5]
6         [6 o más]
9           [NS/NC]


In [1050]:
no_respuesta('HO_NPER',tipo='int')
redefine('HO_NPER','n_hogar')

[ 1.  2.  3.  4.  5.  6. nan]


## 'HO_NPER_TR'  (redefinida como 'n_hogar_tr')

**Número de personas en el hogar que trabajan** actualmente. 

In [1051]:
df['HO_NPER_TR'].unique()

array([1, 2, 0, 3, 9, 4, 6, 5])

In [1052]:
traduce('HO_NPER_TR')

           aux_traduce
                unique
HO_NPER_TR            
0                  [0]
1                  [1]
2                  [2]
3                  [3]
4                  [4]
5                  [5]
6            [6 o más]
9              [NS/NC]


In [1053]:
no_respuesta('HO_NPER_TR',tipo='int')
redefine('HO_NPER_TR','n_hogar_tr')

[ 0.  1.  2.  3.  4.  5.  6. nan]


## 'HO_TIPO'  

**Tipo de hogar**.

In [1054]:
df['HO_TIPO'].unique()

array([6, 1, 5, 4, 8, 3, 7, 2, 9])

In [1055]:
traduce('HO_TIPO')

                                                                                     aux_traduce
                                                                                          unique
HO_TIPO                                                                                         
1                                                                            [Hogar unipersonal]
2                             [Padre o madre solo/a que convive con algún hijo menor de 25 años]
3                        [Padre o madre solo/a que convive con todos sus hijos de 25 años o más]
4                                                    [Pareja sin hijos que convivan en el hogar]
5                            [Pareja con algún hijo menor de 25 años y que conviven en el hogar]
6           [Pareja con algún hijo menor pero todos de 25 años ó más y que conviven en el hogar]
7        [Pareja, o un padre/madre, que conviven con hijos menores de 25 años, y otras personas]
8                             

In [1056]:
no_respuesta('HO_TIPO')

[1. 2. 3. 4. 5. 6. 7. 8. 9.]


# BLOQUE VARIABLES ADICIONALES

Se omite.

## Organización de archivos csv

In [1057]:
df[v_originales+v_originales_L].to_csv('data/Graduados_19_procesado.csv')

In [1058]:
df[v_modelos+v_modelos_L].to_csv('data/Graduados_19_modelos.csv')

In [1059]:
len(v_originales),len(v_originales_L),len(v_modelos),len(v_modelos_L)

(299, 223, 135, 126)

In [1060]:
df_dict = pd.DataFrame.from_dict(D_dict, orient='columns')
df_dict.to_csv('data/D_dict.csv')

In [1061]:
df_describe = pd.DataFrame.from_dict(D_describe, orient='index',columns=['valor'])
df_describe.to_csv('data/D_describe.csv')